In [ ]:
%%capture
!sudo apt-get install tesseract-ocr
!sudo apt-get install poppler-utils --fix-missing
!pip install pytesseract
!pip install selenium
!pip install pyPDF2
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from pathlib import Path
import pandas as pd
from google.colab import drive
import pytesseract
from PIL import Image
from PyPDF2 import PdfFileMerger, PdfFileReader
import os, io, re, sys, time

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
df = pd.read_csv('output.csv', header=None)

raw_urls = df[1].apply(lambda s: s[s.find('http'):].strip()).values
scribd_urls = filter(lambda s: 'scribd' in s, raw_urls)

urls_years = [
    (s, int(re.search(r'\b(20)\d{2}\b', s).group(0))) 
        for s in scribd_urls 
        if re.search(r'\b(20)\d{2}\b', s)
]

urls = [url.replace("'", '') for url, year in sorted(urls_years, key=lambda t: t[1])]
urls = list(reversed(urls))

In [ ]:
drive.mount('/content/drive')
# Folder location of scraped PDF txt files
PDF_DIR: str = Path("./drive/MyDrive/scribd_pdfs")

Mounted at /content/drive


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
def get_text_from_scribd(url: str) -> None:

    wd.get(url)

    try:
        button = wd.find_element(By.CLASS_NAME, 'fullscreen_button')
        button.click()
    except: 
        print(
            'Fullscreen button did not work, '
            'must be something wrong with the link, or pdf '
            'might be set to private', url
        )
        return None

    try:
        s = """
        el = document.getElementsByClassName('auto__doc_page_body_toolbar')[0];
        if (el) { el.remove(); }
        """
        wd.execute_script(s)
    except: 
        print('Could not delete element from:', url)
        return

    try:
        s = """
        el = document.getElementsByClassName('nav_and_banners_fixed')[0];
        if (el) { el.remove(); }
        """
        wd.execute_script(s)
    except:
        print('Could not delete element from:', url)
        return

    try:
        s = """
        el = document.getElementsByClassName('scrolling_component')[0];
        if (el) { el.remove(); }
        """
        wd.execute_script(s)
    except:
        print('Could not delete element from:', url)
        return

    try:
        s = """
        return document.querySelectorAll('[id^=page]').length
        """
        num_pages: int
        num_pages = wd.execute_script(s)
    except:
        print('Could not get number of pages', url)
        return

    try:
        s = """
        el = document.getElementById('pdf_document_scroll_container');
        Array.from(el.children).forEach((el, i) => { if (i !== 0) {el.remove();}})
        """
        wd.execute_script(s)
    except:
        print('Could not remove footer', url)
        return
    pngs = []

    for i in range(1, num_pages+1):
        try:
            # in case scribd shows its 'sign up!' popup
            button = wd.find_element(By.CLASS_NAME, 'close_button')
            button.click()
        except:
            pass
        try:
            s = """
            el = document.getElementsByClassName('wrapper__doc_page_webpack_doc_page_body_document_useful')[0];
            if (el) { el.remove(); }
            """
            wd.execute_script(s)
        except:
            pass


        e = wd.find_element(By.ID, f'page{i}')

        s = f"""
        [...Array({num_pages})].map((_, i) => {{
            pageNum = i + 1;
            el = document.getElementById('page' + pageNum.toString());
            if (pageNum !== {i}) {{ 
                el.style.display = 'none'; 
            }}
            else {{ 
                el.style.display = 'block'; 
            }}
        }});"""
        wd.execute_script(s)

        s = f"""
        el = document.getElementById('page{i}');
        el.style.position = 'fixed';
        el.style.background = 'white';
        el.style.backgroundColor = 'white';
        el.style.zIndex = '100';
        el.style.width = '1170px';
        el.style.height = '901px';
        el.style.left = '50%';
        el.style.top = '50%';
        el.style.transform = 'scale(3) translate(-40%, -65%)';
        el.style.transformOrigin = 'left top';
        """
        wd.execute_script(s)

        time.sleep(1)

        png = wd.get_screenshot_as_png()

        imageStream = io.BytesIO(png)
        imageFile = Image.open(imageStream)

        pngs.append(imageFile)
        # imageFile = imageFile.convert('RGB')

        # img_byte_arr = io.BytesIO()
        # imageFile.save(img_byte_arr, 'PNG', quality=1000) 
        # img_byte_arr = img_byte_arr.getvalue()



        # pngs.append(img_byte_arr)

        # s = f"""
        # el = document.getElementById('page{i}');
        # el.remove();
        # """
        # wd.execute_script(s)

        time.sleep(1)

    
    return pngs

In [ ]:
pdf_files = os.listdir(PDF_DIR)

In [ ]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.set_window_size(901*3, 1170*3)

for i, url in enumerate(urls):
    if url[-1] == '/':
        url = url[:-1]
    file_name = url.split('/')[-1] + '.pdf'

    if i % 10 == 0:
        print(i)

    if file_name in pdf_files:
        continue

    pngs = get_text_from_scribd(url)

    if not pngs:
        print('Something wrong')
        continue

    merger = PdfFileMerger()

    for png in pngs:
        pdf = pytesseract.image_to_pdf_or_hocr(png, extension='pdf')
        pdf = io.BytesIO(pdf)
        merger.append(pdf)

    with open(PDF_DIR / file_name, 'wb') as f:
        merger.write(f)

    merger.close()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
Fullscreen button did not work, must be something wrong with the link, or pdf might be set to private https://www.scribd.com/document/443775237/J-C-T-AXXX-XXX-935-BIA-Dec-12-2019-pdf
Something wrong
740
750
760
770
780
790
800
Fullscreen button did not work, must be something wrong with the link, or pdf might be set to private https://www.scribd.com/document/400033416/E-T-AXXX-XXX-204-BIA-Feb-5-2019-pdf
Something wrong
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
Fullscreen button did not work, must be something wrong with the link, or pdf might be set to private https://www.scribd.com/document/400033734/Marcial-Bastia-Garcia-A216-205-472-BIA-Feb

In [ ]:
urls

In [ ]:
other_urls = [
  "https://www.scribd.com/document/367644102/Wilmer-Jaldon-Orosco-A205-008-796-BIA-Dec-7-2017",
  "https://www.scribd.com/document/367644086/Kyung-Lee-A071-523-654-BIA-Dec-7-2017",
  "https://www.scribd.com/document/367644064/S-A-M-AXXX-XXX-071-BIA-Dec-6-2017",
  "https://www.scribd.com/document/367643930/Roman-Kuot-A094-584-669-BIA-Dec-6-2017",
  "https://www.scribd.com/document/367643907/O-P-S-AXXX-XXX-565-BIA-Dec-6-2017",
  "https://www.scribd.com/document/367643878/Omar-Urbieta-Guerrero-A076-837-552-BIA-Dec-6-2017",
  "https://www.scribd.com/document/367643837/Eh-Moo-A212-085-801-BIA-Dec-6-2017",
  "https://www.scribd.com/document/367643796/Edwin-Carrillo-Mazariego-A208-023-110-BIA-Dec-5-2017",
  "https://www.scribd.com/document/367643742/M-F-A-AXXX-XXX-424-BIA-Dec-4-2017",
  "https://www.scribd.com/document/367643687/Jesus-Efrain-Jamarillo-Guillen-A099-161-946-BIA-Dec-4-2017",
  "https://www.scribd.com/document/367643654/Felipe-Turrubiates-Alegria-A206-491-082-BIA-Dec-4-2017",
  "https://www.scribd.com/document/367643627/D-M-A-AXXX-XXX-210-BIA-Dec-4-2017",
  "https://www.scribd.com/document/367643600/Adrian-Arturo-Gonzalez-Benitez-A208-217-647-BIA-Dec-4-2017",
  "https://www.scribd.com/document/367643568/Adan-Ricardo-Banuelos-Barraza-A206-279-823-BIA-Dec-4-2017",
  "https://www.scribd.com/document/367643533/N-M-AXXX-XXX-196-BIA-Dec-1-2017",
  "https://www.scribd.com/document/367643487/Marcos-Antonio-Acosta-Quirch-A044-971-669-BIA-Dec-1-2017",
  "https://www.scribd.com/document/367643448/Iankel-Ortega-A041-595-509-BIA-Dec-1-2017",
  "https://www.scribd.com/document/367643380/Jesus-Humberto-Zuniga-Romero-A205-215-795-BIA-Dec-1-2017",
  "https://www.scribd.com/document/367643359/Blanca-Doris-Barnes-A094-293-314-BIA-Dec-1-2017",
  "https://www.scribd.com/document/367643328/Vivroy-Kirlew-A058-906-861-BIA-Nov-30-2017",
  "https://www.scribd.com/document/367643189/Odalis-Vanessa-Garay-Murillo-A206-763-642-BIA-Nov-30-2017",
  "https://www.scribd.com/document/367643141/Armando-Fernandes-Lopes-A036-784-465-BIA-Nov-30-2017",
  "https://www.scribd.com/document/367643050/Yovani-Antonio-Mejia-A094-366-877-BIA-Nov-29-2017",
  "https://www.scribd.com/document/367642987/M-J-G-AXXX-XXX-861-BIA-Nov-29-2017",
  "https://www.scribd.com/document/367642940/Mario-Harold-Flores-A043-945-828-BIA-Nov-29-2017",
  "https://www.scribd.com/document/367642886/Jose-Jesus-Trujillo-A092-937-227-BIA-Nov-29-2017",
  "https://www.scribd.com/document/367642849/Jasmen-Basambekyan-A075-665-907-BIA-Nov-28-2017",
  "https://www.scribd.com/document/367642745/Saul-Rincon-Garcia-A034-338-426-BIA-Nov-27-2017",
  "https://www.scribd.com/document/367642533/N-M-H-V-AXXX-XXX-873-BIA-Nov-22-2017",
  "https://www.scribd.com/document/367642503/K-N-A-Z-AXXX-XXX-412-BIA-Nov-22-2017",
  "https://www.scribd.com/document/367642470/Ophelia-Salazar-Ellison-A200-185-139-BIA-Nov-21-2017",
  "https://www.scribd.com/document/367642435/Juan-Godoy-Arrenondo-A205-131-326-BIA-Nov-21-2017",
  "https://www.scribd.com/document/367642397/J-S-A-AXXX-XXX-618-BIA-Nov-21-2017",
  "https://www.scribd.com/document/367642375/H-J-C-AXXX-XXX-416-BIA-Nov-20-2017",
  "https://www.scribd.com/document/367642329/Antonio-Alvarez-Valencia-A094-380-709-BIA-Nov-20-2017",
  "https://www.scribd.com/document/367642301/G-A-A-M-AXXX-XXX-774-BIA-Nov-17-2017",
  "https://www.scribd.com/document/367642239/E-C-Q-AXXX-XXX-146-BIA-Nov-17-2017",
  "https://www.scribd.com/document/367642147/Rigoberto-Osiris-Sorto-Orellana-A078-971-293-BIA-Nov-16-2017",
  "https://www.scribd.com/document/367642105/V-A-C-AXXX-XXX-523-BIA-Nov-15-2017",
  "https://www.scribd.com/document/367642068/Miguel-Angel-Martinez-Guzman-A092-617-875-BIA-Nov-13-2017",
  "https://www.scribd.com/document/365747502/J-D-C-AXXX-XXX-963-BIA-Nov-13-2017",
  "https://www.scribd.com/document/365733396/R-D-R-P-AXXX-XXX-486-BIA-Nov-13-2017",
  "https://www.scribd.com/document/365733322/R-P-C-AXXX-XXX-469-BIA-Nov-13-2017",
  "https://www.scribd.com/document/365733190/Ruben-Masson-Orta-A025-408-267-BIA-Nov-9-2017",
  "https://www.scribd.com/document/365733080/Dariela-Patricia-Alvarado-Mendoza-A208-448-674-BIA-Nov-9-2017",
  "https://www.scribd.com/document/365732987/C-C-B-AXXX-XXX-180-BIA-Nov-7-2017",
  "https://www.scribd.com/document/365732951/J-S-C-AXXX-XXX-869-BIA-Nov-6-2017",
  "https://www.scribd.com/document/365732868/A-H-P-R-AXXX-XXX-112-BIA-Nov-6-2017",
  "https://www.scribd.com/document/365732785/R-D-G-AXXX-XXX-498-BIA-Nov-3-2017",
  "https://www.scribd.com/document/365732723/M-O-F-P-AXXX-XXX-141-BIA-Nov-3-2017",
  "https://www.scribd.com/document/365732669/Elida-Telma-Crisostomo-Ramos-A072-525-903-BIA-Nov-3-2017",
  "https://www.scribd.com/document/365732624/Ana-Silvia-Velasquez-Castellon-A205-016-240-BIA-Nov-2-2017",
  "https://www.scribd.com/document/365732547/Ana-Mercedes-Lemus-Duarte-A206-847-825-BIA-Nov-2-2017",
  "https://www.scribd.com/document/365732434/Gennadiy-Zheleznyak-A071-100-953-BIA-Nov-1-2017",
  "https://www.scribd.com/document/365698568/R-Y-I-AXXX-XXX-459-BIA-Oct-31-2017",
  "https://www.scribd.com/document/365698440/Francisco-T-Sanchez-A079-134-245-BIA-Oct-31-2017",
  "https://www.scribd.com/document/365698336/N-D-E-AXXX-XXX-335-BIA-Oct-30-2017",
  "https://www.scribd.com/document/365698196/Marguy-Gourdet-A096-269-563-BIA-Oct-30-2017",
  "https://www.scribd.com/document/365698104/D-C-G-AXXX-XXX-575-BIA-Oct-30-2017",
  "https://www.scribd.com/document/365697987/Amrollah-Vaziri-A099-237-670-BIA-Oct-27-2017",
  "https://www.scribd.com/document/365697847/I-D-R-C-AXXX-XXX-214-BIA-Oct-27-2017",
  "https://www.scribd.com/document/365697682/Carlos-Lopez-Mazariegos-A044-623-540-BIA-Oct-26-2017",
  "https://www.scribd.com/document/365697523/Brenda-Yamileth-Bonilla-Mejia-A078-277-418-BIA-Oct-26-2017",
  "https://www.scribd.com/document/365696674/J-S-O-R-AXXX-XXX-814-BIA-Oct-25-2017",
  "https://www.scribd.com/document/365696614/J-G-R-I-AXXX-XXX-711-BIA-Oct-25-2017",
  "https://www.scribd.com/document/365695680/Lulezim-Topllari-A076-678-765-BIA-Oct-25-2017",
  "https://www.scribd.com/document/365695538/Natalia-Nieto-Villalobos-A206-956-594-BIA-Oct-24-2017",
  "https://www.scribd.com/document/365695330/Augustine-Peleayz-A208-934-106-BIA-Oct-24-2017",
  "https://www.scribd.com/document/365695240/A-P-AXXX-XXX-411-BIA-Oct-24-2017",
  "https://www.scribd.com/document/365695128/J-L-M-L-AXXX-XXX-491-BIA-Oct-23-2017",
  "https://www.scribd.com/document/365694983/Rodolfo-Matos-Aquiles-A074-023-013-BIA-Oct-20-2017",
  "https://www.scribd.com/document/365694876/Abu-Sufiyan-A088-003-019-BIA-Oct-20-2017",
  "https://www.scribd.com/document/365694774/Miguel-Angel-Gamboa-Najera-A075-253-882-BIA-Oct-19-2017",
  "https://www.scribd.com/document/365694613/N-Y-T-H-AXXX-XXX-854-BIA-Oct-19-2017",
  "https://www.scribd.com/document/365694176/J-B-AXXX-XXX-033-BIA-Oct-19-2017",
  "https://www.scribd.com/document/365694007/J-S-R-AXXX-XXX-154-BIA-Oct-18-2017",
  "https://www.scribd.com/document/365693861/Juan-Xue-A206-683-080-BIA-Oct-17-2017",
  "https://www.scribd.com/document/365693727/Ana-Guadalupe-Landaverde-Morales-A206-722-187-BIA-Oct-17-2017",
  "https://www.scribd.com/document/365693654/Aldo-Reymund-Razo-A096-620-533-BIA-Oct-16-2017",
  "https://www.scribd.com/document/365693530/E-C-N-AXXX-XXX-788-BIA-Oct-13-2017",
  "https://www.scribd.com/document/365693442/M-E-C-AXXX-XXX-039-BIA-Oct-13-2017",
  "https://www.scribd.com/document/365693152/J-W-AXXX-XXX-281-BIA-Oct-13-2017",
  "https://www.scribd.com/document/365693091/Pedro-Daniel-Ramirez-Hernandez-A096-819-806-BIA-Oct-12-2017",
  "https://www.scribd.com/document/365692996/Ayaz-Khan-A071-801-450-BIA-Oct-12-2017",
  "https://www.scribd.com/document/365692961/R-L-P-AXXX-XXX-958-BIA-Oct-12-2017",
  "https://www.scribd.com/document/365692836/M-R-G-AXXX-XXX-770-BIA-Oct-12-2017",
  "https://www.scribd.com/document/362584623/Juan-Carlos-Rivas-Ayala-A208-890-222-BIA-Oct-11-2017",
  "https://www.scribd.com/document/362584550/M-R-R-A-AXXX-XXX-050-BIA-Oct-11-2017",
  "https://www.scribd.com/document/362584502/Vamsidhar-Vurimindi-A096-689-764-BIA-Oct-10-2017",
  "https://www.scribd.com/document/362584439/Sunil-Kumar-Bains-A209-154-541-BIA-Oct-10-2017",
  "https://www.scribd.com/document/362584390/F-N-A-Z-AXXX-XXX-416-BIA-Oct-6-2017",
  "https://www.scribd.com/document/362584329/Jose-Andres-Obando-Segura-A205-118-486-BIA-Oct-6-2017",
  "https://www.scribd.com/document/362584279/W-A-D-C-AXXX-XXX-861-BIA-Oct-6-2017",
  "https://www.scribd.com/document/362584218/B-V-N-AXXX-XXX-060-BIA-Oct-6-2017",
  "https://www.scribd.com/document/362584134/K-D-H-AXXX-XXX-337-BIA-Oct-5-2017",
  "https://www.scribd.com/document/362584076/Ronal-Antonio-Dominguez-A040-541-465-BIA-Oct-3-2017",
  "https://www.scribd.com/document/362584014/Humberto-Aristides-Machado-Brindis-A078-968-678-BIA-Oct-3-2017",
  "https://www.scribd.com/document/362583973/Ana-Paula-Mendes-A096-415-542-BIA-Oct-2-2017",
  "https://www.scribd.com/document/362583899/Ariel-Jonathan-Diaz-Vargas-A044-480-297-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583855/Eric-Omari-Thorpe-A047-924-686-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583817/D-N-R-D-AXXX-XXX-351-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583746/M-C-C-AXXX-XXX-436-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583713/Jordan-Omar-Nunez-Zepeda-A097-315-824-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583663/S-L-M-L-AXXX-XXX-703-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583607/Pedro-Antonio-Avalos-Jarquin-A208-285-701-BIA-Sept-29-2017",
  "https://www.scribd.com/document/362583564/W-S-AXXX-XXX-991-BIA-Sept-28-2017",
  "https://www.scribd.com/document/362583499/J-D-L-L-J-AXXX-XXX-318-BIA-Sept-28-2017",
  "https://www.scribd.com/document/362583429/G-M-V-M-AXXX-XXX-957-BIA-Sept-28-2017",
  "https://www.scribd.com/document/362583376/Flor-de-Maria-Palacios-A070-910-219-BIA-Sept-27-2017",
  "https://www.scribd.com/document/362583297/Luasa-Tshimanga-A209-012-271-BIA-Sept-25-2017",
  "https://www.scribd.com/document/362583246/Kamleshkumar-G-Patel-A087-385-865-BIA-Sept-25-2017",
  "https://www.scribd.com/document/362583187/Gurwinder-Singh-A208-751-159-BIA-Sept-25-2017",
  "https://www.scribd.com/document/362583122/G-P-AXXX-XXX-001-BIA-Sept-25-2017",
  "https://www.scribd.com/document/362583023/Jose-Cruz-Belman-Cano-A057-433-239-BIA-Sept-21-2017",
  "https://www.scribd.com/document/362582935/Yeon-Hee-Kim-A098-871-094-BIA-Sept-21-2017",
  "https://www.scribd.com/document/362582905/I-A-R-R-AXXX-XXX-887-BIA-Sept-20-2017",
  "https://www.scribd.com/document/362582841/Miriam-Awuku-A206-266-819-BIA-Sept-20-2017",
  "https://www.scribd.com/document/362582785/Rene-Enrique-Ascenio-Martinez-A088-968-133-BIA-Sept-20-2017",
  "https://www.scribd.com/document/362582712/Ruben-Arellano-Chairez-A208-212-580-BIA-Sept-20-2017",
  "https://www.scribd.com/document/362582668/Juan-Antonio-Flores-Urena-A204-126-639-BIA-Sept-19-2017",
  "https://www.scribd.com/document/362582546/Gabriela-Alesandra-Pajares-Arcaya-A078-508-085-BIA-Sept-18-2017",
  "https://www.scribd.com/document/362582484/S-S-AXXX-XXX-008-BIA-Sept-15-2017",
  "https://www.scribd.com/document/362582274/Dirav-Dolatbhai-Patel-A201-183-819-BIA-Sept-15-2017",
  "https://www.scribd.com/document/362530956/Dirav-Dolatbhai-Patel-A201-183-819-BIA-Sept-15-2017-pdf",
  "https://www.scribd.com/document/360079120/E-T-AXXX-XXX-069-BIA-Sept-13-2017",
  "https://www.scribd.com/document/360079097/Menoush-Ghazarian-A097-870-514-BIA-Sept-13-2017",
  "https://www.scribd.com/document/360079050/P-R-S-AXXX-XXX-503-BIA-Sept-13-2017",
  "https://www.scribd.com/document/360078837/M-G-G-AXXX-XXX-686-BIA-Sept-13-2017",
  "https://www.scribd.com/document/360078515/Mikheil-Kapanadze-A056-502-590-BIA-Sept-12-2017",
  "https://www.scribd.com/document/360078453/Carlos-Mitchell-A073-066-831-BIA-Sept-11-2017",
  "https://www.scribd.com/document/360078399/C-L-R-AXXX-XXX-328-BIA-Sept-7-2017",
  "https://www.scribd.com/document/360078337/Dieuseul-Alcide-A073-767-336-BIA-Sept-9-2017",
  "https://www.scribd.com/document/360078278/Jose-de-Jesus-Hernandez-Cid-A200-277-242-BIA-Sept-5-2017",
  "https://www.scribd.com/document/360078234/Rolando-Protocarrero-Tello-A097-659-598-BIA-Sept-5-2017",
  "https://www.scribd.com/document/360078191/Gustavo-Urias-Aguilar-A203-097-416-BIA-Sept-5-2017",
  "https://www.scribd.com/document/360078151/Wilphonse-Jean-A079-682-996-BIA-Sept-1-2017",
  "https://www.scribd.com/document/360078101/E-J-H-C-AXXX-XXX-605-BIA-Sept-1-2017",
  "https://www.scribd.com/document/360078043/Isai-Lopez-Hernandez-A046-620-341-BIA-Sept-1-2017",
  "https://www.scribd.com/document/360077980/M-R-G-C-AXXX-XXX-102-BIA-Aug-31-2017",
  "https://www.scribd.com/document/360077935/Oscar-Ramirez-Jimenez-A087-457-576-BIA-Aug-30-2017",
  "https://www.scribd.com/document/360077896/E-D-H-AXXX-XXX-523-BIA-Aug-29-2017",
  "https://www.scribd.com/document/360077856/Sukhdev-Singh-A209-154-612-BIA-Aug-25-2017",
  "https://www.scribd.com/document/360077810/Marco-Antonio-Vasquez-Jacobo-A070-719-141-BIA-Aug-24-2017",
  "https://www.scribd.com/document/360077751/Reyna-Alvarado-Alvarado-A077-531-518-BIA-Aug-23-2017",
  "https://www.scribd.com/document/360077712/Roderico-Geronimo-Tzum-Sum-A071-575-904-BIA-Aug-18-2017",
  "https://www.scribd.com/document/360077657/J-R-T-AXXX-XXX-885-BIA-Aug-18-2017",
  "https://www.scribd.com/document/360077591/Edgar-Marcelo-Alvarado-Turcio-A201-109-166-BIA-Aug-17-2017",
  "https://www.scribd.com/document/360077533/Roberto-Manuel-Ortega-Anaya-A209-245-963-BIA-Aug-17-2017",
  "https://www.scribd.com/document/360077482/K-S-AXXX-XXX-141-BIA-Aug-17-2017",
  "https://www.scribd.com/document/360077436/E-S-AXXX-XXX-170-BIA-Aug-17-2017",
  "https://www.scribd.com/document/360077356/Livinus-Ndubisi-Ezeobi-A061-561-711-BIA-Aug-17-2017",
  "https://www.scribd.com/document/360077242/Victor-Hugo-Ruiz-Reyes-A207-198-666-BIA-Aug-15-2017",
  "https://www.scribd.com/document/360077211/Ai-Sen-Bounthong-A040-408-560-BIA-Aug-15-2017",
  "https://www.scribd.com/document/360077135/Filiberto-Rosales-Gonzalez-A041-276-712-BIA-Aug-14-2017",
  "https://www.scribd.com/document/360077072/Thanh-Hoang-Nguyen-A042-647-333-BIA-Aug-11-2017",
  "https://www.scribd.com/document/360077013/Sai-Sandeep-Kurremula-A208-554-742-BIA-Aug-10-2017",
  "https://www.scribd.com/document/357071637/R-G-H-M-AXXX-XXX-972-BIA-Aug-9-2017",
  "https://www.scribd.com/document/357071575/M-S-V-E-AXXX-XXX-957-BIA-Aug-7-2017",
  "https://www.scribd.com/document/357071465/S-S-P-AXXX-XXX-854-BIA-Aug-4-2017",
  "https://www.scribd.com/document/357071399/Chao-Xing-Chen-A039-001-115-BIA-Aug-2-2017",
  "https://www.scribd.com/document/357071230/M-T-G-AXXX-XXX-648-BIA-July-31-2017",
  "https://www.scribd.com/document/357071131/C-M-O-G-AXXX-XXX-275-BIA-July-31-2017",
  "https://www.scribd.com/document/357071061/Axel-Kotowski-A017-799-451-BIA-July-27-2017",
  "https://www.scribd.com/document/357070967/Magin-Tapia-Cruz-A092-431-879-BIA-July-26-2017",
  "https://www.scribd.com/document/357070878/Nestor-Vicente-Loera-Arellano-A208-023-034-BIA-July-26-2017",
  "https://www.scribd.com/document/357070788/L-V-B-AXXX-XXX-430-BIA-July-26-2017",
  "https://www.scribd.com/document/357070602/C-D-L-G-AXXX-XXX-158-BIA-July-26-2017",
  "https://www.scribd.com/document/357070465/V-R-F-AXXX-XXX-988-BIA-July-25-2017",
  "https://www.scribd.com/document/357070353/R-S-A-AXXX-XXX-165-BIA-July-25-2017",
  "https://www.scribd.com/document/357070259/J-M-C-M-AXXX-XXX-706-BIA-July-21-2017",
  "https://www.scribd.com/document/357070152/V-M-P-O-AXXX-XXX-125-BIA-July-19-2017",
  "https://www.scribd.com/document/357070027/L-L-J-C-AXXX-XXX-378-BIA-July-19-2017",
  "https://www.scribd.com/document/357069926/Jorge-Ezequiel-Cruz-A202-132-224-BIA-July-19-2017",
  "https://www.scribd.com/document/357069690/S-F-Z-M-AXXX-XXX-301-BIA-July-19-2017",
  "https://www.scribd.com/document/357069268/Roxana-Guadalupe-Galindez-Villalba-Garriga-A099-163-817-BIA-July-18-2017",
  "https://www.scribd.com/document/357069113/Y-N-F-AXXX-XXX-018-BIA-July-18-2017",
  "https://www.scribd.com/document/357069008/W-P-M-AXXX-XXX-160-BIA-July-18-2017",
  "https://www.scribd.com/document/357068863/Mohammad-Jashim-Reza-A0093-304-084-BIA-July-18-2016",
  "https://www.scribd.com/document/357068801/Higinio-Rivas-Diaz-A208-362-593-BIA-July-18-2017",
  "https://www.scribd.com/document/357068700/Alhaji-Shoku-Lamin-A079-236-239-BIA-July-18-2017",
  "https://www.scribd.com/document/357068635/Esau-Lopez-Hernandez-A046-620-342-BIA-July-14-2017",
  "https://www.scribd.com/document/357068508/I-M-G-AXXX-XXX-772-BIA-July-28-2017",
  "https://www.scribd.com/document/354781910/T-T-C-AXXX-XXX-319-BIA-June-13-2017",
  "https://www.scribd.com/document/354776695/Teresa-Moreno-Gonzalez-A200-946-740-BIA-June-29-2017",
  "https://www.scribd.com/document/354776330/Roxine-Curlene-Mattis-A036-810-946-BIA-June-13-2017",
  "https://www.scribd.com/document/354776079/Rogelio-Reyes-Rodriguez-A205-920-648-BIA-June-23-2017",
  "https://www.scribd.com/document/354711594/Ricardo-Garcia-Diaz-A202-026-513-BIA-June-29-2017",
  "https://www.scribd.com/document/354711355/Redendo-Demosthene-A055-744-453-BIA-June-29-2017",
  "https://www.scribd.com/document/354710999/O-S-N-M-AXXX-XXX-633-BIA-July-10-2017",
  "https://www.scribd.com/document/354710446/Nishanthkumar-Patel-A073-546-027-BIA-July-11-2017",
  "https://www.scribd.com/document/354710317/Naually-Silva-Santos-Martins-A098-350-369-BIA-June-23-2017",
  "https://www.scribd.com/document/354710147/N-M-C-M-AXXX-XXX-821-BIA-July-5-2017",
  "https://www.scribd.com/document/354709846/Moses-Adams-Brewster-A024-023-175-BIA-June-27-2017",
  "https://www.scribd.com/document/354709217/Martha-Alicia-Perez-Hernandez-A092-773-151-BIA-June-15-2017",
  "https://www.scribd.com/document/354708214/M-A-N-AXXX-XXX-333-BIA-June-30-2017",
  "https://www.scribd.com/document/354707229/Leticia-Aparecida-Barbosa-Santos-A206-803-700-BIA-June-9-2017",
  "https://www.scribd.com/document/354706459/L-K-U-AXXX-XXX-003-BIA-June-16-2017",
  "https://www.scribd.com/document/354706108/K-A-B-V-AXXX-XXX-317-BIA-June-27-2017",
  "https://www.scribd.com/document/354705644/Jose-Luis-Everastico-Sotelo-BIA-June-12-2017",
  "https://www.scribd.com/document/354705376/Jose-Heriberto-Cadet-A027-646-311-BIA-June-26-2017",
  "https://www.scribd.com/document/354705042/Jorge-Albeto-Rodriguez-Vazquez-A205-292-786-BIA-June-15-2017",
  "https://www.scribd.com/document/354704749/J-R-M-AXXX-XXX-954-BIA-June-16-2017",
  "https://www.scribd.com/document/354703171/J-L-H-AXXX-XXX-772-BIA-June-29-2017",
  "https://www.scribd.com/document/354702427/J-J-V-AXXX-XXX-994-BIA-July-18-2017",
  "https://www.scribd.com/document/354701957/I-J-A-AXXX-XXX-630-BIA-June-20-2017",
  "https://www.scribd.com/document/354701211/Hector-Antonio-Jimenez-Santos-A098-489-385-BIA-June-27-2017",
  "https://www.scribd.com/document/354700727/F-S-AXXX-XXX-285-BIA-April-28-2017",
  "https://www.scribd.com/document/354700433/Eulalia-Gaspar-Tomas-A206-462-892-BIA-June-22-2017",
  "https://www.scribd.com/document/354699690/Ernesto-Rodrigo-Lopez-Clemente-A078-686-322-BIA-June-16-2017",
  "https://www.scribd.com/document/354699137/Efrain-Santos-Gijon-A208-837-339-BIA-June-22-2017",
  "https://www.scribd.com/document/354698284/E-E-M-AXXX-XXX-295-BIA-July-10-2017",
  "https://www.scribd.com/document/354697220/Douglas-Crucey-A043-446-797-BIA-June-12-2017",
  "https://www.scribd.com/document/354696891/B-V-N-AXXX-XXX-618-BIA-July-5-2017",
  "https://www.scribd.com/document/354695975/B-Q-AXXX-XXX-485-BIA-June-20-2017",
  "https://www.scribd.com/document/354695273/Augusto-Prado-A088-473-514-BIA-June-16-2017",
  "https://www.scribd.com/document/354694812/A-G-L-AXXX-XXX-744-BIA-July-12-2017",
  "https://www.scribd.com/document/354694303/Anahit-Melkonyan-A075-741-482-BIA-June-21-2017",
  "https://www.scribd.com/document/354693725/Alejando-Garnica-Silva-A098-269-615-BIA-June-29-2017",
  "https://www.scribd.com/document/354692443/A-S-C-H-AXXX-XXX-196-BIA-July-5-2017",
  "https://www.scribd.com/document/354691627/Al-S-Watkins-A047-928-924-BIA-June-9-2017",
  "https://www.scribd.com/document/354689757/A-A-AXXX-XXX-407-BIA-May-12-2017",
  "https://www.scribd.com/document/351906958/Keila-Isabel-Reyes-De-Romero-A206-698-059-BIA-June-8-2017",
  "https://www.scribd.com/document/351906908/Yunlei-Wang-A099-882-279-BIA-June-8-2017",
  "https://www.scribd.com/document/351906861/W-S-AXXX-XXX-033-BIA-June-8-2017",
  "https://www.scribd.com/document/351906807/Kenia-Morillo-Luciano-A060-183-622-BIA-June-8-2017",
  "https://www.scribd.com/document/351906763/Abner-Corgelas-A058-769-254-BIA-June-8-2017",
  "https://www.scribd.com/document/351906615/Cuong-Vi-Ngo-A057-771-073-BIA-June-8-2017",
  "https://www.scribd.com/document/351906565/A-N-AXXX-XXX-803-BIA-June-7-2017",
  "https://www.scribd.com/document/351906507/Manuela-Hernandez-Pasqual-A206-026-422-BIA-June-7-2017",
  "https://www.scribd.com/document/351906460/G-S-AXXX-XXX-625-BIA-June-5-2017",
  "https://www.scribd.com/document/351906401/Widens-Norcilus-A072-041-927-BIA-June-2-2017",
  "https://www.scribd.com/document/351906329/Naser-Noaman-Mohamed-Al-Maotari-A077-251-699-BIA-June-2-2017",
  "https://www.scribd.com/document/351906262/Jesus-Roberto-Corral-Trevizo-A034-979-405-BIA-June-2-2017",
  "https://www.scribd.com/document/351906132/G-R-M-AXXX-XXX-331-BIA-June-2-2017",
  "https://www.scribd.com/document/351906017/M-E-A-S-AXXX-XXX-912-BIA-June-2-2017",
  "https://www.scribd.com/document/351905950/Fernando-Cedillo-Cedillo-A209-763-977-BIA-June-1-2017",
  "https://www.scribd.com/document/351905895/Semsair-Paul-A098-508-518-BIA-May-30-2017",
  "https://www.scribd.com/document/351905851/Rafael-Fernandes-Pacheco-A089-677-735-BIA-May-30-2017",
  "https://www.scribd.com/document/351905796/Claudia-Carolina-Perez-Duran-A206-687-577-BIA-May-30-2017",
  "https://www.scribd.com/document/351905714/T-V-R-AXXX-XXX-286-BIA-May-26-2017",
  "https://www.scribd.com/document/351905638/Felipe-de-Jesus-Serna-Rojas-A205-634-861-BIA-May-26-2017",
  "https://www.scribd.com/document/351905530/A-M-I-U-AXXX-XXX-212-BIA-May-26-2017",
  "https://www.scribd.com/document/351905451/E-A-Y-AXXX-XXX-858-BIA-May-25-2017",
  "https://www.scribd.com/document/351905377/R-R-M-AXXX-XXX-256-BIA-May-24-2017",
  "https://www.scribd.com/document/351905106/Haris-Keserovic-A075-077-076-BIA-May-24-2017",
  "https://www.scribd.com/document/351904409/A-L-G-AXXX-XXX-831-BIA-May-24-2017",
  "https://www.scribd.com/document/351904353/F-A-AXXX-XXX-863-BIA-May-23-2017",
  "https://www.scribd.com/document/351904250/A-D-AXXX-XXX-526-BIA-May-22-2017",
  "https://www.scribd.com/document/351904150/Mingyue-Hu-A205-625-592-BIA-May-19-2017",
  "https://www.scribd.com/document/351904097/Jimmy-Giovanni-Perez-Figueroa-A206-165-088-BIA-May-19-2017",
  "https://www.scribd.com/document/351904038/J-C-G-AXXX-XXX-683-BIA-May-19-2017",
  "https://www.scribd.com/document/351904006/Dorina-I-Peduri-A071-302-021-BIA-May-19-2017",
  "https://www.scribd.com/document/351903957/Bao-Wu-Wang-A029-113-475-BIA-May-19-2017",
  "https://www.scribd.com/document/351903921/H-I-Y-AXXX-XXX-806-BIA-May-18-2017",
  "https://www.scribd.com/document/351903884/Argelia-Romero-Guerra-A078-038-133-BIA-May-18-2017",
  "https://www.scribd.com/document/351903826/M-J-R-AXXX-XXX-084-BIA-May-17-2017",
  "https://www.scribd.com/document/351903776/G-F-L-AXXX-XXX-133-BIA-May-17-2017",
  "https://www.scribd.com/document/351903730/E-D-J-F-T-AXXX-XXX-302-BIA-May-19-2017",
  "https://www.scribd.com/document/351903679/Thien-Thin-Nguyen-A073-279-229-BIA-May-16-2017",
  "https://www.scribd.com/document/351903537/Gaspar-Rosas-Hernandez-A205-147-712-BIA-May-16-2017",
  "https://www.scribd.com/document/351903496/Osmilda-Bonilla-Martinez-A077-670-908-BIA-May-15-2017",
  "https://www.scribd.com/document/351903447/Michael-Jeune-A079-346-938-BIA-May-15-2017",
  "https://www.scribd.com/document/351903367/Guadalupe-Carmona-Saldana-A097-705-178-BIA-May-15-2017",
  "https://www.scribd.com/document/351903301/Arnoldo-Baltazar-Montes-de-Oca-A093-322-719-BIA-May-15-2017",
  "https://www.scribd.com/document/351903086/Juan-Alberto-Mata-Siciliano-A094-790-928-BIA-May-11-2017",
  "https://www.scribd.com/document/349323604/America-Ambrosio-Domingo-A208-898-064-BIA-May-12-2017",
  "https://www.scribd.com/document/349323537/Jose-de-Jesus-Murillo-Gutierrez-A207-105-449-BIA-May-12-2017",
  "https://www.scribd.com/document/349323462/Jacek-Kaminski-A044-014-301-BIA-May-11-2017",
  "https://www.scribd.com/document/349323386/Juan-Camacho-Luz-A206-016-499-BIA-May-10-2017",
  "https://www.scribd.com/document/349322789/J-K-O-AXXX-XXX-418-BIA-May-10-2017",
  "https://www.scribd.com/document/349322729/Rosa-Mansia-Hernandez-A201-103-160-BIA-May-9-2017",
  "https://www.scribd.com/document/349322628/Gregoria-Saucedo-A201-216-144-BIA-May-8-2017",
  "https://www.scribd.com/document/349322552/C-H-C-AXXX-XXX-702-BIA-May-4-2017",
  "https://www.scribd.com/document/349321768/Jesus-Gomez-Marin-A092-096-240-BIA-May-4-2017",
  "https://www.scribd.com/document/349321686/Luis-Fernando-Armas-Villacis-A204-660-398-BIA-May-4-2017",
  "https://www.scribd.com/document/349321596/Adela-Nolasco-Santiago-A205-497-497-BIA-May-3-2017",
  "https://www.scribd.com/document/349321419/Robert-Rudoi-A013-998-118-BIA-May-2-2017",
  "https://www.scribd.com/document/349321262/Obed-Arellano-Aguilar-A205-316-952-BIA-April-28-2017",
  "https://www.scribd.com/document/349320997/John-Nyakundi-Omari-A028-989-663-BIA-April-28-2017",
  "https://www.scribd.com/document/349320848/German-Gallo-Orozco-A091-922-980-BIA-April-28-2017",
  "https://www.scribd.com/document/349320775/Enrique-Romero-Ramirez-A205-788-636-BIA-April-28-2017",
  "https://www.scribd.com/document/349320689/Amos-Zeith-Benka-Coker-A056-082-534-BIA-April-28-2017",
  "https://www.scribd.com/document/349320587/Jose-de-Jesus-Delgadillo-Armas-A092-411-542-BIA-April-27-2017",
  "https://www.scribd.com/document/349320392/Zorina-Pangilinan-A046-403-539-BIA-April-26-2017",
  "https://www.scribd.com/document/349320269/S-D-AXXX-XXX-230-BIA-April-26-2017",
  "https://www.scribd.com/document/349320149/Nasir-Ali-Khan-A059-549-769-BIA-April-26-2017",
  "https://www.scribd.com/document/349320013/Devon-Christie-A042-259-741-BIA-April-26-2017",
  "https://www.scribd.com/document/349319739/David-Matehuala-Grimaldo-A028-889-405-BIA-April-26-2017",
  "https://www.scribd.com/document/349319643/Andrine-Waruguru-Wambui-A089-313-705-BIA-April-26-2017",
  "https://www.scribd.com/document/349319561/Alicia-Noemy-Quinteros-Agueta-A098-113-685-BIA-April-26-2017",
  "https://www.scribd.com/document/349319390/M-R-R-A-AXXX-XXX-496-BIA-April-25-2017",
  "https://www.scribd.com/document/349319328/Lusca-Petitdor-A024-667-417-BIA-April-24-2017",
  "https://www.scribd.com/document/349319254/Beverly-Evadne-Bent-A017-521-783-BIA-April-21-2017",
  "https://www.scribd.com/document/349319177/Ivan-Garcia-Reyes-A200-156-257-BIA-April-20-2017",
  "https://www.scribd.com/document/349318995/E-C-AXXX-XXX-516-Bia-April-20-2017",
  "https://www.scribd.com/document/349318926/Alejando-Pizana-Madera-A044-566-261-BIA-April-20-2017",
  "https://www.scribd.com/document/349318823/Lorne-Allan-Semrau-A022-201-177-BIA-April-19-2017",
  "https://www.scribd.com/document/349318717/Juan-Garcia-Garcia-A087-175-395-BIA-April-18-2017",
  "https://www.scribd.com/document/349318593/Juan-Ruiz-Medina-A200-954-354-BIA-April-18-2017",
  "https://www.scribd.com/document/348650769/Anna-May-de-Leon-Galono-A089-528-341-BIA-Sept-29-2015",
  "https://www.scribd.com/document/346343500/Merlin-Lacayo-Ordonez-A098-490-213-BIA-April-17-2017",
  "https://www.scribd.com/document/346343405/David-Chavez-A070-911-147-BIA-April-17-2017",
  "https://www.scribd.com/document/346343191/Stefan-Abhishek-Vincent-A208-357-277-BIA-April-14-2017",
  "https://www.scribd.com/document/346342951/Virgilio-Fidelino-Cifuentes-A206-401-608-BIA-April-13-2017",
  "https://www.scribd.com/document/346342842/Celso-Ramirez-A072-020-938-BIA-April-12-2017",
  "https://www.scribd.com/document/346342741/Jose-Israel-Chavarria-Reyes-A206-274-376-BIA-April-10-2017",
  "https://www.scribd.com/document/346342646/Ignacio-Castaneda-Galindo-A095-726-150-BIA-April-10-2017",
  "https://www.scribd.com/document/346342267/R-S-A-AXXX-XXX-926-BIA-April-7-2017",
  "https://www.scribd.com/document/346341998/Ramon-Jasso-Arangure-A056-333-337-BIA-April-7-2017",
  "https://www.scribd.com/document/346341858/T-G-AXXX-XXX-353-BIA-April-6-2017",
  "https://www.scribd.com/document/346341539/Samuel-Gomez-A090-921-930-BIA-April-6-2017",
  "https://www.scribd.com/document/346341430/Jose-Augustin-Fernandez-A044-312-625-BIA-April-6-2017",
  "https://www.scribd.com/document/346341345/Melia-Howie-A036-367-206-BIA-April-5-2017",
  "https://www.scribd.com/document/346341239/Abner-Jose-Clara-Vigil-A208-451-969-BIA-April-4-2017",
  "https://www.scribd.com/document/346341125/Md-All-Rabby-Toukir-A058-546-274-BIA-April-4-2017",
  "https://www.scribd.com/document/346341003/Gustavo-Gaitan-A205-712-334-BIA-March-31-2017",
  "https://www.scribd.com/document/346340749/Natividad-de-Jesus-Ramirez-Sibrian-A098-713-951-BIA-March-31-2017",
  "https://www.scribd.com/document/346340498/Felix-Agbor-Anyior-Nkongho-A096-112-440-BIA-March-31-2017",
  "https://www.scribd.com/document/346340380/Eyal-Misha-Aharon-A086-992-514-BIA-March-31-2017",
  "https://www.scribd.com/document/346340249/Jesus-Javier-de-Jesus-Rosario-A056-514-289-BIA-March-31-2017",
  "https://www.scribd.com/document/346340104/Alfredo-Hilario-Alvarado-Gonzalez-A090-853-184-BIA-March-30-2017",
  "https://www.scribd.com/document/346339893/David-Castro-Camacho-A058-107-973-BIA-March-30-2017",
  "https://www.scribd.com/document/346339707/H-M-F-AXXX-XXX-345-BIA-March-29-2017",
  "https://www.scribd.com/document/346337593/Oscar-Moreno-Gomez-A206-653-438-BIA-March-29-2017",
  "https://www.scribd.com/document/346337388/Macario-Jesus-Bonilla-A090-170-253-BIA-March-29-2017",
  "https://www.scribd.com/document/346337237/Jonathan-Jemenez-Nuezca-A087-714-293-BIA-March-28-2017",
  "https://www.scribd.com/document/346336991/Milton-Modesto-Jarama-A073-576-642-BIA-March-28-2017",
  "https://www.scribd.com/document/346336833/Sixto-Delgado-A075-423-408-BIA-March-27-2017",
  "https://www.scribd.com/document/346336602/Francisco-Villagrana-Robles-A094-984-254-BIA-March-27-2017",
  "https://www.scribd.com/document/346336484/Antonio-Garcia-Varela-A074-573-762-BIA-March-27-2017",
  "https://www.scribd.com/document/346336265/I-M-P-H-AXXX-XXX-457-BIA-March-22-2017",
  "https://www.scribd.com/document/346335851/Vera-Sama-A076-581-488-BIA-March-22-2017",
  "https://www.scribd.com/document/346335720/Edith-Esperansa-Sanabria-Martinez-A070-169-937-BIA-March-22-2017",
  "https://www.scribd.com/document/346335583/Ranjeet-Singh-A208-191-523-BIA-March-21-2017",
  "https://www.scribd.com/document/346335291/Vera-Valeryevna-Putro-A099-280-517-BIA-March-21-2017",
  "https://www.scribd.com/document/346335089/Francisco-Zanabria-Arteaga-A075-844-244-BIA-March-21-2017",
  "https://www.scribd.com/document/346334939/Jesus-Omar-Turcios-Castellanos-A200-005-691-BIA-March-17-2017",
  "https://www.scribd.com/document/346334687/F-R-M-C-AXXX-XXX-037-BIA-March-17-2017",
  "https://www.scribd.com/document/346334508/Ketlie-Augustin-A097-199-166-BIA-March-17-2017",
  "https://www.scribd.com/document/346334346/Esperanza-Alcaraz-De-Vasquez-A076-626-660-BIA-March-16-2017",
  "https://www.scribd.com/document/346334118/Lilian-D-Miranda-A074-667-323-BIA-March-16-2017",
  "https://www.scribd.com/document/346333895/D-R-J-B-AXXX-XXX-157-BIA-March-16-2017",
  "https://www.scribd.com/document/346333402/Leon-Christopher-Peters-A046-422-396-BIA-March-15-2017",
  "https://www.scribd.com/document/346333044/Eduarda-de-Gonzalez-A096-699-471-BIA-March-15-2017",
  "https://www.scribd.com/document/342627095/M-M-J-AXXX-XXX-522-BIA-March-15-2017",
  "https://www.scribd.com/document/342626967/C-E-M-M-AXXX-XXX-189-BIA-March-15-2017",
  "https://www.scribd.com/document/342626377/Kevin-George-Marsh-A086-946-931-BIA-March-14-2017",
  "https://www.scribd.com/document/342626185/Armando-Pacheco-Sanchez-A205-462-394-BIA-March-10-2017",
  "https://www.scribd.com/document/342625896/Alejandro-Gonzalo-Trejo-A205-215-507-BIA-March-10-2017",
  "https://www.scribd.com/document/342625642/Tu-Kim-Le-A063-261-404-BIA-March-10-2017",
  "https://www.scribd.com/document/342625441/D-E-C-AXXX-XXX-856-BIA-March-10-2017",
  "https://www.scribd.com/document/342624914/M-K-I-AXXX-XXX-691-BIA-March-9-2017",
  "https://www.scribd.com/document/342623120/J-J-B-M-AXXX-XXX-282-BIA-March-9-2017",
  "https://www.scribd.com/document/342622684/G-A-M-A-AXXX-XXX-034-BIA-March-9-2017",
  "https://www.scribd.com/document/342622208/Roy-Alberto-Ramirez-Calderon-A200-423-316-BIA-March-9-2017",
  "https://www.scribd.com/document/342621959/Maria-Virginia-Mejia-Erazo-A200-114-497-BIA-March-9-2017",
  "https://www.scribd.com/document/342621802/Monica-Lemus-A089-392-828-BIA-March-9-2017",
  "https://www.scribd.com/document/342621558/J-H-C-AXXX-XXX-365-BIA-March-9-2017",
  "https://www.scribd.com/document/342621271/Ka-I-Lei-A046-594-161-BIA-March-8-2017",
  "https://www.scribd.com/document/342620720/J-A-A-G-AXXX-XXX-844-BIA-March-8-2017",
  "https://www.scribd.com/document/342620415/Adrian-Escalona-Dominguez-A201-245-549-BIA-March-7-2017",
  "https://www.scribd.com/document/342620183/Leonidas-Sazo-Hernandez-A206-497-081-BIA-March-6-2017",
  "https://www.scribd.com/document/342620015/Thales-Alberto-Duarte-Oliveria-A201-065-524-BIA-March-6-2017",
  "https://www.scribd.com/document/342619670/Santiago-Gonzalez-Alonzo-A200-883-832-BIA-March-6-2017",
  "https://www.scribd.com/document/342619336/Fidel-Gonzalez-A205-754-142-BIA-March-3-2017",
  "https://www.scribd.com/document/342619150/Manuel-de-Jesus-Castillo-Trejo-A059-745-573-BIA-March-3-2017",
  "https://www.scribd.com/document/342618911/Wayne-Findlay-A041-653-797-BIA-March-3-2017",
  "https://www.scribd.com/document/342618711/Rosa-Elva-Figueroa-Mendoza-A208-598-824-BIA-March-2-2017",
  "https://www.scribd.com/document/342618428/Armando-Aparicio-Sanchez-A206-132-700-BIA-March-2-2017",
  "https://www.scribd.com/document/342618197/Mubarak-O-Raifiu-A209-171-968-BIA-Feb-27-2017",
  "https://www.scribd.com/document/342617903/Luis-Alberto-Alcaraz-A089-775-377-BIA-Feb-27-2017",
  "https://www.scribd.com/document/342617698/Sandra-Viviane-Soares-Santos-A079-446-316-BIA-Feb-27-2017",
  "https://www.scribd.com/document/342617478/Giles-Chibuzor-Mbonu-A057-731-341-BIA-Feb-27-2017",
  "https://www.scribd.com/document/342617209/Yuwikza-Doris-Lucero-A076-534-637-BIA-Feb-24-2017",
  "https://www.scribd.com/document/342616959/David-Sebastian-Arce-Marchant-A077-912-760-BIA-Feb-23-2017",
  "https://www.scribd.com/document/342616682/Einstein-Ofotsu-Djabeng-A205-492-530-BIA-Feb-16-2017",
  "https://www.scribd.com/document/342615484/Saturnino-Medrano-Herrera-A206-786-746-BIA-Feb-16-2017",
  "https://www.scribd.com/document/339937993/Nigel-Omar-Smith-A210-178-324-BIA-Feb-15-2017",
  "https://www.scribd.com/document/339937748/Leonel-Valdez-Garcia-A208-932-090-BIA-Feb-15-2017",
  "https://www.scribd.com/document/339936996/Zhongen-Li-A059-930-907-BIA-Feb-15-2017",
  "https://www.scribd.com/document/339936620/Manuel-Emilio-Melo-A043-691-095-BIA-Feb-15-2017",
  "https://www.scribd.com/document/339936282/Elisabete-Monteiro-Lopes-A041-958-867-BIA-Feb-15-2017",
  "https://www.scribd.com/document/339935696/Jose-Luis-Pimber-Cirerol-A019-117-585-BIA-Feb-15-2017",
  "https://www.scribd.com/document/339935414/Ernesto-Martinez-Antonio-A206-304-248-BIA-Feb-14-2017",
  "https://www.scribd.com/document/339935244/Jason-Mitchell-A205-908-063-BIA-Feb-14-2017",
  "https://www.scribd.com/document/339935072/Tania-Bibiano-Escalante-A205-324-100-BIA-Feb-14-2017",
  "https://www.scribd.com/document/339934884/Mario-Bladimir-Ayala-Monzon-A206-409-571-BIA-Feb-13-2017",
  "https://www.scribd.com/document/339933463/H-M-AXXX-XXX-389-BIA-Jan-30-2017",
  "https://www.scribd.com/document/339927173/Hiram-Abecardo-Gonzalez-Sarat-A205-164-431-BIA-Jan-30-2017",
  "https://www.scribd.com/document/339926372/Rogie-Laquian-Vasquez-A046-562-059-BIA-Jan-19-2017",
  "https://www.scribd.com/document/339925965/Ani-Salesi-Palau-A042-689-094-BIA-Feb-2-2017",
  "https://www.scribd.com/document/339925602/C-E-L-AXXX-XXX-304-BIA-Jan-12-2017",
  "https://www.scribd.com/document/339925350/Lennox-Anthony-Joseph-A072-033-408-BIA-Feb-7-2017",
  "https://www.scribd.com/document/339924742/Fernando-Cardeas-Cazares-A014-273-381-BIA-Jan-27-2017",
  "https://www.scribd.com/document/339924267/F-A-M-AXXX-XXX-992-BIA-Jan-26-2017",
  "https://www.scribd.com/document/339923761/Miguel-Enrique-Matzar-Alvarado-A055-427-275-BIA-Feb-3-2017",
  "https://www.scribd.com/document/339923154/James-Ogunyemi-Friday-A078-510-752-BIA-Jan-30-2017",
  "https://www.scribd.com/document/339921996/M-M-S-AXXX-XXX-057-BIA-Feb-9-2017",
  "https://www.scribd.com/document/339921740/J-F-A-H-AXXX-XXX-011-BIA-Jan-19-2017",
  "https://www.scribd.com/document/339405929/Vasile-George-Husti-A057-497-914-BIA-Feb-7-2017",
  "https://www.scribd.com/document/339405318/Bethel-Wolfgan-Mbaba-A209-158-298-BIA-Feb-3-2017",
  "https://www.scribd.com/document/339405113/Adan-Queriapa-Tomas-A205-232-307-BIA-Feb-3-2017",
  "https://www.scribd.com/document/339404820/Leonel-Olmedo-Avalos-A208-484-533-BIA-Jan-31-2017",
  "https://www.scribd.com/document/339404500/Anabela-Godinez-Perez-A208-202-259-BIA-Jan-31-2017",
  "https://www.scribd.com/document/339404202/Jin-Wei-Gao-A079-692-001-BIA-Jan-31-2007",
  "https://www.scribd.com/document/339404020/Cristobal-Villatoro-A077-751-555-BIA-Jan-31-2017",
  "https://www.scribd.com/document/339402795/Deivis-Rene-Aguilar-A055-815-373-BIA-Jan-30-2017",
  "https://www.scribd.com/document/339402187/Nuria-Esely-Sabillon-Guardado-A208-275-394-BIA-Jan-27-2017",
  "https://www.scribd.com/document/339313973/Stefano-Righi-A200-386-011-BIA-Jan-26-2017",
  "https://www.scribd.com/document/339312117/Saadat-Baig-A043-589-486-BIA-Jan-26-2017",
  "https://www.scribd.com/document/339311786/Manuel-Aaron-Perez-Galvez-A205-907-274-BIA-Jan-26-2017",
  "https://www.scribd.com/document/339311603/Charanjit-Singh-A205-491-215-BIA-Jan-26-2017",
  "https://www.scribd.com/document/339310423/Kirk-Sean-Marlon-Young-A035-922-976-BIA-Jan-25-2017",
  "https://www.scribd.com/document/339310210/Herick-Joel-Vallecillo-Castillo-A200-557-547-BIA-Jan-19-2017",
  "https://www.scribd.com/document/339309928/Jeannine-Evelin-Stevens-A036-377-883-BIA-Jan-19-2017",
  "https://www.scribd.com/document/339309777/Luis-Octavio-Avelar-Ramirez-A205-151-099-BIA-Jan-18-2017",
  "https://www.scribd.com/document/339309605/Abdul-R-Musah-A076-553-210-BIA-Jan-18-2017",
  "https://www.scribd.com/document/339309391/Luis-Alberto-Sosa-Alvarado-A200-718-257-BIA-Jan-17-2017",
  "https://www.scribd.com/document/339238275/Abayneh-Arifcho-Hegana-A062-520-623-BIA-Jan-26-2017",
  "https://www.scribd.com/document/337542906/A-A-L-M-AXXX-XXX-585-BIA-Oct-22-2015",
  "https://www.scribd.com/document/337542701/S-I-B-AXXX-XXX-978-BIA-Dec-16-2016",
  "https://www.scribd.com/document/337542224/Pete-Teichroeb-Peters-A093-518-654-BIA-Jan-13-2017",
  "https://www.scribd.com/document/337542034/S-B-D-AXXX-XXX-274-BIA-Jan-11-2017",
  "https://www.scribd.com/document/337541890/Azmat-Ali-Khattak-A044-904-454-BIA-Jan-11-2017",
  "https://www.scribd.com/document/337541426/Felix-Garcia-De-Luna-A205-493-691-BIA-Jan-6-2017",
  "https://www.scribd.com/document/337541279/Luis-Angel-Cruz-Torres-A204-840-095-BIA-Jan-6-2017",
  "https://www.scribd.com/document/337540999/Victor-J-Ocampo-Lopez-A200-557-585-BIA-Jan-6-2017",
  "https://www.scribd.com/document/337540716/Clebson-Sousa-Carneiro-A078-254-701-BIA-Jan-6-2017",
  "https://www.scribd.com/document/337540534/Arturo-Mandujano-Torres-A091-480-873-BIA-Jan-4-2017",
  "https://www.scribd.com/document/337540189/Roberto-Lopez-Valencia-A090-797-248-BIA-Dec-27-2016",
  "https://www.scribd.com/document/337539141/Bertha-A-Lopez-Tovar-A071-904-519-BIA-Dec-12-2016",
  "https://www.scribd.com/document/337538895/Gerardo-Perez-Alonzo-A046-629-923-BIA-Dec-22-2016",
  "https://www.scribd.com/document/337538434/V-R-AXXX-XXX-467-BIA-Dec-13-2016",
  "https://www.scribd.com/document/337538169/Qiang-Fu-A205-717-346-BIA-Jan-5-2017",
  "https://www.scribd.com/document/337538046/Long-Dang-Nguyen-A098-526-352-BIA-Jan-3-2017",
  "https://www.scribd.com/document/337537902/Akim-Joseph-Trivus-Charles-A040-166-697-BIA-Jan-3-2017",
  "https://www.scribd.com/document/337537700/Gregorio-Antiveros-Rodriguez-A200-241-339-BIA-Dec-30-2016",
  "https://www.scribd.com/document/337537551/Wilson-Eduardo-Pena-Yungasaca-A200-826-015-BIA-Dec-30-2016",
  "https://www.scribd.com/document/337537396/Minh-Phan-A025-085-074-BIA-Dec-30-2016",
  "https://www.scribd.com/document/337537195/Clemilda-Alves-Da-Silva-A098-401-988-BIA-Dec-29-2016",
  "https://www.scribd.com/document/337537002/Doris-Yesenia-Garcia-Enamorado-A076-317-461-BIA-Dec-29-2016",
  "https://www.scribd.com/document/337536726/B-A-M-G-AXXX-XXX-558-BIA-Dec-27-2016",
  "https://www.scribd.com/document/337536365/S-D-A-A-AXXX-XXX-716-BIA-Dec-27-2016",
  "https://www.scribd.com/document/337536202/Manuel-Victor-Lucero-A074-912-171-BIA-Dec-27-2016",
  "https://www.scribd.com/document/337535963/Joel-Hernandez-A074-043-821-BIA-Dec-27-2016",
  "https://www.scribd.com/document/337535635/E-F-H-AXXX-XXX-818-BIA-Dec-22-2016",
  "https://www.scribd.com/document/337535424/Ala-Abdel-Muhsen-Rafati-A087-969-447-BIA-Dec-22-2016",
  "https://www.scribd.com/document/337535281/Gerson-Adonay-Moreno-Reina-A077-607-472-BIA-Dec-22-2016",
  "https://www.scribd.com/document/337535116/E-R-AXXX-XXX-571-BIA-Dec-21-2016",
  "https://www.scribd.com/document/337534911/Victor-Manuel-Lopez-Lopez-A042-916-662-Dec-21-2016",
  "https://www.scribd.com/document/337534692/Majok-Majok-A094-582-812-BIA-Dec-20-2016",
  "https://www.scribd.com/document/337534470/Sergio-Luis-Del-Fierro-Garcia-A043-791-852-BIA-Dec-19-2016",
  "https://www.scribd.com/document/337534153/Jenny-Yesenia-Leiva-Acosta-A208-893-554-BIA-Dec-16-2016",
  "https://www.scribd.com/document/337533766/Sindy-Marbella-Galeas-Ochoa-A208-258-661-BIA-Dec-15-2016",
  "https://www.scribd.com/document/337533566/Ming-Yue-Piao-A096-187-617-BIA-Dec-15-2016",
  "https://www.scribd.com/document/337533397/Waldy-Mena-Lopez-A044-132-788-BIA-Dec-15-2016",
  "https://www.scribd.com/document/337532719/C-J-B-F-AXXX-XXX-418-BIA-Dec-13-2016",
  "https://www.scribd.com/document/336509054/W-L-A-M-AXXX-XXX-560-BIA-Dec-23-2016",
  "https://www.scribd.com/document/334876164/W-K-H-AXXX-XXX-710-BIA-Nov-22-2016",
  "https://www.scribd.com/document/334719113/Sean-Terry-Darwin-Haynes-A036-574-645-BIA-Dec-2-2016",
  "https://www.scribd.com/document/334719092/Kevin-Calvin-Vasley-Lowe-A061-494-852-BIA-Dec-6-2016",
  "https://www.scribd.com/document/334719083/Julio-Ledesma-Moreno-A206-738-790-BIA-Nov-28-2016",
  "https://www.scribd.com/document/334719081/Daniele-Pazi-Alvarez-A047-350-423-BIA-Dec-12-2016",
  "https://www.scribd.com/document/334719069/Kokou-Jonito-Doeh-A042-949-629-BIA-Dec-5-2016",
  "https://www.scribd.com/document/334719052/Isela-Esmeralda-Hernandez-Reyes-A098-719-936-BIA-Dec-9-2016",
  "https://www.scribd.com/document/334719048/Uriel-Garcia-Macedo-A092-058-021-BIA-Dec-1-2016",
  "https://www.scribd.com/document/334719032/Pedro-Amaya-Flores-A094-788-974-BIA-Dec-2-2016",
  "https://www.scribd.com/document/334719022/Mario-Lino-Barbosa-Souto-A040-144-389-BIA-Dec-5-2016",
  "https://www.scribd.com/document/334193252/Hussein-Mohd-Mahd-A044-688-449-BIA-Nov-21-2016",
  "https://www.scribd.com/document/334193225/Luis-Alfredo-Davila-Barrera-A074-583-018-BIA-Nov-23-2016",
  "https://www.scribd.com/document/334193224/Rodolfo-Melendez-Manriquez-A089-599-983-BIA-Nov-25-2016",
  "https://www.scribd.com/document/334193200/Frislid-Mejia-Vasquez-A206-091-158-BIA-Nov-21-2016",
  "https://www.scribd.com/document/334193179/Sergio-Lugo-Resendiz-A034-450-500-BIA-Nov-21-2016",
  "https://www.scribd.com/document/334139459/S-V-C-AXXX-XXX-431-BIA-Nov-1-2016",
  "https://www.scribd.com/document/334139458/Jose-Hernandez-Mendoza-A208-090-199-BIA-Aug-4-2016",
  "https://www.scribd.com/document/334139455/Eder-Lopez-Moncada-A046-081-498-BIA-Nov-17-2016",
  "https://www.scribd.com/document/334139454/Luis-Fernando-Gomez-Otoya-A087-245-163-BIA-Nov-15-2016",
  "https://www.scribd.com/document/334139451/Srikanth-Kasim-Reddy-A208-198-222-BIA-Nov-18-2016",
  "https://www.scribd.com/document/334139450/Nicolas-Parra-Arredondo-A205-704-279-BIA-Nov-18-2016",
  "https://www.scribd.com/document/334139448/Armando-Aparicio-Sanchez-A206-132-700-BIA-Nov-15-2016",
  "https://www.scribd.com/document/334139447/Wilson-Reynoso-Mendez-A206-077-560-BIA-Nov-18-2016",
  "https://www.scribd.com/document/331835991/Julio-Rosendo-Serrano-A206-482-000-BIA-Nov-10-2016",
  "https://www.scribd.com/document/331835788/Oscar-Euceda-Rueda-A077-479-978-BIA-Nov-10-2016",
  "https://www.scribd.com/document/331835546/Felipe-Perez-Lopez-A208-055-200-BIA-Nov-8-2016",
  "https://www.scribd.com/document/331835342/Onyebuchi-Ogbonna-Irokanulo-A099-400-712-BIA-Nov-7-2016",
  "https://www.scribd.com/document/331835014/Lesly-X-Alarcon-Quijada-A208-593-158-BIA-Nov-4-2016",
  "https://www.scribd.com/document/331834682/Maureen-Ngallo-A089-431-155-BIA-Nov-4-2016",
  "https://www.scribd.com/document/331834390/Irene-O-Atseyinku-A058-536-724-BIA-Oct-25-2016",
  "https://www.scribd.com/document/331833963/Milagro-Isolina-Mulatillo-Arrue-A200-060-538-BIA-Nov-3-2016",
  "https://www.scribd.com/document/331833744/Ivan-Amador-Olivares-Abarca-A096-560-171-BIA-Nov-3-2016",
  "https://www.scribd.com/document/331833576/Bryan-Evans-Gamez-Diaz-A200-108-936-BIA-Nov-2-2016",
  "https://www.scribd.com/document/331833362/Jean-Claude-Pierre-A087-107-915-BIA-Nov-2-2016",
  "https://www.scribd.com/document/330542457/Jonathan-Jemenez-Nuezca-A087-714-293-BIA-Oct-31-2016",
  "https://www.scribd.com/document/330541890/Gulin-Cai-A086-994-205-BIA-Oct-28-2016",
  "https://www.scribd.com/document/330541538/Ronald-Ayala-Mejia-A098-885-532-BIA-Oct-27-2016",
  "https://www.scribd.com/document/330541273/A-L-M-D-AXXX-XXX-671-BIA-Oct-26-2016",
  "https://www.scribd.com/document/330541029/Giorg-Rapava-A077-018-104-BIA-Oct-26-2016",
  "https://www.scribd.com/document/330540812/Raymond-Kosschuk-A072-858-139-BIA-Oct-26-2016",
  "https://www.scribd.com/document/330540409/J-R-M-B-AXX-XXX-175-BIA-Oct-25-2016",
  "https://www.scribd.com/document/330539733/Claudia-Molina-Pena-A089-526-024-BIA-Oct-25-2016",
  "https://www.scribd.com/document/330539424/L-A-C-O-AXXX-XXX-598-BIA-Oct-24-2016",
  "https://www.scribd.com/document/330538970/Victoria-Sapon-Ordonez-A072-681-560-BIA-Oct-24-2016",
  "https://www.scribd.com/document/330538575/Nidal-Shawkat-Salem-A046-010-157-BIA-Oct-24-2016",
  "https://www.scribd.com/document/330538288/J-T-M-G-AXXX-XXX-080-BIA-Oct-21-2016",
  "https://www.scribd.com/document/330537922/Michael-Pietrantonio-A078-590-230-BIA-Oct-21-2016",
  "https://www.scribd.com/document/330537673/S-G-P-G-AXXX-XXX-389-BIA-Oct-20-2016",
  "https://www.scribd.com/document/330537440/Yu-Qing-Chen-A077-977-214-BIA-Oct-20-2016",
  "https://www.scribd.com/document/330537210/Kenia-Maleyvi-Solorzano-Martinez-A200-192-464-BIA-Oct-19-2016",
  "https://www.scribd.com/document/330536846/Grace-Lina-Moral-Capiendo-A078-029-521-BIA-Oct-19-2016",
  "https://www.scribd.com/document/330536419/Smith-de-Jesus-Pena-Rojas-A041-588-479-BIA-Oct-19-2016",
  "https://www.scribd.com/document/330536019/John-Adrian-Maldonado-Guinanzaca-A205-873-454-BIA-Oct-18-2016",
  "https://www.scribd.com/document/330535672/Jorge-Leon-Esperanza-A095-750-346-BIA-Oct-18-2016",
  "https://www.scribd.com/document/330535177/Kemanit-Butrad-A205-701-814-BIA-Oct-17-2016",
  "https://www.scribd.com/document/330534775/Ninfa-Rosa-Aguilera-Galeas-A098-986-533-BIA-Oct-17-2016",
  "https://www.scribd.com/document/330534461/Riyade-Nafee-A089-616-155-BIA-Oct-17-2016",
  "https://www.scribd.com/document/330534037/Aman-Kumar-A079-075-984-BIA-Oct-17-2016",
  "https://www.scribd.com/document/330533699/Howard-Anthony-Johnson-A055-559-497-BIA-Oct-17-2016",
  "https://www.scribd.com/document/330533348/Adeliya-Mubarika-Ahmad-A096-297-241-BIA-Oct-14-2016",
  "https://www.scribd.com/document/330533078/Mohammad-Babul-Islam-A045-052-167-BIA-Oct-14-2016",
  "https://www.scribd.com/document/330532473/Jesus-Alberto-Garcia-Nava-A205-721-176-BIA-Oct-13-2016",
  "https://www.scribd.com/document/328176256/F-E-H-AXXX-XXX-193-BIA-Sept-2-2016",
  "https://www.scribd.com/document/328169215/Eladia-Pablo-Mendoza-A208-307-494-BIA-Sept-14-2016",
  "https://www.scribd.com/document/328168241/So-Hee-Lee-A044-726-322-BIA-Sept-30-2016",
  "https://www.scribd.com/document/328167479/Jose-Marildo-Goncalves-A206-278-919-BIA-Sept-29-2016",
  "https://www.scribd.com/document/328167133/Jesus-Blanco-Acuna-A205-631-914-BIA-Sept-29-2016",
  "https://www.scribd.com/document/328166780/Sandra-Gabriela-Martinez-Reyes-A206-459-135-BIA-Sept-28-2016",
  "https://www.scribd.com/document/328166458/Juan-Alejandro-Herrera-Huerta-A200-709-587-BIA-Sept-26-2016",
  "https://www.scribd.com/document/328166107/Jose-Luis-Gutierrez-Rodriguez-A090-835-106-BIA-Sept-26-2016",
  "https://www.scribd.com/document/328164657/Flor-De-Maria-Gonzalez-Lopez-A024-732-563-BIA-Sept-21-2016",
  "https://www.scribd.com/document/328164407/Carlton-Anthony-Green-A300-321-132-BIA-Sept-19-2016",
  "https://www.scribd.com/document/328163707/R-L-O-H-AXXX-XXX-490-BIA-Sept-19-2016",
  "https://www.scribd.com/document/328163377/Mario-Ponce-Zacarias-A093-205-719-BIA-Sept-16-2016",
  "https://www.scribd.com/document/328162186/H-B-AXXX-XXX-608-BIA-July-26-2016",
  "https://www.scribd.com/document/328162086/Jose-Emilio-Alvarado-A208-090-238-BIA-June-2-2016",
  "https://www.scribd.com/document/328161984/J-A-L-AXXX-XXX-205-BIA-Oct-12-2016",
  "https://www.scribd.com/document/328158892/Hamza-Hussein-Idries-A200-355-063-BIA-Oct-12-2016",
  "https://www.scribd.com/document/328158630/Pablo-Aguilar-Garcia-A089-270-019-BIA-Oct-11-2016",
  "https://www.scribd.com/document/328158159/Luis-Angel-Monroy-Quijada-A206-770-841-BIA-Oct-7-2016",
  "https://www.scribd.com/document/328157753/Yessica-Mayeli-Colotl-Coyotl-A200-599-796-BIA-Oct-6-2016",
  "https://www.scribd.com/document/328157078/Victor-Manuel-Gonzalez-Hernandez-A206-412-832-BIA-Oct-4-2016",
  "https://www.scribd.com/document/328156619/Edwin-Nunez-Bencosme-A206-223-455-BIA-Oct-4-2016",
  "https://www.scribd.com/document/328156006/A-J-E-P-AXXX-XXX-498-BIA-Oct-3-2016",
  "https://www.scribd.com/document/324934077/I-M-E-G-AXXX-XXX-997-BIA-Aug-23-2016",
  "https://www.scribd.com/document/324927917/Oscar-Alejandro-Estrada-Rodriguez-A044-565-408-BIA-Sept-9-2016",
  "https://www.scribd.com/document/324927704/Duane-Grant-A099-743-627-BIA-Sept-9-2016",
  "https://www.scribd.com/document/324926826/Jhosimbher-Holar-Rodas-Mazariegos-A206-512-862-BIA-Sept-7-2016",
  "https://www.scribd.com/document/324926366/Jose-L-Zumel-A079-192-469-BIA-Sept-7-2016",
  "https://www.scribd.com/document/324809893/J-A-H-M-AXXX-XXX-192-BIA-Sept-1-2016",
  "https://www.scribd.com/document/324809637/L-D-M-G-AXXX-XXX-834-BIA-Aug-31-2016",
  "https://www.scribd.com/document/324808695/B-A-P-J-AXXX-XXX-863-BIA-Aug-31-2016",
  "https://www.scribd.com/document/324807163/Michael-Hiraldo-Arias-A057-150-954-BIA-Aug-31-2016",
  "https://www.scribd.com/document/324806886/Huy-Ngo-A042-489-147-BIA-Aug-31-2016",
  "https://www.scribd.com/document/324806442/Rodolfo-Huerta-Martinez-A205-294-294-BIA-Aug-30-2016",
  "https://www.scribd.com/document/324801317/Oswaldo-Raul-Herrera-Baltazar-A200-673-573-BIA-Aug-30-2016",
  "https://www.scribd.com/document/324800948/Lisseth-Del-Carmen-Montano-Montano-A200-119-434-BIA-Aug-30-2016",
  "https://www.scribd.com/document/324800678/Janet-Gertrudis-Casillas-A045-627-330-BIA-Aug-30-2016",
  "https://www.scribd.com/document/324800437/Antonin-Marholt-A208-506-084-BIA-Aug-29-2016",
  "https://www.scribd.com/document/324800143/Idalia-Esmeralda-Blanco-Mejia-A077-665-955-BIA-Aug-29-2016",
  "https://www.scribd.com/document/324233095/Gildardo-Daniel-Miranda-Duarte-A206-466-550-BIA-Aug-26-2016-pdf",
  "https://www.scribd.com/document/324232875/Cecilia-Elizabeth-Arias-Reyes-A206-234-448-BIA-Aug-25-2016",
  "https://www.scribd.com/document/324232664/S-L-A-C-AXXX-XXX-633-BIA-Aug-25-2016",
  "https://www.scribd.com/document/324232432/Yossi-Galimidi-A040-177-019-BIA-Aug-24-2016",
  "https://www.scribd.com/document/324232233/Carlos-Adrian-Gonzalez-Castaneda-A204-352-971-BIA-Aug-24-2016",
  "https://www.scribd.com/document/324230360/Einstein-Markov-Dauphin-A099-508-343-BIA-Aug-24-2016-pdf",
  "https://www.scribd.com/document/324229957/Jose-Osmar-de-Figueiredo-A200-029-818-BIA-Aug-23-2016",
  "https://www.scribd.com/document/324229685/Juan-Padilla-Lambert-A055-860-485-BIA-Aug-23-2016-pdf",
  "https://www.scribd.com/document/324229082/Erdwin-Banegas-Berrios-A208-058-470-BIA-Aug-22-2016",
  "https://www.scribd.com/document/324228804/Sara-Yamileth-Rodriguez-Garcia-A206-760-038-BIA-Aug-22-2016",
  "https://www.scribd.com/document/324228657/Felipe-Diaz-Santana-A206-349-898-BIA-Aug-18-2016",
  "https://www.scribd.com/document/324228382/Jose-Rogelio-Ortiz-Almanza-A205-517-790-BIA-Aug-17-2016",
  "https://www.scribd.com/document/324228010/Dario-Enrique-Espinoza-Leon-A060-814-718-BIA-Aug-17-2016",
  "https://www.scribd.com/document/322070142/E-D-R-AXXX-XXX-907-BIA-June-17-2016",
  "https://www.scribd.com/document/322069426/Maricela-Jacquelin-Madrid-Gomez-A206-254-226-BIA-Aug-11-2016",
  "https://www.scribd.com/document/322069248/Rafael-Gutierrez-Martinez-A205-648-373-BIA-Aug-10-2016",
  "https://www.scribd.com/document/322068762/Juan-Francisco-Lopez-A095-805-348-BIA-Aug-10-2016",
  "https://www.scribd.com/document/322068486/J-Marcos-Cisneros-Ramirez-A090-442-154-BIA-Aug-9-2016",
  "https://www.scribd.com/document/322068268/Jose-Elias-Aguilar-Gonzalez-A208-786-831-BIA-Aug-8-2016",
  "https://www.scribd.com/document/322068033/Gustavo-Ruiz-A206-862-055-BIA-Aug-8-2016",
  "https://www.scribd.com/document/322067668/Alexy-Nahum-Silva-Maradiaga-A206-481-827-BIA-Aug-8-2016",
  "https://www.scribd.com/document/322067432/Mario-Rivera-Flores-A094-451-158-BIA-Aug-8-2016",
  "https://www.scribd.com/document/322067136/Jorge-Luis-Solis-A088-590-199-BIA-Aug-8-2016",
  "https://www.scribd.com/document/322066971/Fermin-Becerra-Escobedo-A207-114-965-BIA-Aug-5-2016",
  "https://www.scribd.com/document/322066443/Roniel-Antonio-Capellan-Cruz-A060-955-082-BIA-Aug-4-2016",
  "https://www.scribd.com/document/322066165/Miguel-Alexander-Franco-Lara-A039-258-474-BIA-Aug-4-2016",
  "https://www.scribd.com/document/321494607/Eromarcos-Da-Silva-Ramos-A200-030-627-BIA-Aug-3-2016",
  "https://www.scribd.com/document/321494387/Armando-Marquez-Aguilar-A098-897-922-BIA-July-28-2016",
  "https://www.scribd.com/document/321494158/Santos-Francisco-Maldonado-A092-963-279-BIA-July-28-2016",
  "https://www.scribd.com/document/321492962/M-S-L-R-AXXX-XXX-188-BIA-July-25-2016",
  "https://www.scribd.com/document/321492821/Gabriel-Luna-Bricento-A205-270-408-BIA-July-25-2016",
  "https://www.scribd.com/document/321492635/Sergio-Amilcar-Lemus-Sola-A200-069-257-BIA-July-20-2016",
  "https://www.scribd.com/document/321492471/Jose-Manuel-Conejo-Caceres-A205-867-937-BIA-July-19-2016",
  "https://www.scribd.com/document/321492375/Rodolfo-Hernandez-Dominguez-A099-093-764-BIA-July-19-2016",
  "https://www.scribd.com/document/321492192/Jesus-Ramos-A091-679-605-BIA-July-19-2016",
  "https://www.scribd.com/document/321491971/Jose-Manuel-Zamora-Rosas-A018-509-728-BIA-July-18-2016",
  "https://www.scribd.com/document/321491819/Kewin-Arcenio-Martinez-Cruz-A077-794-158-BIA-July-15-2016",
  "https://www.scribd.com/document/321491636/Wildin-David-Guillen-Acosta-A206-799-049-BIA-July-15-2016",
  "https://www.scribd.com/document/321491399/Moises-Sanchez-Zaragoza-A077-273-744-BIA-July-14-2016",
  "https://www.scribd.com/document/321491188/Daniel-Antonio-Perez-A206-719-389-BIA-July-14-2016",
  "https://www.scribd.com/document/321490975/Tony-Alphonsus-A-Clarke-A086-931-015-BIA-July-13-2016",
  "https://www.scribd.com/document/318848254/Jessica-Lisseth-Lara-Merino-A097-318-914-BIA-July-12-2016",
  "https://www.scribd.com/document/318847952/Marco-Antonio-Esparza-Frausto-A089-981-198-BIA-July-12-2016",
  "https://www.scribd.com/document/318847641/Carmen-Elena-Hernandez-Rivera-A099-524-260-BIA-July-8-2016",
  "https://www.scribd.com/document/318846943/William-Suarez-Marquez-A208-920-275-BIA-July-7-2016",
  "https://www.scribd.com/document/318846707/Andres-Pascual-Manuel-A099-983-058-BIA-July-6-2016",
  "https://www.scribd.com/document/318846475/Elias-Galvan-A092-966-446-BIA-July-6-2016",
  "https://www.scribd.com/document/318845244/Jose-Maria-Delgado-Fernandes-A089-408-582-BIA-July-6-2016",
  "https://www.scribd.com/document/318845018/Ennio-Edgardo-Galvez-A095-006-115-BIA-July-5-2016",
  "https://www.scribd.com/document/318844625/Javier-Ortiz-Villegas-A200-975-995-BIA-July-1-2016",
  "https://www.scribd.com/document/318844376/Tunde-Olayinka-Awaye-A076-971-012-BIA-July-1-2016",
  "https://www.scribd.com/document/318844238/E-A-R-C-AXXX-XXX-125-BIA-June-30-2016",
  "https://www.scribd.com/document/318843682/Nelson-Rene-Bueso-Guerra-A077-799-300-BIA-June-30-2016",
  "https://www.scribd.com/document/318843150/Richard-Ricardo-Stewart-A047-169-818-BIA-June-30-2016",
  "https://www.scribd.com/document/318828163/Juvenal-Valdovinos-Lopez-A200-684-816-BIA-June-29-2016",
  "https://www.scribd.com/document/318759136/A-M-G-B-A206-773-739-BIA-June-28-2016",
  "https://www.scribd.com/document/318758884/J-R-S-AXXX-XXX-803-BIA-June-27-2016",
  "https://www.scribd.com/document/318758632/M-J-A-B-AXXX-XXX-307-BIA-June-27-2016",
  "https://www.scribd.com/document/318758375/Jose-Ramos-Menjivar-A098-979-537-BIA-June-27-2016",
  "https://www.scribd.com/document/318756723/Jorge-Luis-Gonzalez-Hernandez-A078-016-721-BIA-June-27-2016",
  "https://www.scribd.com/document/318756108/Ruben-Elizondo-Gonzalez-A034-076-469-BIA-June-27-2016",
  "https://www.scribd.com/document/318755951/Moses-Chukwujiekwu-Josiah-A204-053-513-BIA-June-23-2016",
  "https://www.scribd.com/document/318755563/Ajla-Vricic-A071-727-572-BIA-June-22-2016",
  "https://www.scribd.com/document/318753899/Christian-Trejo-Ortiz-A099-647-473-BIA-June-21-2016",
  "https://www.scribd.com/document/318753711/Antonio-Ivarra-A205-506-209-BIA-June-20-2016",
  "https://www.scribd.com/document/318753522/Azim-Aziz-Hooda-A205-132-314-BIA-June-20-2016",
  "https://www.scribd.com/document/318710279/Oneyda-Carolina-Sierra-Manca-A206-835-344-BIA-June-17-2016",
  "https://www.scribd.com/document/318710014/Yecsi-Folerk-Baires-Portillo-A201-185-570-BIA-June-17-2016",
  "https://www.scribd.com/document/318709804/Blanca-Lopez-Gutierrez-A099-294-885-BIA-June-17-2016",
  "https://www.scribd.com/document/318709490/Dimas-Aquielo-Mendosa-Sosa-A074-868-193-BIA-June-17-2016",
  "https://www.scribd.com/document/318707650/Anselmo-Carlos-Solis-A092-904-043-BIA-June-16-2016",
  "https://www.scribd.com/document/318707455/Bappi-Khan-A204-054-091-BIA-June-15-2016",
  "https://www.scribd.com/document/318590392/Fatimo-Eyitayo-Renke-Smith-Joda-A079-134-441-BIA-June-15-2016",
  "https://www.scribd.com/document/318589342/K-F-R-F-AXXX-XXX-533-BIA-June-14-2016",
  "https://www.scribd.com/document/318586699/Adriana-Elizabeth-Arevalo-Lopez-A098-121-311-BIA-June-14-2016",
  "https://www.scribd.com/document/318583614/Jonathon-Mora-Espinoza-A079-069-141-BIA-June-14-2016",
  "https://www.scribd.com/document/318583178/Mario-Echemendia-A024-666-407-BIA-June-14-2016",
  "https://www.scribd.com/document/318581937/K-A-C-Y-AXXX-XXX-560-BIA-June-13-2016",
  "https://www.scribd.com/document/318581173/P-A-S-S-AXXX-XXX-982-BIA-June-13-2016",
  "https://www.scribd.com/document/318580365/Ruben-Fabian-Andres-A205-604-280-BIA-June-13-2016",
  "https://www.scribd.com/document/318580052/Felipe-Mendez-Garcia-A089-818-449-BIA-June-13-2016",
  "https://www.scribd.com/document/316013300/Richard-Austin-Palmer-A061-494-802-BIA-June-9-2016",
  "https://www.scribd.com/document/316011891/Fernando-Salgado-Hernandez-A206-263-000-BIA-June-7-2016",
  "https://www.scribd.com/document/316008512/Claudia-Patricia-Claros-Garcia-A097-742-486-BIA-June-7-2016",
  "https://www.scribd.com/document/316007702/Sunilkumar-Ramabhal-Patel-A205-131-752-BIA-June-3-2016",
  "https://www.scribd.com/document/316007402/Josefa-Carrillo-Pablo-A202-097-908-BIA-June-2-2016",
  "https://www.scribd.com/document/316006610/K-N-M-T-AXXX-XXX-495-BIA-May-31-2016",
  "https://www.scribd.com/doc/316004968/Pascal-Innocent-A073-455-685-BIA-May-31-2016",
  "https://www.scribd.com/document/315706491/Arley-Max-Dos-Santos-A200-068-519-BIA-May-27-2016",
  "https://www.scribd.com/document/315706034/Maria-Cecilia-Peraza-de-Orellana-A206-841-467-BIA-May-26-2016",
  "https://www.scribd.com/doc/315705366/Kalolaine-Taunaholo-A073-974-004-BIA-May-25-2016",
  "https://www.scribd.com/doc/315703118/Antonio-Palomino-Herrera-A205-765-691-BIA-May-24-2016",
  "https://www.scribd.com/doc/315702963/Samuel-Rubio-Covarrubias-A045-189-208-BIA-May-24-2016",
  "https://www.scribd.com/document/315607319/Ali-Aminzadem-A028-218-004-BIA-May-20-2016",
  "https://www.scribd.com/document/315606582/Victor-Manuel-Lucero-A074-912-171-BIA-May-20-2016",
  "https://www.scribd.com/document/315606306/Kavita-Kirit-Malaviya-A076-959-176-BIA-May-20-2016",
  "https://www.scribd.com/document/315605358/Daniel-Suarez-Cortes-A093-233-989-BIA-May-20-2016",
  "https://www.scribd.com/document/315604161/Jose-Ignacio-Ayala-Garcia-A202-022-479-BIA-May-19-2016",
  "https://www.scribd.com/document/315602192/Francisco-Rodriguez-Trinidad-A044-892-640-BIA-May-18-2016",
  "https://www.scribd.com/document/313686166/George-Domena-Boateng-A096-874-721-BIA-May-13-2016",
  "https://www.scribd.com/document/313685922/Cevada-Azizyan-A044-428-950-BIA-May-13-2016",
  "https://www.scribd.com/document/313685727/Francisco-Ramirez-Ramos-A208-414-180-BIA-May-12-2016",
  "https://www.scribd.com/document/313685551/Rene-Wilfredo-Acevedo-Solis-A092-920-374-BIA-May-11-2016",
  "https://www.scribd.com/document/313274114/Esmerio-Padilla-Andrade-A208-443-549-BIA-May-9-2016",
  "https://www.scribd.com/document/313273798/Y-B-S-H-AXXX-XXX-187-BIA-May-6-2016",
  "https://www.scribd.com/document/313273500/Yesenia-Iveth-Pacheco-Figueroa-A205-733-029-BIA-May-6-2016",
  "https://www.scribd.com/document/313273183/Edilberto-Aguilar-Garcia-A206-472-568-BIA-May-4-2016",
  "https://www.scribd.com/document/313272826/Silvia-Altamirano-Altamirano-A071-488-092-BIA-May-4-2016",
  "https://www.scribd.com/document/312391527/L-H-A-AXXX-XXX-320-BIA-April-29-2016",
  "https://www.scribd.com/document/312388627/Marie-Boubbov-A091-779-680-BIA-April-29-2016",
  "https://www.scribd.com/document/312388271/Leili-Guerra-Villeda-A206-310-445-BIA-April-29-2016",
  "https://www.scribd.com/document/312388024/Juan-Jose-Martinez-Lopez-A046-274-029-BIA-April-29-2016",
  "https://www.scribd.com/document/312387791/Zayuri-Lopez-Lopez-A206-678-764-BIA-April-27-2016",
  "https://www.scribd.com/doc/312387598/Gerardo-Torres-Herrera-A094-379-889-BIA-April-26-2016",
  "https://www.scribd.com/document/312387132/Zakaria-Sheriff-A060-519-204-BIA-April-25-2016",
  "https://www.scribd.com/document/312386783/N-K-B-AXXX-XXX-425-BIA-March-31-2016",
  "https://www.scribd.com/document/311769998/Mauricio-Hernandez-Santiago-A206-706-192-BIA-April-22-2016",
  "https://www.scribd.com/document/311769757/Eligio-Trujillo-Mavie-A205-656-755-BIA-April-22-2016",
  "https://www.scribd.com/document/311769435/Carlos-Fleitas-Carrazana-A076-460-879-BIA-April-22-2016",
  "https://www.scribd.com/document/311769185/Benjamin-Valentin-Brito-A056-358-016-BIA-April-22-2016",
  "https://www.scribd.com/document/311748182/Evelyn-Mildred-Paz-Ochoa-A206-765-278-BIA-April-20-2016",
  "https://www.scribd.com/document/311745752/Leonardo-Estabillo-Guerpo-A091-849-187-BIA-April-20-2016",
  "https://www.scribd.com/document/311745469/Manuel-Ramos-Perez-A205-711-051-BIA-April-19-2016",
  "https://www.scribd.com/document/311745250/Erika-Yesenia-Acencio-Lopez-A200-065-661-BIA-April-19-2016",
  "https://www.scribd.com/document/311743623/Jose-Miguel-Iracheta-Gonzales-A208-279-037-BIA-April-18-2016",
  "https://www.scribd.com/document/311741839/Saul-Alberto-Baltazar-Salcedo-A205-934-715-BIA-April-18-2016",
  "https://www.scribd.com/document/311740136/Noemi-Espinal-A075-928-918-BIA-April-15-2016",
  "https://www.scribd.com/document/311739698/Kirit-Malaviya-A074-998-626-BIA-April-15-2016",
  "https://www.scribd.com/document/311739401/Hongli-Huang-A061-700-071-BIA-April-15-2016",
  "https://www.scribd.com/document/311738935/Bibek-Neupane-A206-424-962-BIA-April-13-2016",
  "https://www.scribd.com/document/311736830/H-R-M-AXXX-XXX-381-BIA-March-14-2016",
  "https://www.scribd.com/document/309725468/Luis-Garibay-Sanchez-A037-442-516-BIA-April-12-2016",
  "https://www.scribd.com/document/309725045/Maria-Isabel-Alfaro-Sorto-A206-759-599-BIA-April-11-2016",
  "https://www.scribd.com/document/309724769/Catalina-Ramirez-Pablo-A202-067-149-BIA-April-11-2016",
  "https://www.scribd.com/document/309724385/Henry-Noe-Hernandez-Diaz-A088-347-822-BIA-April-11-2016",
  "https://www.scribd.com/document/309723992/Jose-Francisco-Lopez-Fiallos-A200-216-689-BIA-April-8-2016",
  "https://www.scribd.com/doc/309723525/Claudionizio-Alves-Leal-A098-892-990-BIA-April-8-2016",
  "https://www.scribd.com/document/309723178/Olivio-Oliveira-Gondinho-A097-337-592-BIA-March-8-2016",
  "https://www.scribd.com/document/309718769/Nancy-Ocampo-Ulloa-A205-227-706-BIA-April-5-2016",
  "https://www.scribd.com/document/309718304/Howard-Alexander-Wilson-A205-165-410-BIA-April-5-2016",
  "https://www.scribd.com/document/309718011/Ojambert-Lamour-A056-495-361-BIA-April-5-2016",
  "https://www.scribd.com/document/309717530/Samuel-Perez-Hernandez-A200-155-042-BIA-April-4-2016",
  "https://www.scribd.com/document/309717053/Ricardo-Germaine-Elwin-A210-139-442-BIA-March-31-2016",
  "https://www.scribd.com/document/309716088/Ana-Maria-Herrera-Figueroa-A202-138-702-BIA-March-31-2016",
  "https://www.scribd.com/document/309715826/Alexander-Alvarez-Medina-A200-233-359-BIA-March-31-2016",
  "https://www.scribd.com/document/309715566/Christian-Rudolf-Pflugler-A095-721-949-BIA-March-31-2016",
  "https://www.scribd.com/document/309352475/Maria-Grave-Ortiz-A206-884-152-BIA-March-29-2016",
  "https://www.scribd.com/document/309350894/Mercedes-Castro-Antunes-A078-957-308-BIA-March-29-2016",
  "https://www.scribd.com/document/309347785/K-M-A-G-AXXX-XXX-053-BIA-March-24-2016",
  "https://www.scribd.com/document/309346742/Victor-Manuel-Carreiro-A203-159-537-BIA-March-24-2016",
  "https://www.scribd.com/document/309344196/Cuahutemoc-Mora-Coronoa-A205-403-701-BIA-March-23-2016",
  "https://www.scribd.com/document/309343370/Deuk-Su-Kim-A098-156-260-BIA-March-23-2016",
  "https://www.scribd.com/document/309342763/Rosa-Flor-Bonilla-Santos-A094-923-164-BIA-March-23-2016",
  "https://www.scribd.com/document/309311578/German-Torres-A074-291-150-BIA-March-22-2016",
  "https://www.scribd.com/document/309309876/Juliano-Cristiano-Pizzetti-A096-176-126-BIA-March-22-2016",
  "https://www.scribd.com/document/309309520/Iris-Yessenia-Reyes-Zavala-A206-775-262-BIA-March-22-2016",
  "https://www.scribd.com/document/309306579/Jairo-Carrera-Reyes-A074-259-776-BIA-March-21-2016",
  "https://www.scribd.com/document/309306110/Sergio-Maldonado-A078-567-541-BIA-March-21-2016",
  "https://www.scribd.com/document/309304196/Israel-Francisco-Lagunas-A200-154-729-BIA-March-21-2016",
  "https://www.scribd.com/document/307732936/Jose-Alexander-Nieto-Montenegro-A097-315-872-BIA-March-18-2016",
  "https://www.scribd.com/document/307731624/Jose-Orlando-Matute-Delarca-A205-723-875-BIA-March-18-2016",
  "https://www.scribd.com/document/307730674/Yuris-Ismary-Herrera-A098-559-063-BIA-April-8-2016",
  "https://www.scribd.com/document/307729425/Jorge-Alberto-Batres-Romero-A206-189-330-BIA-March-17-2016",
  "https://www.scribd.com/document/307728429/Patricia-Yanira-Guzman-Deras-A099-678-562-BIA-March-15-2016",
  "https://www.scribd.com/document/307726386/Milton-Mauricio-Medina-Hernandes-A098-993-235-BIA-March-15-2016",
  "https://www.scribd.com/document/307463850/Elcy-Leticia-Benitez-Sosa-A078-964-763-BIA-March-15-2016",
  "https://www.scribd.com/document/307462220/Juan-Antonio-Alvarenga-A206-233-826-BIA-March-15-2016",
  "https://www.scribd.com/document/307461306/Ramon-Jasso-Arangure-A056-333-337-BIA-March-15-2016",
  "https://www.scribd.com/document/307460704/Julian-Sebastian-Giraldo-Arteaga-A088-744-005-BIA-March-14-2016",
  "https://www.scribd.com/document/307459899/Calixto-Rene-Arana-Sanchez-A205-278-468-BIA-March-14-2016",
  "https://www.scribd.com/document/307459548/Hector-Fransua-Mach-Chavez-A206-888-919-BIA-March-14-2016",
  "https://www.scribd.com/document/307459325/Leonel-Hernandez-Medina-A206-015-042-BIA-March-14-2016",
  "https://www.scribd.com/document/305116665/A-J-J-AXXX-XXX-419-BIA-March-10-2016",
  "https://www.scribd.com/document/305116085/Faustino-Perez-Mote-A205-934-720-BIA-March-9-2016",
  "https://www.scribd.com/doc/305115604/Jasvir-Singh-A202-051-542-BIA-March-9-2016",
  "https://www.scribd.com/document/305115209/Paula-de-Jesus-Obando-de-Miranda-A095-724-673-BIA-March-9-2016",
  "https://www.scribd.com/document/305113140/Maria-Vega-Vargas-A079-189-914-BIA-March-9-2016",
  "https://www.scribd.com/document/305112228/Dimas-Santillan-A076-329-2911-BIA-March-9-2016",
  "https://www.scribd.com/document/304851692/Mynor-Sebastian-Tulul-Tum-A205-756-304-BIA-March-8-2016",
  "https://www.scribd.com/document/304851478/Ivan-Velazco-Castellano-A205-056-436-BIA-March-8-2016",
  "https://www.scribd.com/document/304851249/Victor-Manuel-Moreno-Leyva-A206-413-255-BIA-March-7-2016",
  "https://www.scribd.com/document/304204449/Sandra-Lisseth-Amaya-Banegas-A206-886-601-BIA-March-3-2016",
  "https://www.scribd.com/document/304204254/Julio-Lopez-De-Dios-A206-011-402-BIA-March-1-2016",
  "https://www.scribd.com/document/304204056/Adan-Ramirez-Rios-A088-658-419-BIA-Feb-29-2016",
  "https://www.scribd.com/document/304203912/Miguel-A-Hernandez-Xoqui-A205-526-157-BIA-Feb-26-2016",
  "https://www.scribd.com/document/304203664/Nickoyan-Nkrumah-Wallace-A041-654-413-BIA-Feb-26-2016",
  "https://www.scribd.com/document/304203483/Dario-Galvan-Gutierrez-A098-035-686-BIA-Feb-25-2016",
  "https://www.scribd.com/document/304203305/Pa-Ousman-Joof-A099-271-700-BIA-Feb-24-2016",
  "https://www.scribd.com/document/304203055/Miguel-Hernandez-Hernandez-A202-137-893-BIA-Feb-22-2016",
  "https://www.scribd.com/document/303212223/Zulfiqar-Ali-Mirza-A099-395-768-BIA-Feb-19-2016",
  "https://www.scribd.com/document/303211988/Donald-Jose-Nogura-Luna-A027-181-497-BIA-Feb-19-2016",
  "https://www.scribd.com/document/303211666/Michael-Tachie-Mensah-A205-497-725-BIA-Feb-18-2016",
  "https://www.scribd.com/document/303207682/Pedro-Dominguez-Trillo-A200-760-650-BIA-Feb-18-2016",
  "https://www.scribd.com/document/303206526/Javier-Alejandro-Rosales-de-La-Cruz-A088-806-933-BIA-Feb-18-2016",
  "https://www.scribd.com/document/303206204/Vitelia-Marilu-Gutierrez-Contrerras-A078-326-812-BIA-Feb-18-2016",
  "https://www.scribd.com/document/303205848/Rogelio-Picazo-Romero-A074-083-669-BIA-Feb-18-2016",
  "https://www.scribd.com/document/303205262/Jorge-Bolivar-Maldonado-Lopez-A206-881-099-BIA-Feb-17-2016",
  "https://www.scribd.com/document/303204885/Felix-Ramon-Contreras-Largaespada-A014-701-083-BIA-Feb-12-2016",
  "https://www.scribd.com/document/303204084/Maria-Guadalupe-Silva-Ramirez-A205-527-911-BIA-Feb-11-2016",
  "https://www.scribd.com/document/303203515/Oscar-Diaz-Medina-A096-188-908-BIA-Feb-11-2016",
  "https://www.scribd.com/document/303203095/Bernabe-Cun-Balan-A205-305-480-BIA-Feb-10-2016",
  "https://www.scribd.com/document/303199814/Randeep-Singh-Toor-A201-279-745-BIA-Feb-10-2016",
  "https://www.scribd.com/document/303198556/Adrian-Alvarado-Avila-A201-072-243-Feb-10-2016",
  "https://www.scribd.com/document/301193841/Francisco-Rodriguez-Trinidad-A044-892-640-BIA-Feb-24-2016",
  "https://www.scribd.com/document/299575904/Antonio-Medina-Leon-A090-919-097-BIA-Feb-8-2016",
  "https://www.scribd.com/document/299575696/Tenisini-Taufalele-A200-673-398-BIA-Feb-5-2016",
  "https://www.scribd.com/document/299575403/Renato-Andre-Romero-Ruiz-A204-442-747-BIA-Feb-4-2016",
  "https://www.scribd.com/document/299575081/Jose-Alfredo-Quijada-A092-041-082-BIA-Feb-4-2016",
  "https://www.scribd.com/document/299574825/Emerson-Roberto-Martinez-Fuentes-A206-718-042-BIA-Feb-3-2016",
  "https://www.scribd.com/document/299574109/Roberto-Estrada-Pacheco-A071-608-938-BIA-Feb-3-2016",
  "https://www.scribd.com/document/299284703/Adama-Kebe-A206-705-958-BIA-Jan-29-2016",
  "https://www.scribd.com/document/299284596/Mariano-Rafael-Paz-A202-070-379-BIA-Jan-29-2016",
  "https://www.scribd.com/document/299284439/Cesar-Miguel-Felix-Lugo-A201-147-761-BIA-Jan-29-2016",
  "https://www.scribd.com/document/299284235/Guillermo-Lopez-A096-616-343-BIA-Jan-29-2016",
  "https://www.scribd.com/document/299284071/Ezequiel-Eugenio-Castillo-Guzman-A097-573-118-BIA-Jan-28-2016",
  "https://www.scribd.com/document/299283872/Erika-Karem-Chumpitaz-A096-982-337-BIA-Jan-28-2016",
  "https://www.scribd.com/document/299283497/Carlos-Avalos-Mendez-A074-332-332-BIA-Jan-22-2016",
  "https://www.scribd.com/document/299283282/Blanca-Esmerelda-Cortez-Hernandez-A206-806-626-BIA-Jan-20-2016",
  "https://www.scribd.com/document/299012730/Ema-Gomez-Mitchell-A099-199-518-BIA-Jan-20-2015",
  "https://www.scribd.com/document/299012441/J-J-M-V-AXXX-XXX-700-BIA-Jan-19-2016",
  "https://www.scribd.com/document/299011923/Amabel-Amado-Alcivar-Salavarria-A043-910-846-BIA-Jan-19-2016",
  "https://www.scribd.com/document/298584103/Eder-Vagner-Ribeiro-A205-154-797-BIA-Jan-15-2016",
  "https://www.scribd.com/document/298583228/Ovidio-Perez-A094-470-918-BIA-Jan-15-2016",
  "https://www.scribd.com/document/298582939/Daniel-Lopez-Ramirez-A089-082-649-BIA-Jan-12-2016",
  "https://www.scribd.com/document/298582474/Wagner-Aneudis-Martinez-A043-447-800-BIA-Jan-12-2016",
  "https://www.scribd.com/document/298580589/Maria-Alarcon-Suarez-A201-071-840-BIA-Jan-11-2016",
  "https://www.scribd.com/doc/298579450/J-Santos-Serrano-Chavez-A200-564-867-BIA-Jan-8-2016",
  "https://www.scribd.com/document/298578696/Juana-Esperanza-Romero-Gonzalez-A097-739-329-BIA-Jan-8-2016",
  "https://www.scribd.com/document/298577881/Guardado-Osmin-A094-497-543-BIA-Jan-8-2016",
  "https://www.scribd.com/doc/298577693/Jose-Lopez-Guerra-A205-659-456-BIA-Jan-7-2016",
  "https://www.scribd.com/doc/298576814/Juan-Guillen-Arvizu-A091-425-613-BIA-Jan-7-2016",
  "https://www.scribd.com/document/298576454/Ygnacio-de-Leon-Mendez-A077-048-349-BIA-Jan-7-2016",
  "https://www.scribd.com/document/298576087/Deborah-Eloise-Bainton-A070-966-647-BIA-Jan-7-2016",
  "https://www.scribd.com/document/295341765/Miguel-Hugo-Vallejo-A096-201-718-BIA-Jan-6-2016",
  "https://www.scribd.com/document/295341531/Macjhay-Yagao-A044-946-445-BIA-Jan-6-2016",
  "https://www.scribd.com/document/295340208/Tafari-St-Aubyn-Lewis-A210-109-301-BIA-Jan-5-2016",
  "https://www.scribd.com/document/295158475/James-Edison-Bahamonde-Michilena-A088-069-238-BIA-Dec-31-2015",
  "https://www.scribd.com/document/295158125/Andres-Gonzalez-Murcia-A077-532-371-BIA-Dec-31-2015",
  "https://www.scribd.com/document/295157380/Victor-Manuel-Santamaria-A089-713-036-BIA-Dec-30-2015",
  "https://www.scribd.com/document/295150199/Jesus-Salvador-Valle-Anguiano-A073-863-636-BIA-Dec-30-2015",
  "https://www.scribd.com/document/295149326/Sylvia-Cundall-A075-776-185-BIA-Dec-29-2015",
  "https://www.scribd.com/document/295148797/Alejandro-Cruz-Cruz-A205-393-424-BIA-Dec-23-2015",
  "https://www.scribd.com/document/295148352/Sohrab-Rajebhai-Dhuka-A205-164-321-BIA-Dec-23-2015",
  "https://www.scribd.com/document/295147536/Jose-Manuel-Oliva-Ramirez-A206-700-849-BIA-Dec-22-2015",
  "https://www.scribd.com/document/295145659/Lorenzo-Sanchez-Nino-A090-090-672-BIA-Dec-18-2015",
  "https://www.scribd.com/document/295145027/Francisco-Garcia-Hernandez-A205-152-120-BIA-Dec-17-2015",
  "https://www.scribd.com/document/295144296/Sandra-Carolina-Alvarenga-Torres-A099-536-077-BIA-Dec-17-2015",
  "https://www.scribd.com/document/295143854/Glenda-Herrera-Xivir-A079-648-307-BIA-Dec-16-2015",
  "https://www.scribd.com/document/295141334/Aleyda-Ruiz-Bahena-A208-123-196-BIA-Dec-15-2015",
  "https://www.scribd.com/document/293858727/N-R-R-AXXX-XXX-938-BIA-Dec-14-2015",
  "https://www.scribd.com/document/293858479/Renato-de-Bartolo-A018-330-617-BIA-Dec-11-2015",
  "https://www.scribd.com/document/293858313/A-C-D-AXXX-XXX-646-BIA-Dec-11-2015",
  "https://www.scribd.com/document/293625197/Gaspar-Raymundo-Velasquez-A205-854-679-BIA-Dec-10-2015",
  "https://www.scribd.com/document/293625039/B-N-G-M-AXXX-XXX-323-BIA-Dec-9-2015",
  "https://www.scribd.com/document/293624677/Cesar-Alcantar-A077-520-504-BIA-Dec-9-2015",
  "https://www.scribd.com/document/293624063/Donald-Josue-Bone-Manzanares-A047-542-190-BIA-Dec-9-2015",
  "https://www.scribd.com/document/293623745/Marco-Antonio-Rodriguez-Gutierrez-A204-502-251-BIA-Dec-8-2015",
  "https://www.scribd.com/document/293623526/J-G-B-V-AXXX-XXX-018-BIA-Dec-7-2015",
  "https://www.scribd.com/document/293623270/Hiram-Abecardo-Gonzalez-Sarat-A205-164-431-BIA-Nov-23-2015",
  "https://www.scribd.com/document/293451148/Juan-Gabriel-Soria-Escobedo-A091-381-609-BIA-Dec-1-2015",
  "https://www.scribd.com/document/293450773/Jose-Eulalio-Chacon-A094-474-544-BIA-Nov-30-2015",
  "https://www.scribd.com/document/293450425/M-K-S-T-AXXX-XXX-711-BIA-Nov-30-2015",
  "https://www.scribd.com/document/293449928/Joel-Bernabe-Chinchabel-A205-152-029-BIA-Nov-30-2015",
  "https://www.scribd.com/document/293449650/Emilia-Hernandez-Gomez-A205-403-862-BIA-Nov-30-2015",
  "https://www.scribd.com/document/293448360/Felipe-Granados-Solano-A075-540-798-BIA-Nov-27-2015",
  "https://www.scribd.com/document/293447793/Adonay-Villatorto-A094-085-163-BIA-Nov-27-2015",
  "https://www.scribd.com/document/293447476/Roberto-Zitlatl-Perez-A200-143-102-BIA-Nov-27-2015",
  "https://www.scribd.com/document/293446550/Mohamed-Bouras-A089-601-787-BIA-Nov-24-2015",
  "https://www.scribd.com/document/293445044/Miguel-Angel-Martinez-Lopez-A200-243-233-BIA-Nov-24-2015",
  "https://www.scribd.com/document/293444660/Seidi-Elda-Sandoval-Landero-A088-056-174-BIA-Nov-20-2015",
  "https://www.scribd.com/document/293443575/Jose-Laurentino-Vasquez-Chavez-A088-8778-376-BIA-Nov-20-2015",
  "https://www.scribd.com/document/291133737/Blanca-Estela-Martinez-Cruz-A206-912-104-BIA-Nov-19-2015",
  "https://www.scribd.com/document/291132912/Wendy-Mosqueda-A096-814-323-BIA-Nov-19-2015",
  "https://www.scribd.com/document/291132258/Jose-Guerrero-Soto-A091-225-150-BIA-Nov-17-2015",
  "https://www.scribd.com/document/291131809/Augustine-Kadiri-A095-322-597-BIA-Nov-16-2015",
  "https://www.scribd.com/document/291131589/Moones-Mellouli-A087-317-931-BIA-Nov-13-2015",
  "https://www.scribd.com/document/291131059/Juan-Agripin-Lecler-de-Jesus-A055-001-959-BIA-Nov-13-2015",
  "https://www.scribd.com/document/290748981/E-J-E-B-AXXX-XXX-122-BIA-Nov-13-2015",
  "https://www.scribd.com/document/290748750/Marco-Delgado-Lopez-A205-152-331-BIA-Nov-12-2015",
  "https://www.scribd.com/document/290748533/Omar-Arana-Romero-A208-155-509-BIA-Nov-10-2015",
  "https://www.scribd.com/document/290748309/M-Y-I-E-AXXX-XXX-979-BIA-Nov-10-2015",
  "https://www.scribd.com/document/290747835/Mirna-Molinas-Pena-A097-318-251-BIA-Nov-10-2015",
  "https://www.scribd.com/document/290747514/Carlos-Perez-Guerrero-A079-746-629-BIA-Nov-6-2015",
  "https://www.scribd.com/document/290747117/Lazaro-Randy-Diaz-Javech-A095-512-138-BIA-Nov-5-2015",
  "https://www.scribd.com/document/290746626/Cicilia-Y-Rivas-Mojica-A202-175-637-BIA-Nov-4-2015",
  "https://www.scribd.com/document/290080285/L-Y-O-B-AXXX-XXX-267-BIA-Nov-2-2015",
  "https://www.scribd.com/document/290079633/Luis-Alfredo-Castro-A200-226-899-BIA-Oct-29-2015",
  "https://www.scribd.com/document/290079091/Y-A-L-L-AXXX-XXX-594-BIA-Oct-29-2015",
  "https://www.scribd.com/document/290078842/C-C-D-J-A202-058-964-BIA-Oct-29-2015",
  "https://www.scribd.com/document/290078456/Amadou-Mockhtar-Dia-A093-433-947-BIA-Oct-29-2015",
  "https://www.scribd.com/document/290078270/Rolando-de-Jesus-Gregorio-Dias-A200-000-324-BIA-Oct-28-2015",
  "https://www.scribd.com/document/290077878/Kevin-Patricio-Damian-A061-342-980-BIA-Oct-27-2015",
  "https://www.scribd.com/document/290071119/Elias-Aguilar-Jacobo-A200-882-885-BIA-Oct-27-2015",
  "https://www.scribd.com/document/290070753/Sandra-Yaneth-Fuentes-Majano-A202-120-724-BIA-Oct-27-2015",
  "https://www.scribd.com/document/290069621/Sergio-Pedro-Villafana-A087-749-264-BIA-Oct-27-2015",
  "https://www.scribd.com/document/290068354/Haresh-Kumar-Vishanubhai-Patel-A202-063-738-BIA-Oct-26-2015",
  "https://www.scribd.com/document/290067875/Aracely-Del-Carmen-Mendoza-Robles-A095-724-672-BIA-Oct-26-2015",
  "https://www.scribd.com/document/290067189/Sergio-Gabriel-Raya-Dominguez-A043-488-730-BIA-Oct-22-2015",
  "https://www.scribd.com/document/290066823/Francisco-Manuel-Mata-Martinez-A044-108-993-BIA-Oct-22-2015",
  "https://www.scribd.com/document/289752430/Maricel-Tan-Reyes-A099-862-495-BIA-Oct-21-2015",
  "https://www.scribd.com/document/289752150/Damone-Howell-A099-250-547-BIA-Oct-21-2015",
  "https://www.scribd.com/document/289751751/Otoniel-Rincon-Velasquez-A089-284-279-BIA-Oct-21-2015",
  "https://www.scribd.com/document/289751001/Tony-Alphonsus-A-Clarke-A086-931-015-BIA-Oct-21-2015",
  "https://www.scribd.com/document/289750478/Edmar-Petit-Seabra-A075-393-099-BIA-Oct-21-2015",
  "https://www.scribd.com/document/289748699/S-P-B-AXXX-XXX-947-BIA-Oct-20-2015",
  "https://www.scribd.com/document/289748401/Diana-Garcia-Vargas-A200-668-352-BIA-Oct-19-2015",
  "https://www.scribd.com/document/289747564/Dung-Huynh-A096-055-977-BIA-Oct-16-2015",
  "https://www.scribd.com/document/289493283/Harpreet-Singh-Multani-A206-885-165-BIA-Oct-15-2015",
  "https://www.scribd.com/document/289492938/Ravi-Satulo-Laicer-A096-675-284-BIA-Oct-15-2015",
  "https://www.scribd.com/doc/289492669/Alan-Soares-Renaut-A095-285-899-BIA-Oct-15-2015",
  "https://www.scribd.com/document/286370971/Mario-Melgar-A200-550-222-BIA-Oct-14-2015",
  "https://www.scribd.com/document/286370097/Aleesha-Marshalle-Pinnock-A074-089-677-BIA-Oct-14-2015",
  "https://www.scribd.com/document/286353928/O-I-M-P-AXXX-XXX-085-BIA-Oct-8-2015",
  "https://www.scribd.com/document/286352595/Vitalina-Bautista-Perez-A202-079-302-BIA-Oct-7-2015",
  "https://www.scribd.com/document/286352135/Jose-Luis-Ortiz-Cabrera-A205-713-028-BIA-Oct-1-2015",
  "https://www.scribd.com/document/286350083/Jorge-Perez-A205-406-702-BIA-Oct-1-2015",
  "https://www.scribd.com/document/286349580/Perminus-Oteyo-Achoka-A096-131-218-BIA-Oct-1-2015",
  "https://www.scribd.com/document/286348996/M-A-J-AXXX-XXX-274-BIA-Sept-30-2015",
  "https://www.scribd.com/document/286347985/J-C-D-R-AXXX-XXX-705-BIA-Sept-29-2015",
  "https://www.scribd.com/document/286345370/Dawel-Rafael-Lantigua-A058-491-837-BIA-Sept-29-2015",
  "https://www.scribd.com/document/285939917/Noel-Chege-Micho-A079-302-034-BIA-Sept-28-2015",
  "https://www.scribd.com/document/285939450/Alphonse-Pierre-A047-900-205-BIA-Sept-25-2015",
  "https://www.scribd.com/document/285938690/Luis-Eduardo-Munoz-Castellon-A094-230-428-BIA-Sept-24-2015",
  "https://www.scribd.com/document/285938429/Josefa-Carrillo-Pablo-A202-097-908-BIA-Sept-21-2015",
  "https://www.scribd.com/document/285938149/Jenine-Tameka-Jones-A206-501-154-BIA-Sept-21-2015",
  "https://www.scribd.com/document/285239784/Hernan-Antonio-Reyes-A095-084-205-BIA-Sept-18-2015",
  "https://www.scribd.com/document/285238940/Hector-Alfonso-Ordonez-Marroquin-A078-947-716-BIA-Sept-17-2015",
  "https://www.scribd.com/document/285238129/Jose-Garcia-Garcia-A206-156-310-BIA-Sept-17-2015",
  "https://www.scribd.com/document/285236527/Victor-Hugo-Ortega-Cruz-A200-727-032-BIA-Sept-17-2015",
  "https://www.scribd.com/document/285235361/Martha-Patricia-Garcia-Castillo-A092-395-313-BIA-Sept-17-2015",
  "https://www.scribd.com/document/285234285/Shahin-Emamian-Rasekh-A098-336-382-BIA-Sept-16-2015",
  "https://www.scribd.com/document/285232477/M-R-G-AXXX-XXX-922-BIA-Sept-16-2015",
  "https://www.scribd.com/document/285230390/Juan-Emigdio-Giron-A060-304-016-BIA-Sept-14-2015",
  "https://www.scribd.com/document/285229528/Raymundo-Ramirez-Sanchez-A099-621-267-BIA-Sept-14-2015",
  "https://www.scribd.com/document/285228863/Radamez-Sanchez-Luciano-A098-373-359-BIA-Sept-14-2015",
  "https://www.scribd.com/document/285228335/M-D-D-C-AXXX-XXX-677-BIA-Sept-14-2015",
  "https://www.scribd.com/document/285227720/Ernest-Antwi-Asamoah-A087-310-643-BIA-Sept-11-2015",
  "https://www.scribd.com/document/285227167/Sung-Pan-Liu-A095-704-688-BIA-Sept-11-2015",
  "https://www.scribd.com/document/281355815/A-M-R-D-AXXX-XXX-596-BIA-Sept-10-2015",
  "https://www.scribd.com/document/281337766/R-D-D-K-AXXX-XXX-728-BIA-Sept-1-2015",
  "https://www.scribd.com/document/281177982/Yoselin-Edith-Vasquez-Godinez-A206-767-922-BIA-Sept-10-2015",
  "https://www.scribd.com/document/281177192/Ramiro-Bravo-Nolasco-A205-854-686-BIA-Sept-10-2015",
  "https://www.scribd.com/document/281176411/Neil-Williams-Ulli-A205-713-938-BIA-Sept-10-2015",
  "https://www.scribd.com/document/281175403/Dickson-Kwesi-Asamoah-A086-973-978-BIA-Sept-10-2015",
  "https://www.scribd.com/document/281173112/Luis-Reyes-Rojo-A201-185-006-BIA-Sept-8-2015",
  "https://www.scribd.com/doc/281172224/Dorcas-Nangila-Wanyonyi-A097-683-041-BIA-Sept-4-2015",
  "https://www.scribd.com/document/281171466/Luzvenia-Cortina-A046-870-073-BIA-Sept-4-2015",
  "https://www.scribd.com/document/281170813/Iris-Jeanette-Benites-Maurisio-A099-679-349-BIA-Sept-3-2015",
  "https://www.scribd.com/document/281169687/Charles-Borromeo-Ajaelu-A058-739-058-BIA-Sept-3-2015",
  "https://www.scribd.com/document/281168186/M-E-P-C-AXXX-XXX-389-BIA-Sept-2-2015",
  "https://www.scribd.com/document/281166973/Raquel-Luzia-Ferreira-Pereira-A099-202-663-BIA-Sept-2-2015",
  "https://www.scribd.com/document/281166283/Orlando-Antonio-Bonilla-Molina-A094-246-276-BIA-Sept-2-2015",
  "https://www.scribd.com/document/281165090/Baltazar-Bucio-Avalos-A088-721-069-BIA-Sept-2-2015",
  "https://www.scribd.com/document/281164184/G-K-AXXX-XXX-758-BIA-Sept-2-2015",
  "https://www.scribd.com/document/281163438/Mario-A-Perez-Perez-A206-550-580-BIA-Aug-31-2015",
  "https://www.scribd.com/document/281162529/S-L-AXXX-XXX-361-BIA-Aug-31-2015",
  "https://www.scribd.com/document/281160790/Mauricio-Jose-Gomez-Argueta-A091-261-217-BIA-Aug-18-2015",
  "https://www.scribd.com/document/278127324/Julio-Mujica-A090-373-986-BIA-Aug-28-2015",
  "https://www.scribd.com/document/278125536/Samuel-Rico-Tarin-A078-681-106-BIA-Aug-28-2015",
  "https://www.scribd.com/document/278124055/Francis-Ndegwa-Nyaga-A205-286-616-BIA-Aug-27-2015",
  "https://www.scribd.com/document/278123271/Maria-Nicolasa-Ayala-Guevara-A077-444-128-BIA-Aug-27-2015",
  "https://www.scribd.com/document/278119585/Hector-Manuel-Sanchez-Garcia-A205-632-066-BIA-Aug-26-2015",
  "https://www.scribd.com/document/278118455/Oscar-Emilio-Contreras-Saldana-A089-535-565-BIA-Aug-25-2015",
  "https://www.scribd.com/document/278105015/Martha-Romero-Duran-A201-221-138-BIA-Aug-24-2015",
  "https://www.scribd.com/document/278102791/Salvador-Perez-Sandoval-A205-131-868-BIA-Aug-20-2015",
  "https://www.scribd.com/document/278101802/Carlos-Soto-Martinez-A205-450-529-BIA-Aug-19-2015",
  "https://www.scribd.com/document/278100916/Simran-Dholasania-A200-006-948-BIA-Aug-19-2015",
  "https://www.scribd.com/document/278098127/Jhonathan-Victoria-Javier-A059-303-967-BIA-Aug-19-2015",
  "https://www.scribd.com/document/278094377/Julio-Cesar-Delgado-Garcia-A205-570-293-BIA-Aug-18-2015",
  "https://www.scribd.com/document/278091283/Antony-Muthaka-Muhia-A086-914-490-BIA-Aug-14-2015",
  "https://www.scribd.com/document/274998422/Adrian-Valverde-Martinez-A200-569-794-BIA-Aug-11-2015",
  "https://www.scribd.com/document/274523419/Sergio-Flores-Gonzalez-A200-142-845-BIA-Aug-7-2015",
  "https://www.scribd.com/document/274522888/Rene-Juarez-Escobar-A205-850-513-BIA-Aug-5-2015",
  "https://www.scribd.com/document/274522542/Daniel-Duarte-Mendez-A205-208-430-BIA-Aug-5-2015",
  "https://www.scribd.com/document/274519627/Gustavo-Soto-Enriquez-A087-274-650-BIA-Aug-5-2015",
  "https://www.scribd.com/document/274519247/Caino-Kenaris-Reid-A204-697-163-BIA-Aug-4-2015",
  "https://www.scribd.com/document/274518776/Y-Y-C-AXXX-XXX-786-BIA-Aug-4-2015",
  "https://www.scribd.com/document/274517889/Jardan-Levin-A087-084-094-BIA-Aug-4-2015",
  "https://www.scribd.com/document/274517618/Kemol-Decosta-Bowman-A093-336-729-BIA-July-30-2015",
  "https://www.scribd.com/document/274516981/Jose-Angel-Parada-Montufar-A074-092-123-BIA-July-30-2015",
  "https://www.scribd.com/document/274516167/Bladimir-Morales-A029-376-366-BIA-Aug-14-2015",
  "https://www.scribd.com/document/274515829/Maria-Berlinda-Bonilla-Chacon-A096-179-693-BIA-July-24-2015",
  "https://www.scribd.com/document/274515137/Esvin-Escobar-Diaz-A201-238-048-BIA-July-24-2015",
  "https://www.scribd.com/document/274514587/Zsolt-Bara-A205-668-646-BIA-July-24-2015",
  "https://www.scribd.com/document/274512940/Wayne-Linton-Morris-A074-303-761-BIA-July-23-2015",
  "https://www.scribd.com/document/274512595/Rodolfo-Castillo-Lozano-A097-398-041-BIA-July-22-2015",
  "https://www.scribd.com/document/274512169/Bilal-Hassan-Dyook-A076-305-737-BIA-July-22-2015",
  "https://www.scribd.com/document/274428643/Miguel-Avila-Martinez-A200-561-097-BIA-July-21-2015",
  "https://www.scribd.com/document/274428365/Mohammed-Shaiful-Islam-A202-160-901-BIA-July-20-2015",
  "https://www.scribd.com/document/274427739/Vera-Sama-A076-581-488-BIA-July-17-2015",
  "https://www.scribd.com/document/274427332/Ramon-Zamorano-Mendez-A205-656-586-BIA-July-15-2015",
  "https://www.scribd.com/document/274427030/Leon-Christopher-Peters-A046-422-396-BIA-July-16-2015",
  "https://www.scribd.com/document/274426196/Jorge-Arturo-Cazarin-Cabrera-A205-500-769-BIA-July-16-2015",
  "https://www.scribd.com/document/274425834/Alkaly-Keita-A098-071-179-BIA-July-16-2015",
  "https://www.scribd.com/document/274425189/Jermaine-Horatio-Dussard-A037-332-033-BIA-July-16-2015",
  "https://www.scribd.com/document/274424238/Daniel-Alberto-Velasquez-Mejia-A078-972-091-BIA-July-15-2015",
  "https://www.scribd.com/document/274423897/Rene-Macario-Rodriguez-A205-273-172-BIA-July-15-2015",
  "https://www.scribd.com/document/274423255/W-E-P-M-AXXX-XXX-859-BIA-July-15-2015",
  "https://www.scribd.com/document/274422979/Mariana-Gonzalez-Hernandez-A205-324-473-BIA-July-14-2015",
  "https://www.scribd.com/document/274422527/Ivan-Torres-Resendiz-A200-626-963-BIA-July-14-2015",
  "https://www.scribd.com/document/274422113/Jonathan-Enrique-Silva-A036-404-266-BIA-July-14-2015",
  "https://www.scribd.com/document/274421291/Milton-Escamilla-Iraheta-A039-258-200-BIA-July-13-2015",
  "https://www.scribd.com/document/274420825/F-M-A-AXXX-XXX-656-BIA-July-13-2015",
  "https://www.scribd.com/document/274420366/Felix-Rosario-A075-930-968-BIA-July-10-2015",
  "https://www.scribd.com/document/274419844/Gabino-Rolando-Cortez-Talavera-A095-748-656-BIA-July-10-2015",
  "https://www.scribd.com/document/274419096/Agustin-Hernandez-Zuniga-A205-721-753-BIA-July-10-2015",
  "https://www.scribd.com/document/274417515/J-E-H-AXXX-XXX-133-BIA-June-29-2015",
  "https://www.scribd.com/document/274416940/Francisco-Peralta-A075-911-963-BIA-June-29-2015",
  "https://www.scribd.com/document/271673332/Jose-Juan-Rodriguez-Garcia-A044-581-069-BIA-May-6-2015",
  "https://www.scribd.com/document/271672670/Lidia-Maritza-Santos-Ardon-A073-904-816-BIA-Aug-27-2013",
  "https://www.scribd.com/document/271671394/Pedro-Igor-Duarte-A095-874-153-BIA-July-8-2015",
  "https://www.scribd.com/document/271671073/Socorro-Menendez-A078-111-837-BIA-July-8-2015",
  "https://www.scribd.com/document/271670811/Angel-Luna-Aguirre-A091-802-632-BIA-July-7-2015",
  "https://www.scribd.com/document/271669798/A-G-M-AXXX-XXX-127-BIA-July-2-2015",
  "https://www.scribd.com/document/271669168/Edulio-Elizondo-Zavaleta-A206-222-496-BIA-July-2-2015",
  "https://www.scribd.com/document/271668453/Zoila-Enunice-Elvir-Guevara-A098-589-427-BIA-July-2-2015",
  "https://www.scribd.com/document/271667571/Imran-Wahid-A047-700-704-BIA-July-1-2015",
  "https://www.scribd.com/document/271666822/Ivan-Alain-Herrera-Garcia-A205-153-898-BIA-June-30-2015",
  "https://www.scribd.com/document/271666168/J-S-R-AXXX-XXX-304-BIA-June-30-2015",
  "https://www.scribd.com/document/271665832/Pedro-Adalberto-Portillo-Portillo-A094-474-552-BIA-Jan-30-2015",
  "https://www.scribd.com/document/271665361/Arturo-Mandujano-Torres-A091-480-873-BIA-June-30-2015",
  "https://www.scribd.com/document/271665118/Sami-Dominguez-Vitela-A076-703-827-BIA-June-30-2015",
  "https://www.scribd.com/document/271354416/D-M-R-BIA-June-9-2015",
  "https://www.scribd.com/document/271353122/E-M-BIA-Feb-18-2015",
  "https://www.scribd.com/document/271045728/Aldo-Gustavo-Ramirez-Pedrosa-A205-700-672-BIA-June-26-2015",
  "https://www.scribd.com/document/271045279/Mai-The-Lu-A099-049-720-BIA-June-26-2015",
  "https://www.scribd.com/document/271044918/Dermith-Orlando-Barrientos-Vivas-A205-164-292-BIA-June-25-2015",
  "https://www.scribd.com/document/271044529/Miguel-Angel-Meraz-Puentes-A201-220-260-BIA-June-25-2015",
  "https://www.scribd.com/document/271044114/Carmen-Dinora-Vega-Palomino-A200-557-725-BIA-June-25-2015",
  "https://www.scribd.com/document/271043512/Noe-Govea-Recendiz-A206-841-626-BIA-June-24-2015",
  "https://www.scribd.com/document/271043086/Cesar-Herrera-Munoz-A097-708-351-BIA-June-24-2015",
  "https://www.scribd.com/document/271042519/Sumiko-Furui-A095-723-429-BIA-June-24-2015",
  "https://www.scribd.com/document/271042252/Benito-Peredo-Ramirez-A076-507-852-BIA-June-22-2015",
  "https://www.scribd.com/document/271041895/A-O-V-V-AXXX-XXX-740-BIA-June-19-2015",
  "https://www.scribd.com/document/271040634/J-F-AXXX-XXX-328-BIA-June-19-2015",
  "https://www.scribd.com/document/271040159/Gagik-Khalatyan-A078-665-900-BIA-June-19-2015",
  "https://www.scribd.com/document/271039747/Luis-Arturo-Redrovan-Quinteros-A087-513-649-BIA-June-18-2014",
  "https://www.scribd.com/document/271039362/J-S-P-AXXX-XXX-178-BIA-June-17-2015",
  "https://www.scribd.com/document/270098357/Angel-Rafael-Chan-Tapia-A205-466-970-BIA-June-16-2015",
  "https://www.scribd.com/document/270098145/Fernando-Flores-Maceda-A205-506-988-BIA-June-15-2015",
  "https://www.scribd.com/document/270097854/Juan-Esteban-Zapata-A203-248-502-BIA-June-15-2015",
  "https://www.scribd.com/document/270097597/Jessy-Nkechi-Ogbunefi-Kelly-A099-256-275-BIA-June-15-2015",
  "https://www.scribd.com/document/270096588/Patrick-Cadet-A041-584-694-BIA-June-15-2015",
  "https://www.scribd.com/document/270095263/Magda-Esperanza-Cordova-Ramirez-A205-485-321-BIA-June-12-2015",
  "https://www.scribd.com/document/270095048/Pedro-Sanchez-Virgil-A200-836-945-BIA-June-12-2015",
  "https://www.scribd.com/document/270094753/Culian-Feng-A077-920-583-BIA-June-12-2015",
  "https://www.scribd.com/document/270094364/Gurson-Oswald-Gourzong-A038-201-760-BIA-June-12-2015",
  "https://www.scribd.com/document/268858593/Bryan-Emmanuel-Raudales-Zuniga-A206-795-052-BIA-June-11-2015",
  "https://www.scribd.com/document/268858343/Tyron-Canute-Francis-A047-224-961-BIA-June-10-2015",
  "https://www.scribd.com/document/268845846/J-A-L-G-AXXX-XXX-971-BIA-June-11-2015",
  "https://www.scribd.com/document/268763019/M-G-M-M-AXXX-XXX-194-BIA-June-1-2015",
  "https://www.scribd.com/document/268761838/Darwin-Alfredo-Sanchez-Gonzalez-A200-775-544-BIA-June-9-2015",
  "https://www.scribd.com/document/268761575/Benigno-Uribe-Ramirez-A205-163-479-BIA-June-8-2015",
  "https://www.scribd.com/document/268761375/Ihab-Darwish-A029-878-318-BIA-June-2-2015",
  "https://www.scribd.com/document/268293839/Raul-Ruiz-Miramon-A093-467-134-BIA-June-1-2015",
  "https://www.scribd.com/document/268292933/Moises-Alfonso-Rodriguez-A205-458-035-BIA-May-29-2015",
  "https://www.scribd.com/document/268292312/Gurvinder-Kaur-A201-143-721-BIA-May-28-2015",
  "https://www.scribd.com/document/268290455/Jean-Andre-Aine-A091-359-621-BIA-May-28-2015",
  "https://www.scribd.com/document/268290086/Huyen-V-Nguyen-A076-127-741-BIA-May-28-2015",
  "https://www.scribd.com/document/268287900/Antonio-Stephenson-A041-457-044-BIA-May-28-2015",
  "https://www.scribd.com/document/268287603/Javin-Albert-Teekasingh-A041-064-293-BIA-May-28-2015",
  "https://www.scribd.com/document/268286522/Nicolas-Flores-Aragon-A205-789-923-BIA-May-26-2015",
  "https://www.scribd.com/document/268285515/O-M-L-AXXX-XXX-351-BIA-May-26-2015",
  "https://www.scribd.com/document/268284642/Norma-Angelica-Ponce-Arriaga-A205-131-985-BIA-May-22-2015",
  "https://www.scribd.com/document/268284277/Mukuch-Vahanyan-A097-109-203-BIA-May-21-2015",
  "https://www.scribd.com/document/268283731/Jose-Manuel-Zapata-Laguna-A090-916-088-BIA-May-21-2015",
  "https://www.scribd.com/document/268281908/J-A-G-AXXX-XXX-711-BIA-May-20-2015",
  "https://www.scribd.com/document/268281254/David-Aguinaga-Melendez-A200-759-135-BIA-May-19-2015",
  "https://www.scribd.com/document/268280918/Raonel-Rodriguez-Avila-A206-375-387-BIA-May-15-2015",
  "https://www.scribd.com/document/268280612/Domingo-Vallecillo-Navarro-A200-888-968-BIA-May-15-2015",
  "https://www.scribd.com/document/268279559/Narindra-Lall-A046-107-800-BIA-May-15-2015",
  "https://www.scribd.com/document/268091314/E-H-AXXX-XXX-689-BIA-May-20-2015",
  "https://www.scribd.com/document/266020216/Lesy-Erenio-Sancehz-Matute-A205-458-848-BIA-May-14-2015",
  "https://www.scribd.com/document/266019857/Francisco-Leiva-Jimenez-A094-375-896-BIA-May-14-2015",
  "https://www.scribd.com/document/266019305/Roberto-Javier-Blanco-Perez-A092-981-108-BIA-May-14-2015",
  "https://www.scribd.com/document/266018885/G-I-C-T-AXXX-XXX-791-BIA-May-12-2015",
  "https://www.scribd.com/document/266018409/R-S-P-AXXX-XXX-593-BIA-May-11-2015",
  "https://www.scribd.com/document/266017722/C-C-G-AXXX-XXX-045-BIA-May-11-2015",
  "https://www.scribd.com/document/266017301/Prakaschandra-Rameschchandra-Patel-A200-661-745-BIA-May-8-2015",
  "https://www.scribd.com/document/266016941/Pau-Soc-Chau-A028-009-220-BIA-May-8-2015",
  "https://www.scribd.com/document/266016494/Sara-Del-Carmen-Guzman-A202-125-309-BIA-May-7-2015",
  "https://www.scribd.com/doc/266016066/Devon-Anthony-Allen-A037-087-427-BIA-May-7-2015",
  "https://www.scribd.com/document/266015773/D-R-A-AXXX-XXX-935-BIA-May-6-2015",
  "https://www.scribd.com/document/266015196/Karla-de-Jesus-Alfaro-Martinez-A202-076-417-BIA-May-6-2015",
  "https://www.scribd.com/document/266013843/Mariano-Gonzalez-Martinez-A097-845-427-BIA-May-6-2015",
  "https://www.scribd.com/document/266013537/Israel-Sarmiento-Adamae-A070-932-703-BIA-May-6-2015",
  "https://www.scribd.com/document/265987752/Sandra-Alcantar-Vega-A205-717-119-BIA-May-5-2015",
  "https://www.scribd.com/document/265987349/Juan-Hinojosa-A098-239-282-BIA-May-5-2015",
  "https://www.scribd.com/doc/265986142/E-F-AXXX-XXX-802-BIA-May-5-2015",
  "https://www.scribd.com/document/265945788/Jose-Lopez-Martinez-A205-717-934-BIA-May-4-2015",
  "https://www.scribd.com/document/265944731/Jose-Alberto-Lopez-Ramirez-A087-417-602-BIA-May-4-2015",
  "https://www.scribd.com/document/265943445/Salome-Feehi-Lomo-Mensah-A096-710-971-BIA-May-1-2015",
  "https://www.scribd.com/doc/265943328/Salome-Feehi-Lomo-Mensah-A096-710-971-BIA-May-1-2015-pdf",
  "https://www.scribd.com/document/265784872/Lester-Emilio-Garay-Garcia-A099-240-872-BIA-April-30-2015",
  "https://www.scribd.com/document/265784546/Lewis-Nyayiemi-Rogito-A087-484-820-BIA-April-30-2015",
  "https://www.scribd.com/document/265783542/Jorge-Argenis-Velazquez-Garcia-A097-563-851-BIA-April-29-2015",
  "https://www.scribd.com/document/265782628/Claudia-Yareni-Medina-Martinez-A079-038-287-BIA-April-28-2015",
  "https://www.scribd.com/document/265778885/Gicely-Sarai-Flores-Velasquez-A205-277-572-BIA-April-24-2015",
  "https://www.scribd.com/document/265778079/Ma-Evelyn-Valdemor-A094-216-703-BIA-April-24-2015",
  "https://www.scribd.com/document/265777737/Alejandro-Vega-Morado-A205-207-060-BIA-April-23-2015",
  "https://www.scribd.com/document/265776445/Estraya-de-La-Paz-Hernandez-A205-994-000-BIA-April-22-2015",
  "https://www.scribd.com/document/265776185/Boris-Armando-Juarez-Hernandez-A200-122-340-BIA-April-22-2015",
  "https://www.scribd.com/document/265775066/Joyce-Jenrola-Davis-A087-297-576-BIA-April-21-2015",
  "https://www.scribd.com/document/265774264/Marco-Antonio-Cruz-Martinez-A070-726-184-BIA-April-21-2015",
  "https://www.scribd.com/document/265774105/Anthony-Martinez-Lopez-A205-920-662-BIA-April-20-2015",
  "https://www.scribd.com/document/265773714/E-A-P-AXXX-XXX-852-BIA-Apr-17-2015",
  "https://www.scribd.com/document/262590897/Rocael-Rodas-Ambrocio-A200-557-411-BIA-Apr-9-2015",
  "https://www.scribd.com/document/262590898/Marcin-Merchelski-A043-700-889-BIA-Apr-9-2015",
  "https://www.scribd.com/document/262590899/Jorge-Israel-Herrera-Lopez-A091-389-123-BIA-Apr-15-2015",
  "https://www.scribd.com/document/262590900/Samuel-Antonio-Rivera-A205-719-877-BIA-Apr-15-2015",
  "https://www.scribd.com/document/262590895/Saleh-Faisal-Ali-A095-917-975-BIA-Apr-15-2015",
  "https://www.scribd.com/document/262590896/Javier-Avalos-Camacho-A089-229-522-BIA-Apr-8-2015",
  "https://www.scribd.com/document/262525004/Noe-Asael-Serna-Lara-A201-070-135-BIA-Apr-2-2015",
  "https://www.scribd.com/document/262524726/Antonio-Ruiz-Avalos-A077-170-048-BIA-Apr-2-2015",
  "https://www.scribd.com/document/262524453/Juan-Jose-Diaz-Juarez-A205-496-491-BIA-Apr-1-2015",
  "https://www.scribd.com/document/262524080/J-F-L-K-AXXX-XXX-261-BIA-March-31-2015",
  "https://www.scribd.com/document/262523687/Zapata-Chavez-A200-227-250-BIA-March-30-2015",
  "https://www.scribd.com/document/262523463/V-M-B-B-AXXX-XXX-723-BIA-March-27-2015",
  "https://www.scribd.com/document/262078276/Juan-Martin-Ocegueda-A092-345-098-BIA-March-25-2015",
  "https://www.scribd.com/document/262076130/Miguel-Garcia-Olvera-A091-983-344-BIA-March-25-2015",
  "https://www.scribd.com/document/262075923/Juan-Salvador-Martinez-Polanco-A205-159-407-BIA-March-23-2015",
  "https://www.scribd.com/document/262075458/Laila-Shabudin-Maknojia-A205-115-978-BIA-March-20-2015",
  "https://www.scribd.com/document/262074269/Alma-Nelida-Martinez-A076-824-333-BIA-March-19-2015",
  "https://www.scribd.com/document/262072571/Ricardo-de-Hoyos-Neaves-A205-955-704-BIA-March-19-2015",
  "https://www.scribd.com/document/262072305/Joseff-Henry-Gargar-Te-A205-279-813-BIA-March-18-2015",
  "https://www.scribd.com/document/262071436/Anibal-de-Jesus-Fuentes-A094-762-542-BIA-March-18-2015",
  "https://www.scribd.com/document/262070970/Hoang-Van-Tran-A027-309-219-BIA-March-18-2015",
  "https://www.scribd.com/document/262070365/Alfredo-Pimentel-Aragon-A201-142-906-BIA-March-17-2015",
  "https://www.scribd.com/document/262069820/Luis-Aramis-Cuello-Alcantara-A017-726-072-BIA-March-17-2015",
  "https://www.scribd.com/document/259231601/Katerina-Solcova-A200-383-523-BIA-March-16-2015",
  "https://www.scribd.com/document/259230847/Daniel-B-Newman-A029-682-240-BIA-March-16-2015",
  "https://www.scribd.com/document/259230625/Koyode-Akinniyi-A093-014-174-BIA-March-12-2015",
  "https://www.scribd.com/document/259230276/Sixto-Delgado-A075-423-408-BIA-March-12-2015",
  "https://www.scribd.com/document/259195401/L-R-N-AXXX-XXX-278-BIA-March-10-2015",
  "https://www.scribd.com/document/259195000/Wilson-Humberto-Ismatul-Cordova-A206-563-666-BIA-March-10-2015",
  "https://www.scribd.com/document/259194536/Valery-Muriel-Perez-Toro-A200-204-318-BIA-March-10-2015",
  "https://www.scribd.com/document/259194297/Rodrigo-Vasquez-Lopez-A048-138-084-BIA-March-10-2015",
  "https://www.scribd.com/document/259193791/Rodrigo-Chavez-Tellez-A200-777-839-BIA-March-9-2015",
  "https://www.scribd.com/document/259193527/Jose-Matias-Pretzantizin-A097-535-298-BIA-March-9-2015",
  "https://www.scribd.com/document/259193173/Kamil-Cwalinski-A047-081-455-BIA-March-9-2015",
  "https://www.scribd.com/document/259192419/Felipe-Jimenez-A046-294-792-BIA-March-4-2015",
  "https://www.scribd.com/document/258904486/Prince-Walker-A059-968-087-BIA-March-3-2015",
  "https://www.scribd.com/document/258904199/Pekepeka-Laiseni-A205-272-741-BIA-Feb-27-2015",
  "https://www.scribd.com/document/258903799/Eber-Salgado-Gutierrez-A205-154-421-BIA-Feb-27-2015",
  "https://www.scribd.com/document/258903173/Juan-Carlos-Torres-De-Santiago-A089-474-810-BIA-Feb-27-2015",
  "https://www.scribd.com/document/258902910/Franklin-Chukwuma-Nwagbo-A040-079-914-BIA-Feb-27-2015",
  "https://www.scribd.com/document/258902642/Harvey-Neville-Thomas-A027-043-777-BIA-Feb-27-2015",
  "https://www.scribd.com/document/258902322/Marcelo-Nunez-Rodriguez-A076-516-026-BIA-Feb-25-2015",
  "https://www.scribd.com/document/258902015/Hardikkumar-Dipakkumar-Patel-A203-236-986-BIA-Feb-23-2015",
  "https://www.scribd.com/document/258901700/Mohammed-Jammeh-A205-274-106-BIA-Feb-23-2015",
  "https://www.scribd.com/document/258274254/Jose-Guillermo-Rios-Ventura-A090-470-341-BIA-Feb-20-2015",
  "https://www.scribd.com/document/258273675/Nikola-Stepanovic-A200-313-381-BIA-Feb-19-2015",
  "https://www.scribd.com/document/258273121/Binh-Ngoc-Trinh-A041-556-896-BIA-Feb-19-2015",
  "https://www.scribd.com/document/258272651/Maria-Antonia-Lugo-de-Esperance-A200-481-736-BIA-Feb-18-2015",
  "https://www.scribd.com/document/258272323/Daniel-Joseph-Scarry-A012-129-154-BIA-Feb-18-2015",
  "https://www.scribd.com/document/258272049/Augusto-Guzman-Chavez-A099-226-737-BIA-Feb-13-2015",
  "https://www.scribd.com/document/258271541/M-B-AXXX-XXX-672-BIA-Sept-25-2014",
  "https://www.scribd.com/document/256276264/Clayton-Hugh-Anthony-Stewart-A043-399-408-BIA-Feb-11-2015",
  "https://www.scribd.com/document/256276073/Edmundo-Dimas-Lopez-A076-799-635-BIA-Feb-10-2015",
  "https://www.scribd.com/document/256275694/Erick-Ricardo-Plascencia-A206-650-761-BIA-Feb-9-2015",
  "https://www.scribd.com/document/256275442/Estella-Ibonne-Arrendono-A200-760-314-BIA-Feb-9-2015",
  "https://www.scribd.com/document/256274629/Motilall-Persaud-A039-091-797-BIA-Feb-6-2015",
  "https://www.scribd.com/document/256274272/Antonio-Figueroa-A094-495-476-BIA-Feb-5-2015",
  "https://www.scribd.com/document/256273981/Jose-Antonio-Cue-A072-552-808-BIA-Feb-5-2015",
  "https://www.scribd.com/document/256273735/Maritza-Salcido-Rocha-A200-832-424-BIA-Feb-4-2015",
  "https://www.scribd.com/document/256273276/Anita-Janeth-Cubero-A074-771-617-BIA-Feb-4-2015",
  "https://www.scribd.com/document/255675035/Jane-Oroni-Moore-A099-816-638-BIA-Feb-3-2015",
  "https://www.scribd.com/document/255674817/Jhow-Willie-Calderon-A056-122-271-BIA-Jan-30-2015",
  "https://www.scribd.com/document/255674650/A-M-D-AXXX-XXX-416-BIA-Jan-30-2015",
  "https://www.scribd.com/document/255674076/Jose-Torres-Morales-A200-895-241-BIA-Jan-29-2015",
  "https://www.scribd.com/document/254828049/Andre-Luckman-Ibrahim-A097-680-747-BIA-Jan-28-2015",
  "https://www.scribd.com/document/254827488/S-B-AXXX-XXX-223-BIA-Jan-27-2015",
  "https://www.scribd.com/document/254827067/Luis-Muniz-A200-141-905-BIA-Jan-26-2015",
  "https://www.scribd.com/document/254826543/Saleh-Mohammed-Otaifah-A055-775-988-BIA-Jan-26-2015",
  "https://www.scribd.com/document/254825663/Patrick-Cadet-A041-584-694-BIA-Jan-23-2015",
  "https://www.scribd.com/document/254147134/Rellio-Emmanuel-Johnson-A096-825-391-BIA-Jan-22-2015",
  "https://www.scribd.com/document/254146960/Maria-Jose-Ilheus-A200-564-852-BIA-Jan-21-2015",
  "https://www.scribd.com/document/254146583/Ezequias-Enoc-Castro-Tolentino-A087-968-919-BIA-Jan-21-2015",
  "https://www.scribd.com/document/254146169/Irene-O-Atseyinku-A058-536-724-BIA-Jan-21-2015",
  "https://www.scribd.com/document/254145782/Jesus-Gregorio-Manzano-Ruiz-A206-407-386-BIA-Jan-20-2015",
  "https://www.scribd.com/document/254145429/Rajinder-Singh-A205-935-843-BIA-Jan-20-2015",
  "https://www.scribd.com/document/254115704/Perla-Adiatna-Benitez-Marban-A205-210-387-BIA-Jan-20-2015",
  "https://www.scribd.com/document/254113717/Luis-Sinchi-Sinchi-A206-779-736-BIA-Jan-16-2015",
  "https://www.scribd.com/document/254112247/Stylianos-Bitzos-A205-019-980-BIA-Jan-15-2015",
  "https://www.scribd.com/document/254111997/Lizabeth-Adriana-Flores-Mares-A200-580-864-BIA-Jan-15-2015",
  "https://www.scribd.com/document/254111746/Javier-O-Dominguez-Parra-A090-109-290-BIA-Jan-15-2015",
  "https://www.scribd.com/document/254103556/Mardin-Jeovany-Moz-Giron-A047-300-159-BIA-Jan-15-2015",
  "https://www.scribd.com/document/254102787/S-G-G-AXXX-XXX-730-BIA-Jan-2-2015",
  "https://www.scribd.com/document/253194153/Nicholas-Jermaine-Gumbs-A040-159-442-BIA-Jan-13-2015",
  "https://www.scribd.com/document/253193913/Axel-Omar-Angel-Zapet-A205-570-975-BIA-Jan-12-2015",
  "https://www.scribd.com/document/253193649/Juan-Martinez-Hernandez-A200-778-895-BIA-Jan-12-2015",
  "https://www.scribd.com/document/253193412/Dalton-Donovan-Miller-A043-721-663-BIA-Jan-12-2015",
  "https://www.scribd.com/document/252639453/Candido-Noyola-Montalvo-A23-006-885-BIA-Dec-5-2005",
  "https://www.scribd.com/document/252524377/Jose-Pedro-Reyes-Torres-A206-727-120-BIA-Jan-7-2015",
  "https://www.scribd.com/document/252524087/Rajesh-Chitherbhai-Makwana-A088-578-134-BIA-Jan-5-2015",
  "https://www.scribd.com/document/252523874/Sixto-Delgado-A075-423-408-BIA-Jan-2-2015",
  "https://www.scribd.com/document/252523205/Uffot-Emem-Williams-A079-011-172-BIA-Dec-31-2014",
  "https://www.scribd.com/document/252522867/Francisco-Javier-Soto-Cruz-A205-760-690-BIA-Dec-30-2014",
  "https://www.scribd.com/document/252522563/Gisela-Vargas-Albarracin-A097-168-593-BIA-Dec-30-2014",
  "https://www.scribd.com/document/252522256/Marcelino-Segundo-Sanchez-A200-248-313-BIA-Dec-29-2014",
  "https://www.scribd.com/document/252406811/Welington-Pereira-Mota-A200-133-947-BIA-Dec-23-2014",
  "https://www.scribd.com/document/252406301/Antonio-Carralero-A088-015-223-BIA-Dec-23-2014",
  "https://www.scribd.com/document/252405399/Arnold-Manuel-Warmels-A055-195-818-BIA-Dec-23-2014",
  "https://www.scribd.com/document/252405024/Jose-Alberto-Arias-Benitez-A041-267-639-BIA-Dec-23-2014",
  "https://www.scribd.com/document/252404355/Andrew-Aburu-Misumi-A094-075-414-BIA-Dec-22-2014",
  "https://www.scribd.com/document/252391611/Chhrey-Chea-A027-321-642-BIA-Dec-22-2014",
  "https://www.scribd.com/document/252390959/Alain-Patrana-A025-441-027-BIA-Dec-22-2014",
  "https://www.scribd.com/document/252390581/Eddie-Miguel-Centeno-Valle-A088-017-849-BIA-Dec-19-2014",
  "https://www.scribd.com/document/252390186/Rakesh-Kumar-Natvarbhai-Patel-A200-961-784-BIA-Dec-18-2014",
  "https://www.scribd.com/document/252389711/Guadalupe-Ramirez-Moran-A095-445-013-BIA-Dec-18-2014",
  "https://www.scribd.com/document/252389314/Rene-Baca-Chavez-A200-726-166-BIA-Dec-16-2014",
  "https://www.scribd.com/document/252078216/Jihyun-Kim-A089-245-712-BIA-Dec-15-2014",
  "https://www.scribd.com/document/252077905/Cecilio-Perez-Alvarez-A206-412-951-BIA-Dec-12-2014",
  "https://www.scribd.com/document/252077507/Muhamad-Yusuf-Luwaga-A097-750-414-BIA-Dec-12-2014",
  "https://www.scribd.com/document/252076946/Winsome-Elaine-Vassell-A091-146-392-BIA-Dec-12-2014",
  "https://www.scribd.com/document/252076632/Gurpreet-Singh-A060-605-541-BIA-Dec-12-2014",
  "https://www.scribd.com/document/252075116/Juan-Lopez-Lopez-A205-920-665-BIA-Dec-11-2014",
  "https://www.scribd.com/document/252074720/Edwin-Esnit-Umanzor-Cabrera-A036-411-408-BIA-Dec-11-2014",
  "https://www.scribd.com/document/252074316/Godofredo-Marroquin-Dimas-A098-793-008-BIA-Dec-10-2014",
  "https://www.scribd.com/document/252073879/Bernartita-Moses-A206-352-760-BIA-Dec-9-2014",
  "https://www.scribd.com/document/249798056/Sergio-Ponciano-Pacheco-A097-681-655-BIA-Dec-8-2014",
  "https://www.scribd.com/document/249796408/Pedro-Reyes-Guillen-A076-703-814-BIA-Dec-5-2014",
  "https://www.scribd.com/document/249795970/Nicholas-Anthony-Fairclough-A046-509-672-BIA-Dec-4-2014",
  "https://www.scribd.com/document/249795796/Jose-Fabio-Alpizar-A028-898-871-BIA-Dec-4-2014",
  "https://www.scribd.com/document/249795166/Richard-Letts-A201-110-997-BIA-Dec-1-2014",
  "https://www.scribd.com/document/249794221/Yadira-Chaparro-de-Martinez-A043-362-190-BIA-Nov-28-2014",
  "https://www.scribd.com/document/249691736/Cirilo-Hernandez-Pacheco-A200-280-956-BIA-Nov-26-2014",
  "https://www.scribd.com/document/249691038/Rajendrasinh-Babubhai-Makwana-A088-143-880-BIA-Nov-26-2014",
  "https://www.scribd.com/document/249689936/B-G-AXXX-XXX-481-BIA-Nov-25-2014",
  "https://www.scribd.com/document/249689296/Elmer-A-Mangandi-A094-408-360-BIA-Nov-25-2014",
  "https://www.scribd.com/document/249689064/Jose-Espana-A088-745-137-BIA-Nov-25-2014",
  "https://www.scribd.com/document/249688158/Francisco-Fabian-Medrano-A205-150-585-BIA-Nov-24-2014",
  "https://www.scribd.com/document/249687478/Mohammed-Kharbach-A095-427-618-BIA-Nov-24-2014",
  "https://www.scribd.com/document/249686763/Jesus-Felix-Carrillo-A087-765-342-BIA-Nov-24-2014",
  "https://www.scribd.com/document/249686449/Jose-Luis-Cruz-Ortiz-A205-308-999-BIA-Nov-21-2014",
  "https://www.scribd.com/document/249685822/Moses-Mendy-A096-598-212-BIA-Nov-21-2014",
  "https://www.scribd.com/document/249681278/Raul-Galvez-Valdovinos-A090-066-373-BIA-Nov-21-2014",
  "https://www.scribd.com/document/249681077/Francisco-Alberto-German-A045-320-817-BIA-Nov-21-2014",
  "https://www.scribd.com/document/248571293/Hector-Arteaga-Ochoa-A205-653-001-BIA-Nov-20-2014",
  "https://www.scribd.com/document/248570419/Kwame-Duodu-Opoku-A087-050-826-BIA-Nov-20-2014",
  "https://www.scribd.com/document/248569897/Gregorio-Chodakowsky-A072-228-232-BIA-Nov-20-2014",
  "https://www.scribd.com/document/248569536/Grace-Estrellado-A089-056-676-BIA-Nov-19-2014",
  "https://www.scribd.com/document/248569280/Christian-Gerardo-Garcia-Velazquez-A205-132-924-BIA-Nov-18-2014",
  "https://www.scribd.com/document/248568921/N-V-AXXX-XXX-550-BIA-Nov-18-2014",
  "https://www.scribd.com/document/248568429/Edwin-Alexander-Jandres-Aguiluz-A073-674-189-BIA-Nov-13-2014",
  "https://www.scribd.com/document/248319162/Homero-Berber-Castro-A200-140-768-BIA-Nov-10-2014",
  "https://www.scribd.com/document/248315111/Zeberio-Amwona-A096-129-849-BIA-Nov-10-2014",
  "https://www.scribd.com/document/246510182/Rigoberto-Rodriguez-Gil-A200-808-429-BIA-Nov-5-2014",
  "https://www.scribd.com/doc/246509676/Kirk-Gabriel-Laurencin-A060-176-754-BIA-Nov-5-2014",
  "https://www.scribd.com/document/246508889/Mario-Mancia-Lopez-A094-417-094-BIA-Nov-4-2014",
  "https://www.scribd.com/document/246508468/Jaime-Serrano-Rojas-A094-222-350-BIA-Nov-4-2014",
  "https://www.scribd.com/document/246508121/Milenko-Ilic-A079-845-453-BIA-Nov-4-2014",
  "https://www.scribd.com/document/246507532/Gabriel-Vidales-Moreno-A074-409-506-BIA-Nov-4-2014",
  "https://www.scribd.com/document/246506097/Benjamin-Milton-Geovani-Gale-Vargas-A205-718-884-BIA-Oct-31-2014",
  "https://www.scribd.com/document/246505098/Chungrim-Chung-A095-583-753-BIA-Oct-31-2014",
  "https://www.scribd.com/document/245653291/Ledvin-Isabel-Urbina-Padilla-A099-481-442-BIA-Oct-30-2014",
  "https://www.scribd.com/document/245652804/Jesus-Castillo-Farias-A201-185-255-BIA-Oct-29-2014",
  "https://www.scribd.com/document/245652313/Flora-Obushere-Amwayi-A205-133-952-BIA-Oct-28-2014",
  "https://www.scribd.com/document/245651894/Jose-Neftan-Fuentes-A089-955-126-BIA-Oct-28-2014",
  "https://www.scribd.com/document/245650906/Adrian-Demone-Malcolm-A026-552-740-BIA-Oct-28-2014",
  "https://www.scribd.com/document/245649188/J-Jose-Ceciliano-Rodriguez-A076-508-042-BIA-Oct-24-2014",
  "https://www.scribd.com/document/244995424/Byron-Rodriguez-Ramos-A089-933-665-BIA-Oct-23-2014",
  "https://www.scribd.com/document/244995315/Francisco-M-Jacobo-Rios-A099-101-953-BIA-Oct-21-2014",
  "https://www.scribd.com/document/244995214/Bertin-Toscano-Cervantes-A205-297-907-BIA-Oct-20-2014",
  "https://www.scribd.com/document/244995022/Zia-Ur-Rehman-Khan-A079-076-751-BIA-Oct-17-2014",
  "https://www.scribd.com/document/244994948/Nelson-Ventura-Ventura-A205-700-837-BIA-Oct-15-2014",
  "https://www.scribd.com/document/244994837/Orlando-N-Garcia-A046-041-451-BIA-Oct-15-2014",
  "https://www.scribd.com/document/244994712/Cesar-Liriano-A076-574-471-BIA-Oct-14-2014",
  "https://www.scribd.com/document/243335290/Robinson-W-Bordamonte-A035-979-158-BIA-Oct-8-2014",
  "https://www.scribd.com/document/243160510/J-E-C-AXXX-XXX-940-BIA-Aug-21-2014",
  "https://www.scribd.com/document/242844765/M-Y-R-H-AXXX-XXX-316-BIA-Aug-7-2014",
  "https://www.scribd.com/document/242546113/Armando-Cerda-Reyes-A029-923-675-BIA-Oct-7-2014",
  "https://www.scribd.com/document/242405237/Kwei-Genego-A047-376-145-BIA-Oct-2-2014",
  "https://www.scribd.com/document/242403931/Craig-Hanush-Thompson-A044-854-402-BIA-Oct-1-2014",
  "https://www.scribd.com/document/241725382/Mauro-Martinez-Reyes-A205-002-826-BIA-Sept-25-2014",
  "https://www.scribd.com/document/241725028/Karina-J-Alvarez-Fernandez-A046-941-394-BIA-Sept-23-2014",
  "https://www.scribd.com/document/241724464/Oscar-Ocampo-Mellida-A089-242-452-BIA-Sept-23-2014",
  "https://www.scribd.com/document/241724239/Alberto-Deleon-Suhul-A206-133-325-BIA-Sept-23-2014",
  "https://www.scribd.com/document/241723400/Eduardo-Enriquez-Alvarez-A070-731-338-BIA-Sept-22-2014",
  "https://www.scribd.com/document/241723160/Wiliam-Armando-Valle-Mendieta-A097-656-455-BIA-Sept-22-2014",
  "https://www.scribd.com/document/241454560/Marcelino-Gutierrez-Zavala-A206-077-351-BIA-Sept-19-2014",
  "https://www.scribd.com/document/241452779/Mi-Jung-Kim-A098-768-574-BIA-Sept-19-2014",
  "https://www.scribd.com/document/240847542/Alejandro-Perez-Serrano-A205-394-591-BIA-Sept-17-2014",
  "https://www.scribd.com/document/240846244/Hildegardo-Jaime-Davila-Ortiz-A034-324-746-BIA-Sept-17-2014",
  "https://www.scribd.com/document/240845783/Nana-Kwadwo-Badu-Nkrawire-A096-716-342-BIA-Sept-15-2014",
  "https://www.scribd.com/document/240844791/Frandy-Frederic-A205-503-631-BIA-Sept-10-2014",
  "https://www.scribd.com/document/240844242/Joel-Vera-Lopez-A205-154-644-BIA-Sept-8-2014",
  "https://www.scribd.com/document/240843313/Patrick-Uriah-Thaxter-A078-494-561-BIA-May-2-2014",
  "https://www.scribd.com/document/239443314/Carlos-Alberto-Zambrano-A088-741-973-BIA-Sept-5-2014",
  "https://www.scribd.com/document/239442509/Leandro-de-Souza-A098-891-606-BIA-Sept-2-2014",
  "https://www.scribd.com/document/239442148/Edwin-Rolando-Velasquez-Mejia-A201-241-986-BIA-Aug-29-2014",
  "https://www.scribd.com/document/239441752/Jose-Atilo-Reyes-Herrera-A200-586-985-BIA-Aug-29-2014",
  "https://www.scribd.com/document/239441275/Amalia-Georgios-Frazi-A046-880-198-BIA-Aug-29-2014",
  "https://www.scribd.com/document/238564421/Otoniel-Rincon-Velasquez-A089-284-279-BIA-Aug-27-2014",
  "https://www.scribd.com/document/238564216/Den-Yi-Lei-A047-597-940-BIA-Aug-27-2014",
  "https://www.scribd.com/document/238563779/Zulema-Espinoza-A087-679-061-BIA-Aug-26-2014",
  "https://www.scribd.com/document/238177224/Marcelo-Adriano-Batista-Izaias-A074-165-264-BIA-Aug-22-2014",
  "https://www.scribd.com/document/238176657/Mario-Enrique-Ramirez-Lainez-A205-236-187-BIA-Aug-21-2014",
  "https://www.scribd.com/document/238176091/Errol-Stafford-Brown-A079-095-690-BIA-Aug-19-2014",
  "https://www.scribd.com/document/238104401/J-G-T-AXX-XXX-698-BIA-Aug-19-2014",
  "https://www.scribd.com/document/238104114/Gilberto-Farias-Cler-A204-045-294-BIA-Aug-15-2014",
  "https://www.scribd.com/document/238103604/Francisco-Garcia-Hernandez-A096-420-438-BIA-Aug-15-2014",
  "https://www.scribd.com/document/237992124/Birgit-Esther-Urban-A091-397-634-BIA-Aug-13-2014",
  "https://www.scribd.com/document/237991852/Yeraldi-Magdalena-Rivas-Romero-A200-039-291-BIA-Aug-12-2014",
  "https://www.scribd.com/document/237991492/Maurilio-Perez-Cante-A087-459-483-BIA-Aug-11-2014",
  "https://www.scribd.com/document/237957430/Julio-David-Godinez-Morales-A205-296-691-BIA-Aug-7-2014",
  "https://www.scribd.com/document/237936911/Maria-Rosario-Rodriguez-Ramirez-A076-732-051-BIA-Aug-7-2014",
  "https://www.scribd.com/doc/237936573/Michael-Anthony-Gowie-A089-581-520-BIA-Aug-5-2014",
  "https://www.scribd.com/document/237871386/C-A-E-L-AXX-XXX-485-BIA-July-3-2014",
  "https://www.scribd.com/document/237870289/K-C-AXX-XXX-101-BIA-June-23-2014",
  "https://www.scribd.com/document/237868986/B-V-C-J-AXXX-XXX-939-BIA-Aug-30-2013",
  "https://www.scribd.com/document/237120295/Carlos-Gonzalez-Huertero-A070-682-789-BIA-July-31-2014",
  "https://www.scribd.com/document/237119636/Tomas-Ramirez-A026-843-980-BIA-July-31-2014",
  "https://www.scribd.com/document/237119159/Frederick-Ansa-Quartey-A096-642-201-BIA-July-30-2014",
  "https://www.scribd.com/document/237118759/Ahmed-Dahy-Abdelghany-Ahmed-A059-671-140-BIA-July-30-2014",
  "https://www.scribd.com/document/237118565/Victor-Manuel-Martinez-A029-084-542-BIA-July-30-2014",
  "https://www.scribd.com/document/236494383/Abdul-Hakeim-Thabet-Ahmed-A041-989-550-BIA-July-29-2014",
  "https://www.scribd.com/document/236491127/Servando-Banos-Pelaez-A200-146-812-BIA-July-28-2014",
  "https://www.scribd.com/document/236490635/Lorenzo-Perez-Flores-A095-419-451-BIA-July-28-2014",
  "https://www.scribd.com/document/236490132/Carlos-Pereira-Figueiredo-A200-550-034-BIA-July-23-2014",
  "https://www.scribd.com/document/236489821/Nicole-Natalie-Wilson-A087-397-965-BIA-July-23-2014",
  "https://www.scribd.com/document/236489177/Athikar-Moung-A079-196-338-BIA-July-18-2014",
  "https://www.scribd.com/document/236488650/Luca-Utzeri-A087-211-857-BIA-July-16-2014",
  "https://www.scribd.com/document/236488011/Paulino-Gaspar-Sanchez-A200-836-708-BIA-July-15-2014",
  "https://www.scribd.com/document/236487810/Jilmar-Espinoza-Rivera-A097-149-155-BIA-July-14-2014",
  "https://www.scribd.com/document/236145852/Jairo-Ferino-Sanchez-A094-216-521-BIA-July-11-2014",
  "https://www.scribd.com/document/236145608/Miguel-Vasquez-Jimenez-A205-655-834-BIA-July-10-2014",
  "https://www.scribd.com/document/236144700/Edwin-Alexander-Aquino-Artero-A206-432-493-BIA-July-8-2014",
  "https://www.scribd.com/document/236144305/O-G-C-M-A200-152-829-BIA-July-8-2014",
  "https://www.scribd.com/document/236143862/Maria-Jose-Reyes-A076-916-481-BIA-July-8-2014",
  "https://www.scribd.com/document/236142670/Miguel-Angel-Teves-A089-627-272-BIA-July-1-2014",
  "https://www.scribd.com/document/235825661/Pedro-Cordero-Cordero-A205-569-129-BIA-June-30-2014",
  "https://www.scribd.com/document/235824061/Crisoforo-Ramos-Najera-A200-945-567-BIA-June-27-2014",
  "https://www.scribd.com/document/235823173/Walter-Crimi-A095-434-965-BIA-June-27-2014",
  "https://www.scribd.com/document/235822594/Maria-Ludivia-Sanchez-Medina-A087-944-781-BIA-June-27-2014",
  "https://www.scribd.com/document/235821235/Camilo-Ernesto-Sanchez-Fajardo-A077-675-707-BIA-June-27-2014",
  "https://www.scribd.com/document/235820536/Hermelinda-Alonzo-de-Carrasco-A203-114-022-BIA-June-26-2014",
  "https://www.scribd.com/document/235820025/Cosijoeza-Andrade-Zamora-A200-580-631-BIA-June-25-2014",
  "https://www.scribd.com/document/235819710/Armando-Martinez-Vazquez-A089-856-873-BIA-June-25-2014",
  "https://www.scribd.com/document/235819208/Yury-Emmanuel-Morales-Abrego-A099-534-776-BIA-June-24-2014",
  "https://www.scribd.com/document/235818439/Jose-Eduardo-Montoya-Renderos-A095-020-672-BIA-June-23-2014",
  "https://www.scribd.com/document/235817725/Maria-Carmela-Flores-A205-136-055-BIA-June-19-2014",
  "https://www.scribd.com/document/235816841/Mirna-Isabel-Sorto-Viera-A076-316-387-BIA-June-19-2014",
  "https://www.scribd.com/document/235816618/Pankajkumar-Patel-A088-152-759-BIA-June-18-2014",
  "https://www.scribd.com/document/235816246/Necitas-Mallari-Dangan-A205-285-545-BIA-June-16-2014",
  "https://www.scribd.com/document/235815956/Juan-Manuel-Martinez-Montoya-A201-234-395-BIA-June-16-2014",
  "https://www.scribd.com/document/235462344/Hugo-Lorenzo-Cabrera-A200-610-998-BIA-June-13-2014",
  "https://www.scribd.com/document/235462134/Jose-Rubesindo-Rivera-Andrade-A094-458-895-BIA-June-13-2014",
  "https://www.scribd.com/document/235461632/Roldimy-Montinar-A025-454-851-BIA-June-13-2014",
  "https://www.scribd.com/document/235461371/Fernando-Saenz-Ledesma-A024-938-852-BIA-June-13-2014",
  "https://www.scribd.com/document/235460376/Luisa-Margarita-Cruz-Molina-A077-048-260-BIA-June-11-2014",
  "https://www.scribd.com/document/235459839/Johana-Altagracia-Peguero-A079-128-590-BIA-June-10-2014",
  "https://www.scribd.com/document/235293387/Taylor-Jose-Branco-Antonio-A075-795-143-BIA-June-10-2014",
  "https://www.scribd.com/document/235292998/Stanley-Anozie-Obi-A210-114-098-BIA-June-9-2014",
  "https://www.scribd.com/document/235292790/Fegauiai-Gaumatau-Neemia-A096-214-416-BIA-June-9-2014",
  "https://www.scribd.com/document/235292377/Carlos-David-Meza-A092-470-765-BIA-June-9-2014",
  "https://www.scribd.com/document/235292210/Mahmoud-Nabeel-Hamdan-A087-240-107-BIA-June-9-2014",
  "https://www.scribd.com/document/235291851/Carlos-Ermas-Aguilera-Martinez-A205-074-111-BIA-June-6-2014",
  "https://www.scribd.com/document/235291373/Dilipkumar-Dashrathbhai-Patel-A200-939-111-BIA-June-6-2014",
  "https://www.scribd.com/document/235291107/Marvin-Raul-Morales-Mateo-A087-681-944-BIA-June-5-2014",
  "https://www.scribd.com/document/235290044/Ramon-Garcia-Fonseca-A075-535-094-BIA-June-5-2014",
  "https://www.scribd.com/document/234901557/Leslie-Onyesoh-A205-462-342-BIA-June-4-2014",
  "https://www.scribd.com/document/234901098/Ruven-Lopez-Olvera-A075-244-372-BIA-June-4-2014",
  "https://www.scribd.com/document/234900742/Javier-Harnandez-Roderiguez-A037-518-812-BIA-June-4-2014",
  "https://www.scribd.com/document/234900186/Luis-Manuel-Pacheco-Da-Costa-A036-049-783-BIA-June-4-2014",
  "https://www.scribd.com/document/234696612/Mirkhond-Mirorifovich-Kabilov-A205-379-500-BIA-June-3-2014",
  "https://www.scribd.com/document/234696276/Irma-Yadira-Murillo-Ayala-A078-964-772-BIA-June-3-2014",
  "https://www.scribd.com/document/234695820/Simon-Brambila-Serrano-A200-557-577-BIA-June-2-2014",
  "https://www.scribd.com/document/234695405/Elena-Abella-A087-184-034-BIA-June-2-2014",
  "https://www.scribd.com/document/234694859/Gustavo-Barba-Arreaga-A077-772-631-BIA-June-2-2014",
  "https://www.scribd.com/document/234124912/Karla-Figueroa-Briceno-A095-084-539-BIA-May-30-2014",
  "https://www.scribd.com/document/234124127/Elocastico-Guillermo-Pacas-A094-189-099-BIA-May-30-2014",
  "https://www.scribd.com/document/234123443/Valerie-Lee-Maedgen-A093-407-562-BIA-May-30-2014",
  "https://www.scribd.com/document/233345410/Juan-Martinez-Alvarez-A200-759-323-BIA-May-29-2014",
  "https://www.scribd.com/document/233344516/Joan-Mwaniki-A200-224-423-BIA-May-29-2014",
  "https://www.scribd.com/document/233343581/Patrick-Kipkurui-Rotich-Kemboi-A097-683-060-BIA-May-29-2014",
  "https://www.scribd.com/document/233343159/J-German-Benitez-Lopez-A092-298-255-BIA-May-29-2014",
  "https://www.scribd.com/document/233257185/Ricardo-Fonseca-Haro-A097-369-036-BIA-May-27-2014",
  "https://www.scribd.com/document/233208464/Oligario-Tonche-Medina-A091-767-234-BIA-May-27-2014",
  "https://www.scribd.com/document/233206795/Dale-Anderson-Telesford-A079-020-706-BIA-May-27-2014",
  "https://www.scribd.com/document/233206094/Frank-Meyer-Flores-Rodriguez-A098-941-790-BIA-May-23-2014",
  "https://www.scribd.com/document/232313781/Jorge-Hernandez-Hernandez-A045-582-968-BIA-May-20-2014",
  "https://www.scribd.com/document/232311912/Brett-Richard-Godke-A059-435-050-BIA-May-16-2014",
  "https://www.scribd.com/document/232310503/Mohammad-M-Qatanani-A076-133-969-BIA-May-13-2014",
  "https://www.scribd.com/document/232306814/Frankie-Eduardo-Rendon-Zambrano-A206-037-021-BIA-May-12-2014",
  "https://www.scribd.com/document/232306129/Richard-Alonzo-Mullins-McBride-A043-563-331-BIA-May-8-2014",
  "https://www.scribd.com/doc/232305281/Julius-Ochieng-Olola-A058-982-193-BIA-May-6-2014",
  "https://www.scribd.com/document/232304226/Juan-Ignacio-Ruela-A077-485-879-BIA-May-5-2014",
  "https://www.scribd.com/document/232301640/Benjamin-Ayemoba-Ayebae-A073-602-922-BIA-May-1-2014",
  "https://www.scribd.com/document/232157722/Nyaken-Ansumana-Gamanga-A077-615-717-BIA-Apr-29-2014",
  "https://www.scribd.com/document/232156827/Clarence-Alexander-Fritz-A079-089-179-BIA-Apr-28-2014",
  "https://www.scribd.com/document/232155848/Roberto-Mike-Menelas-A047-031-788-BIA-Apr-28-2014",
  "https://www.scribd.com/document/232154734/David-Kirwa-Kurgat-A089-003-420-BIA-Apr-25-2014",
  "https://www.scribd.com/document/232154170/Justo-Rojop-Hernandez-A205-147-894-BIA-Apr-23-2014",
  "https://www.scribd.com/document/232150797/Dominga-M-Garcia-Javier-A077-599-243-BIA-Apr-11-2014",
  "https://www.scribd.com/document/232150249/Iris-Jeanette-Benites-Maurisio-A099-679-349-BIA-Apr-11-2014",
  "https://www.scribd.com/document/230981052/Deborah-Ann-Romero-A017-176-264-BIA-June-10-2014",
  "https://www.scribd.com/document/230242084/B-J-G-AXXX-XXX-333-BIA-May-29-2014",
  "https://www.scribd.com/document/229570601/Manuel-Lopez-Rivera-A089-235-276-BIA-June-10-2014",
  "https://www.scribd.com/document/229245579/Rebecca-Harris-A096-201-703-BIA-June-5-2014",
  "https://www.scribd.com/document/229242161/Rebecca-Bamfo-A096-201-703-BIA-June-5-2014",
  "https://www.scribd.com/document/228447860/G-M-H-AXXX-XXX-060-BIA-May-29-2014",
  "https://www.scribd.com/document/225423441/Roberson-Joseph-A078-360-606-BIA-Nov-18-2013",
  "https://www.scribd.com/document/222926623/Samuel-John-Philip-Lloyd-A029-971-939-BIA-Apr-28-2014",
  "https://www.scribd.com/document/222884792/Sung-Min-Hong-A097-143-991-BIA-Apr-29-2014",
  "https://www.scribd.com/document/220722633/Juan-Carlos-Clase-A043-986-617-BIA-Apr-25-2014",
  "https://www.scribd.com/document/218676903/Jose-Armando-Cruz-A087-241-021-BIA-Apr-9-2014",
  "https://www.scribd.com/document/218674940/Luis-Alonso-Castillo-A094-305-773-BIA-Apr-9-2014",
  "https://www.scribd.com/document/218673550/J-G-AXX-XXX-110-BIA-Apr-9-2014",
  "https://www.scribd.com/document/218672524/Ramone-Borrome-A044-824-479-BIA-Apr-7-2014",
  "https://www.scribd.com/document/217287837/David-Antonio-Quezada-Gonzalez-A099-539-214-BIA-Apr-4-2014",
  "https://www.scribd.com/document/217285802/So-Ann-Taneisha-Morgan-A099-166-966-BIA-Apr-3-2014",
  "https://www.scribd.com/document/217282391/Jermaine-St-Aubin-Foster-A088-444-744-BIA-Apr-3-2014",
  "https://www.scribd.com/document/217281251/Santino-Fabian-Alarcon-Gomez-A201-227-554-BIA-Apr-2-2014",
  "https://www.scribd.com/document/217279935/Daniel-Teye-Lowor-A093-460-067-BIA-Apr-2-2014",
  "https://www.scribd.com/document/217278769/Elena-Hernandez-Hernandez-A074-571-777-BIA-Mar-31-2014",
  "https://www.scribd.com/document/217278353/Luis-Miguel-Ramirez-Moz-A072-377-892-BIA-Mar-31-2014",
  "https://www.scribd.com/document/217276223/Vladimir-Perez-Santana-A045-748-290-BIA-Mar-31-2014",
  "https://www.scribd.com/document/216811151/Eduardo-Gomez-Jurado-A090-764-102-BIA-Mar-28-2014",
  "https://www.scribd.com/document/216807754/Maciel-Carrillo-Nunez-A200-226-594-BIA-Mar-28-2014",
  "https://www.scribd.com/document/216805482/Sang-Han-Yang-A091-254-530-BIA-Mar-28-2014",
  "https://www.scribd.com/document/216803670/Lionel-Ramos-Chavez-A024-255-707-BIA-Mar-28-2014",
  "https://www.scribd.com/document/216802429/B-S-AXX-XXX-454-BIA-Mar-27-2014",
  "https://www.scribd.com/document/216793374/Myoung-Sook-Lee-A098-772-839-BIA-Mar-27-2014",
  "https://www.scribd.com/document/215970281/Victor-Maya-Lopez-A201-177-467-BIA-Mar-26-2014",
  "https://www.scribd.com/document/215970018/Felipe-Gomez-Alonzo-A200-144-055-BIA-Mar-26-2014",
  "https://www.scribd.com/document/215969012/Esther-Wangari-Kinyanjui-A097-632-790-BIA-Mar-26-2014",
  "https://www.scribd.com/doc/215965153/Ashwin-Patel-A097-550-796-BIA-Mar-24-2014",
  "https://www.scribd.com/document/215964623/Sandra-Elizabeth-Paredes-Avila-A088-139-120-BIA-Mar-24-2014",
  "https://www.scribd.com/document/215960495/Theophilus-Anum-Sowah-A078-393-756-BIA-Mar-24-2014",
  "https://www.scribd.com/document/215721245/Aaron-Manuel-Solis-Tadeo-A077-290-731-BIA-Aug-8-2012",
  "https://www.scribd.com/document/215715906/Mohamed-Lamine-Sanogo-A099-163-486-BIA-Jul-23-2012",
  "https://www.scribd.com/document/215714429/Jose-Jesus-Estrada-Garcia-A087-749-680-BIA-July-20-2012",
  "https://www.scribd.com/document/215038490/Roger-Anthony-Simmonds-A034-062-738-BIA-Mar-24-2014",
  "https://www.scribd.com/document/214862114/Dashnor-Norra-A088-207-785-BIA-Mar-21-2014",
  "https://www.scribd.com/document/214861214/Vladislav-Margulis-A070-233-197-BIA-Mar-21-2014",
  "https://www.scribd.com/document/214859334/Maurice-Leroy-Neille-A045-856-729-BIA-Mar-21-2014",
  "https://www.scribd.com/document/214858497/George-Anton-Toma-A095-722-220-BIA-Mar-20-2014",
  "https://www.scribd.com/document/214857716/Adama-Quedraogo-A095-427-765-BIA-Mar-20-2014",
  "https://www.scribd.com/document/214856207/Pilar-Obsequio-David-A089-556-687-BIA-Mar-20-2014",
  "https://www.scribd.com/document/214854421/Angela-Reyes-Rodriguez-de-Jesus-A087-321-551-BIA-Mar-20-2014",
  "https://www.scribd.com/document/214704630/Christian-Gerardo-Solis-Rojas-A205-572-958-BIA-Mar-19-2014",
  "https://www.scribd.com/document/214702849/Luis-Jose-Tipaz-Poncio-A201-069-002-BIA-Mar-19-2014",
  "https://www.scribd.com/document/214701189/Pangiotis-Tsaglas-A097-324-516-BIA-Mar-18-2014",
  "https://www.scribd.com/document/214700867/Rigoberto-Antonio-Munoz-Barahona-A094-301-715-BIA-Mar-18-2014",
  "https://www.scribd.com/document/214694007/Jose-Cristian-Argueta-Martinez-A200-117-964-BIA-Mar-14-2014",
  "https://www.scribd.com/document/214693513/Brandon-Fuentes-Vasquez-A205-857-112-BIA-Mar-13-2014",
  "https://www.scribd.com/document/214691487/Vannak-Sot-A096-718-092-BIA-Mar-13-2014",
  "https://www.scribd.com/document/214687715/Octavio-Ugarte-Delgado-A205-829-687-BIA-Mar-13-2014",
  "https://www.scribd.com/document/213585448/Paula-Andrea-Miles-A200-542-498-BIA-Mar-12-2014",
  "https://www.scribd.com/document/213584695/Lorenzo-Martinez-Delgado-A074-103-780-BIA-Mar-11-2014",
  "https://www.scribd.com/document/213581769/Hoai-Ich-Pham-A043-007-964-BIA-Mar-11-2014",
  "https://www.scribd.com/document/213180083/Raul-Sainz-Rivera-A091-684-104-BIA-Mar-10-2014",
  "https://www.scribd.com/document/213179806/Victor-Hugo-Alvarado-Cortez-A070-781-971-BIA-Mar-10-2014",
  "https://www.scribd.com/document/213179620/Jose-Alfredo-Fonseca-Velasquez-A200-586-281-BIA-Mar-10-2014",
  "https://www.scribd.com/document/213179342/Joe-Sam-A097-963-699-BIA-Mar-10-2014",
  "https://www.scribd.com/document/213179164/Brenda-Dumlao-Pumaras-A044-943-630-BIA-Mar-10-2014",
  "https://www.scribd.com/document/213178557/Pedro-Mateo-Mateo-A029-376-113-BIA-Mar-10-2014",
  "https://www.scribd.com/document/213177809/Andre-Philip-Cole-A206-193-520-BIA-Mar-7-2014",
  "https://www.scribd.com/document/211898421/Douglas-Shitote-A200-224-343-BIA-March-6-2014",
  "https://www.scribd.com/document/211897682/Abel-Garcia-Jimenez-A205-489-791-BIA-Feb-28-2014",
  "https://www.scribd.com/document/211897334/Magdalena-Dominga-Lopez-Lopez-A200-811-421-BIA-Feb-28-2014",
  "https://www.scribd.com/document/211896647/Luis-Alberto-Perez-A095-790-490-BIA-Feb-27-2014",
  "https://www.scribd.com/document/211681147/Erik-Mendoza-Olivas-A044-547-316-BIA-Feb-26-2014",
  "https://www.scribd.com/document/211678842/Jose-Toledo-Alvarado-A204-259-296-BIA-Feb-26-2014",
  "https://www.scribd.com/document/211677028/Kwabena-Adom-Fosu-A099-138-401-BIA-Feb-26-2014",
  "https://www.scribd.com/document/211676101/Wilfredo-Hernandez-Garcia-A094-217-628-BIA-Feb-26-2014",
  "https://www.scribd.com/document/211674451/Antonia-Leanos-Torres-A089-822-467-BIA-Feb-26-2014",
  "https://www.scribd.com/document/211091707/Fernando-Reyes-Chavez-A205-911-040-BIA-Feb-25-2014",
  "https://www.scribd.com/document/211090906/Kwang-Soo-Kim-A097-971-951-BIA-Feb-25-2014",
  "https://www.scribd.com/document/211090611/Edelma-M-Montejo-Trujillo-A094-329-654-BIA-Feb-25-2014",
  "https://www.scribd.com/document/211089880/Patricia-Lino-A077-152-460-BIA-Feb-25-2014",
  "https://www.scribd.com/document/211088939/Yessenia-Marcelina-Silva-A038-923-988-BIA-Feb-24-2014",
  "https://www.scribd.com/document/211087891/R-P-AXXX-XXX-024-BIA-Feb-19-2014",
  "https://www.scribd.com/document/210616374/Manuel-Jesus-Olivas-Motta-A021-179-705-BIA-Feb-21-2014",
  "https://www.scribd.com/document/210615431/Daniela-Gonzalez-A096-813-955-BIA-Feb-21-2014",
  "https://www.scribd.com/document/210614159/Ngochoa-T-Tran-A076-751-841-BIA-Feb-21-2014",
  "https://www.scribd.com/document/210613207/Sergio-Alberto-Macal-Ventura-A200-589-069-BIA-Feb-20-2014",
  "https://www.scribd.com/document/210610704/Rosa-Lidia-Alfaro-A088-966-288-BIA-Feb-20-2014",
  "https://www.scribd.com/document/210609463/Abdullah-A-Tunis-A087-335-805-BIA-Feb-20-2014",
  "https://www.scribd.com/document/210608485/Chong-Sik-Kim-A098-728-227-BIA-Feb-19-2014",
  "https://www.scribd.com/doc/210560854/Felix-Rodriguez-Rojas-A200-673-246-BIA-Jul-12-2012",
  "https://www.scribd.com/document/210558615/Darwin-Leonardo-Cabrera-Guartasaca-A205-134-459-BIA-Jul-9-2012",
  "https://www.scribd.com/document/210554283/Assetou-Doumbia-A089-952-722-BIA-Jun-28-2012",
  "https://www.scribd.com/document/210553062/Hilario-Rivas-Melendrez-A030-799-483-BIA-Jun-28-2012",
  "https://www.scribd.com/document/210549433/Benito-Pulido-Gomez-A019-987-183-BIA-Jun-27-2012",
  "https://www.scribd.com/document/210546044/Muhammad-Salman-Rais-A079-692-636-BIA-Jun-20-2012",
  "https://www.scribd.com/document/210543026/Luis-Rios-Alatorre-A090-791-693-BIA-Jun-14-2012",
  "https://www.scribd.com/document/210393184/Terrance-Daniel-Bailey-A089-010-106-BIA-June-13-2012",
  "https://www.scribd.com/document/210388023/Loan-Nguyen-Thu-Pham-A095-245-758-BIA-Jun-11-2012",
  "https://www.scribd.com/document/210386369/Benyiahia-Hebbar-A079-374-882-BIA-Jun-11-2012",
  "https://www.scribd.com/document/210383492/Damion-Robinson-A088-009-491-BIA-June-5-2012",
  "https://www.scribd.com/document/210381826/Raunel-Arroyo-A087-309-888-BIA-May-25-2012",
  "https://www.scribd.com/document/210380275/Reddy-Shireesha-A078-722-997-BIA-May-23-2012",
  "https://www.scribd.com/document/210376971/John-Neewaly-Tarpeh-A078-780-390-BIA-May-21-2012",
  "https://www.scribd.com/document/210367640/Roberto-Arellano-Sarabia-A059-086-627-BIA-May-21-2012",
  "https://www.scribd.com/document/210364638/Constantin-Florin-Salgau-A078-734-430-BIA-May-14-2012",
  "https://www.scribd.com/document/210303264/Tyrone-O-Shane-Hill-A035-596-053-BIA-May-14-2012",
  "https://www.scribd.com/document/210301949/Angelito-Castillo-Notarte-A070-466-715-BIA-May-11-2012",
  "https://www.scribd.com/document/210299550/Margarita-Sanchez-Perez-A087-148-850-BIA-May-9-2012",
  "https://www.scribd.com/document/209576002/Yun-Ho-Choi-A099-870-596-BIA-Feb-18-2014",
  "https://www.scribd.com/document/209575427/Esperanza-Alcaraz-De-Vasquez-A076-626-660-BIA-Feb-18-2014",
  "https://www.scribd.com/document/209572760/Gabriel-Villanueva-Ojanama-A201-242-343-BIA-Feb-18-2014",
  "https://www.scribd.com/document/209572447/Jose-Dennis-Alvarado-Canas-A046-041-673-BIA-Feb-14-2014",
  "https://www.scribd.com/document/208887895/Ricardo-Salazar-Aguilar-A087-448-782-BIA-Feb-12-2014",
  "https://www.scribd.com/document/208886857/Elhadji-Mbaye-Kende-A093-439-866-BIA-Feb-12-2014",
  "https://www.scribd.com/document/208885796/Joao-Coutinho-A098-236-413-BIA-Feb-11-2014",
  "https://www.scribd.com/document/208883482/Collins-Anakor-A201-141-550-BIA-Feb-10-2014",
  "https://www.scribd.com/document/208882647/Jose-Maria-Sanchez-Cazarez-A205-943-262-BIA-Feb-10-2014",
  "https://www.scribd.com/document/208882070/Hector-Ajche-A205-495-095-BIA-Feb-10-2014",
  "https://www.scribd.com/document/208247529/Luis-Alberto-Rivas-Camader-A040-111-602-BIA-Feb-7-2014",
  "https://www.scribd.com/document/208247123/Armando-Castillo-Ocampo-A205-489-036-BIA-Feb-7-2014",
  "https://www.scribd.com/document/208246334/Adam-Laurence-A200-867-388-BIA-Feb-7-2014",
  "https://www.scribd.com/document/208245460/Jose-Manuel-Barrios-Rojas-A090-145-871-BIA-Feb-7-2014",
  "https://www.scribd.com/document/208244844/Zuri-Marlene-Melgar-de-Iglesias-A206-525-223-BIA-Feb-6-2014",
  "https://www.scribd.com/document/208214537/S-G-AXXX-XXX-756-BIA-Dec-14-2009",
  "https://www.scribd.com/document/208206075/M-N-AXX-XXX-094-BIA-Oct-12-2000",
  "https://www.scribd.com/document/208202914/M-A-H-AXX-XXX-562-BIA-Sept-29-2003",
  "https://www.scribd.com/document/208167027/J-M-S-B-W-AXX-XXX-109-BIA-Apr-4-2003",
  "https://www.scribd.com/document/208163546/B-P-AXX-XXX-269-BIA-Oct-15-2002",
  "https://www.scribd.com/document/208161828/E-M-E-AXXX-XXX-420-BIA-Jan-31-2012",
  "https://www.scribd.com/document/208159504/C-A-AXX-XXX-698-BIA-May-11-2000",
  "https://www.scribd.com/document/208157423/A-H-AXX-XXX-769-BIA-Aug-8-2006",
  "https://www.scribd.com/document/206147008/Rony-Abidail-Pimentel-Giron-A205-120-942-BIA-Feb-5-2014",
  "https://www.scribd.com/document/206146044/Margarita-Barrera-Ceja-A200-827-191-BIA-Feb-5-2014",
  "https://www.scribd.com/document/206144436/Victor-Manuel-Contreras-A091-313-427-BIA-Feb-5-2014",
  "https://www.scribd.com/document/206142977/Nonhlanhla-Guguletha-Ngema-A087-948-239-BIA-Feb-5-2014",
  "https://www.scribd.com/document/206141853/Jin-Hee-Shin-A047-415-708-BIA-Feb-4-2014",
  "https://www.scribd.com/document/205871959/M-G-O-AXXX-XXX-611-BIA-Feb-4-2014",
  "https://www.scribd.com/doc/205845986/Francisco-Javier-Montes-Flores-A205-215-638-BIA-Feb-3-2014",
  "https://www.scribd.com/document/205844441/Enereo-Mederos-Sanchez-A205-132-022-BIA-Feb-3-2014",
  "https://www.scribd.com/document/205843144/Brenda-Iriselda-Murillo-Velasquez-A099-667-090-BIA-Feb-3-2014",
  "https://www.scribd.com/document/205841308/Teresa-Flores-Razo-A092-326-584-BIA-Jan-31-2014",
  "https://www.scribd.com/document/205840630/Rogelio-Chavez-Rubio-A205-656-470-BIA-Jan-30-2014",
  "https://www.scribd.com/document/205838546/Leymi-Sahira-Moreno-Grullon-A099-994-900-BIA-Jan-30-2014",
  "https://www.scribd.com/doc/205837445/Erik-Salvador-Cruz-A097-687-092-BIA-Jan-30-2014",
  "https://www.scribd.com/document/205835748/Dwayne-Anthony-Shane-Patterson-A047-114-478-BIA-Jan-30-2014",
  "https://www.scribd.com/document/205826089/Gerald-Morisset-Boisvert-A041-905-505-BIA-Jan-29-2014",
  "https://www.scribd.com/document/205822273/Jean-Pierre-Batcha-Samba-A088-046-199-BIA-Dec-19-2013",
  "https://www.scribd.com/document/205820728/Pablo-Quintero-Madrigal-A090-394-871-BIA-Oct-4-2013",
  "https://www.scribd.com/document/205818806/Everline-Gesare-Nyabwari-A097-683-208-BIA-Sept-26-2013",
  "https://www.scribd.com/document/205378440/David-Antonio-Lara-Torres-A094-218-294-BIA-Jan-28-2014",
  "https://www.scribd.com/document/205373882/Dieuvu-Forvilus-A071-552-965-BIA-Jan-28-2014",
  "https://www.scribd.com/document/205372425/D-G-Axxx-xx5-108-BIA-Jan-28-2014",
  "https://www.scribd.com/document/205371273/Miguel-Cruz-Gonzalez-A200-679-853-BIA-Jan-27-2014",
  "https://www.scribd.com/document/204277050/Ali-Mahmoud-Ibrahim-Ibrahim-A089-276-533-BIA-Jan-24-2014",
  "https://www.scribd.com/document/204276201/Paul-Kibua-Wanyoike-A078-496-780-BIA-Jan-24-2014",
  "https://www.scribd.com/document/204275361/Enrique-Reyes-Francisco-A075-418-435-BIA-Jan-24-2014",
  "https://www.scribd.com/document/204274129/Rodrigo-Vasquez-Lopez-A048-138-084-BIA-Jan-24-2014",
  "https://www.scribd.com/document/204272303/Nuno-Helder-Da-Silva-Barros-A200-548-277-BIA-Jan-23-2014",
  "https://www.scribd.com/document/203258831/Omar-Daniel-Ruiz-A095-640-803-BIA-Jan-23-2014",
  "https://www.scribd.com/document/203257950/Glorofer-Angelica-Nufable-Villatuya-A089-445-773-BIA-Jan-23-2014",
  "https://www.scribd.com/document/203256894/Ivan-Francisco-Moreno-Virrey-A042-847-549-BIA-Jan-23-2014",
  "https://www.scribd.com/document/203256252/Eunice-Nsheka-Miller-A098-421-064-BIA-Jan-22-2014",
  "https://www.scribd.com/document/203255395/Blanca-Josefina-Rinquillo-de-Corrales-A074-662-392-BIA-Jan-22-2014",
  "https://www.scribd.com/doc/203254362/Mario-Flores-Corona-A205-719-317-BIA-Jan-17-2014",
  "https://www.scribd.com/document/203249566/Juan-Gabriel-Rodriguez-A092-389-472-BIA-Jan-17-2014",
  "https://www.scribd.com/document/202579347/Silvia-Cabrera-Carillo-A089-168-827-BIA-Apr-30-2012",
  "https://www.scribd.com/document/202577678/Seung-Min-Hong-A072-971-915-BIA-Apr-30-2012",
  "https://www.scribd.com/document/202575731/Myleen-Cuyson-Pascual-A086-963-266-BIA-Apr-30-2012",
  "https://www.scribd.com/document/202572052/Eduardo-Lagunez-Bravo-A205-013-184-BIA-Apr-30-2012",
  "https://www.scribd.com/document/202569113/Patrick-Jonah-Kagau-A099-258-131-BIA-Apr-26-2012",
  "https://www.scribd.com/document/202560626/Frederick-Monyoncho-Kerina-A093-442-983-BIA-Apr-24-2012",
  "https://www.scribd.com/document/202559144/Veronica-Delores-Ashley-A097-996-130-BIA-Apr-24-2012",
  "https://www.scribd.com/document/202549649/Sallahadin-Birhan-A076-911-298-BIA-Apr-23-2012",
  "https://www.scribd.com/document/202546549/Jose-Ismael-Aguilar-Rivera-A077-701-378-BIA-Apr-20-2012",
  "https://www.scribd.com/document/202541891/Benno-Rivens-A026-156-068-BIA-Apr-13-2012",
  "https://www.scribd.com/document/202439434/Bacilio-Hernandez-Garcia-A098-237-003-BIA-Apr-11-2012",
  "https://www.scribd.com/document/202437269/Jose-DeJesus-Torres-Miranda-A088-361-024-BIA-Apr-9-2012",
  "https://www.scribd.com/document/202435393/Erika-Ortiz-Garcia-A076-708-985-BIA-Apr-3-2012",
  "https://www.scribd.com/document/202432322/Winston-Everton-Samuels-A055-938-461-BIA-Mar-29-2012",
  "https://www.scribd.com/document/202430511/Roderick-Peralta-Morin-A098-270-728-BIA-Mar-26-2012",
  "https://www.scribd.com/document/202429501/Elsaid-Elsayed-Elkenawi-A088-426-340-BIA-Mar-26-2012",
  "https://www.scribd.com/document/202410716/Lawrence-Eric-Downs-A076-916-078-BIA-Mar-21-2012",
  "https://www.scribd.com/document/202407219/Braulio-Aladino-Sinche-Vera-A097-957-323-BIA-Mar-20-2012",
  "https://www.scribd.com/document/202405521/Washington-Vicente-Pena-Flores-A091-546-852-BIA-Mar-19-2012",
  "https://www.scribd.com/document/202399395/Jose-Luis-Tapia-Fierro-A091-456-635-BIA-Mar-19-2012",
  "https://www.scribd.com/document/202397351/Agostino-Accardo-A035-089-741-BIA-Mar-15-2012",
  "https://www.scribd.com/document/202386790/Maria-Perez-Cabrera-A072-365-767-BIA-Mar-15-2012",
  "https://www.scribd.com/document/202384779/Aneury-Jose-Reyes-Ozuna-A056-533-868-BIA-Mar-13-2012",
  "https://www.scribd.com/document/202383412/Vaclav-Ondrisek-A078-965-231-BIA-Mar-9-2012",
  "https://www.scribd.com/document/202380637/Adolman-E-Sanchez-Turcios-A094-415-405-BIA-Mar-8-2012",
  "https://www.scribd.com/document/202378199/Merilande-Da-Silva-Oliveira-Albino-A200-189-375-BIA-Feb-28-2012",
  "https://www.scribd.com/document/202376040/Alcides-Marte-Blanco-A077-888-145-BIA-Feb-24-2012",
  "https://www.scribd.com/document/202373981/Edgar-Antonio-Castellon-Lemus-A091-603-226-BIA-Feb-17-2012",
  "https://www.scribd.com/document/202373103/Wilber-Jose-Chavarria-A029-961-066-BIA-Feb-13-2012",
  "https://www.scribd.com/document/202272052/Victor-Manuel-Gomez-Gomez-A099-669-457-BIA-Feb-10-2012",
  "https://www.scribd.com/document/202270691/Ponniah-Ramanan-A087-629-177-BIA-Feb-9-2012",
  "https://www.scribd.com/document/202267976/Christian-Cortes-Duran-A096-587-962-BIA-Jan-31-2012",
  "https://www.scribd.com/document/202226472/Moises-Martinez-Hernandez-A089-476-569-BIA-Jan-31-2012",
  "https://www.scribd.com/document/202224335/Robinson-Cordero-Perez-A079-739-416-BIA-Jan-31-2012",
  "https://www.scribd.com/document/202223336/Waldemar-Stanislaw-Mroczkowski-A099-657-732-BIA-Jan-30-2012",
  "https://www.scribd.com/document/202221386/Lenin-Patino-A089-013-179-BIA-Jan-20-2012",
  "https://www.scribd.com/document/202220306/Patricia-Esmeralda-Valladares-Bardales-A094-098-339-BIA-Jan-20-2012",
  "https://www.scribd.com/document/202216334/Francisco-Hernandez-Pina-A073-976-639-BIA-Jan-19-2012",
  "https://www.scribd.com/document/202214795/Basilio-Estevez-A044-921-877-BIA-Jan-18-2012",
  "https://www.scribd.com/document/202211952/Angela-Marie-Murillo-Martinez-A089-839-693-BIA-Jan-11-2012",
  "https://www.scribd.com/document/202210909/Yessy-Lizeth-Ortiz-Caceras-A055-864-797-BIA-Jan-6-2012",
  "https://www.scribd.com/document/201948923/Mohammed-Anwar-Hazuri-A034-373-877-BIA-Jan-16-2014",
  "https://www.scribd.com/document/201948542/Maria-Leticia-Garcia-Aguilar-A098-237-255-BIA-Jan-15-2014",
  "https://www.scribd.com/document/201947025/Ravi-Satulo-Laicer-A096-675-284-BIA-Jan-15-2014",
  "https://www.scribd.com/document/201944785/Salvador-Alejo-Reta-A060-663-553-BIA-Jan-15-2014",
  "https://www.scribd.com/document/201943893/Trisarana-Suvarnasara-A089-183-715-BIA-Jan-14-2014",
  "https://www.scribd.com/document/201941039/Jorge-Alberto-Contreras-A044-176-076-BIA-Jan-14-2014",
  "https://www.scribd.com/document/200951909/Oswen-Fitzherbert-Cameron-A088-810-944-BIA-Jan-13-2014",
  "https://www.scribd.com/document/200949779/Manuel-Iruegas-Gomez-A077-623-337-BIA-Jan-13-2014",
  "https://www.scribd.com/document/200949245/Jacinto-Moises-Carbonell-Desliz-A074-054-226-BIA-Jan-13-2014",
  "https://www.scribd.com/document/199455855/Omar-Malleh-Boye-A093-429-545-BIA-Jan-8-2014",
  "https://www.scribd.com/document/199454855/Guillermo-Cases-Moya-A074-713-088-BIA-Jan-8-2014",
  "https://www.scribd.com/document/199454128/Ramiro-Torres-Godinez-A205-765-589-BIA-Jan-6-2014",
  "https://www.scribd.com/doc/199453498/Maria-Del-Carmen-Guzman-A087-089-870-BIA-Jan-6-2014",
  "https://www.scribd.com/document/199451836/Manuel-Lopez-Rivera-A089-235-276-BIA-Jan-3-2014",
  "https://www.scribd.com/document/199450203/R-C-Axxx-xxx-xxx-BIA-Aug-1-2013",
  "https://www.scribd.com/document/199445648/Edgar-Daniel-Nava-A099-317-368-BIA-March-30-2012",
  "https://www.scribd.com/document/199418719/Jose-Alberto-Armas-Barranzuela-A094-812-114-BIA-Feb-26-2010",
  "https://www.scribd.com/document/199382880/Lufty-Abraham-Abassy-Oqueli-A029-886-971-BIA-Dec-30-2011",
  "https://www.scribd.com/document/199380760/Javier-Rios-Bustamante-A037-585-346-BIA-Dec-30-2011",
  "https://www.scribd.com/document/199379462/Jesus-Gaytan-Castro-A078-047-050-BIA-Dec-29-2011",
  "https://www.scribd.com/document/199378047/Karla-G-Espinoza-A029-184-276-BIA-Dec-23-2011",
  "https://www.scribd.com/document/199357918/Manuel-de-Jesus-Chavarria-Lopez-A089-823-873-BIA-Dec-15-2011",
  "https://www.scribd.com/document/199350301/Aldair-Ivan-Cervantes-A200-630-917-BIA-Dec-8-2011",
  "https://www.scribd.com/document/199343369/Akim-Rahman-A079-687-751-BIA-Dec-6-2011",
  "https://www.scribd.com/doc/199326776/Jose-Manuel-Silva-A019-321-130-BIA-Nov-30-2011",
  "https://www.scribd.com/document/199322864/Sokrat-Stambolliu-A200-548-126-BIA-Nov-28-2011",
  "https://www.scribd.com/document/199295210/Luis-Rueda-Perez-A098-241-147-BIA-Nov-17-2011",
  "https://www.scribd.com/document/199291160/Altagracia-Filpo-de-Vasquez-A300-317-878-BIA-Nov-15-2011",
  "https://www.scribd.com/document/199192878/Ion-Alexander-Beza-A200-630-890-BIA-Oct-27-2011",
  "https://www.scribd.com/document/199187672/Hugo-Ivan-Cardoso-A073-957-149-BIA-Oct-19-2011",
  "https://www.scribd.com/document/199186268/Edgar-Galeas-A091-351-851-BIA-Oct-14-2011",
  "https://www.scribd.com/document/199185203/Kim-Stevens-A035-172-124-BIA-Oct-12-2011",
  "https://www.scribd.com/document/199180309/Sang-Mae-Lee-A089-125-467-BIA-Oct-11-2011",
  "https://www.scribd.com/document/199176996/Hemantkumar-Patel-A047-463-053-BIA-Sep-27-2011",
  "https://www.scribd.com/document/199171712/Maria-Dalila-Hernandez-A087-709-313-BIA-Sept-21-2011",
  "https://www.scribd.com/document/199170628/Terrance-Daniel-Bailey-A089-101-106-BIA-Sep-14-2011",
  "https://www.scribd.com/document/199166424/Nicasio-Mendoza-Carrasco-A095-725-537-BIA-Sep-8-2011",
  "https://www.scribd.com/document/199164525/Musa-Imeri-A088-740-874-BIA-Sep-8-2011",
  "https://www.scribd.com/doc/199158553/Jose-Adan-Robles-A073-567-320-BIA-Aug-30-2011",
  "https://www.scribd.com/document/199156257/Eva-Hernandez-A093-291-466-BIA-Aug-30-2011",
  "https://www.scribd.com/document/199154029/Shahilia-Reid-Goldson-A087-095-113-BIA-Aug-5-2011",
  "https://www.scribd.com/document/199150834/Maria-Theresa-Ganzon-Domecillo-A029-688-986-BIA-July-18-2011",
  "https://www.scribd.com/document/199148829/Anibal-Rolando-Arrobo-A075-449-130-BIA-July-1-2011",
  "https://www.scribd.com/document/199146207/Isabel-Balderas-Marquez-A095-810-160-BIA-June-30-2011",
  "https://www.scribd.com/document/199144060/Raul-Maldonado-Ochoa-A028-892-793-BIA-June-30-2011",
  "https://www.scribd.com/document/199142519/Pedro-Larios-Quixan-A077-007-158-BIA-June-28-2011",
  "https://www.scribd.com/document/199133557/Roderick-Gary-Levy-A035-954-926-BIA-June-27-2011",
  "https://www.scribd.com/document/199132383/Ramiro-Hernandez-A079-350-585-BIA-June-27-2011",
  "https://www.scribd.com/document/199130695/Helio-Alves-Ribeiro-A070-981-726-BIA-June-23-2011",
  "https://www.scribd.com/document/199056157/Eloy-Sanchez-Rubio-A077-405-856-BIA-June-20-2011",
  "https://www.scribd.com/document/199052072/Gloria-Monica-Price-A089-410-415-BIA-June-20-2011",
  "https://www.scribd.com/document/199051286/Aniello-Moschillo-A011-521-320-BIA-June-17-2011",
  "https://www.scribd.com/doc/199050806/Aniello-Moschillo-A011-521-320-BIA-June-17-2011",
  "https://www.scribd.com/document/199047723/Herminio-Robles-Quintano-A074-351-862-BIA-June-9-2011",
  "https://www.scribd.com/document/199039943/Mykola-Nykholat-A087-261-881-BIA-June-3-2011",
  "https://www.scribd.com/document/198932742/Daniel-Sierra-A074-026-895-BIA-June-1-2011",
  "https://www.scribd.com/document/198930830/Ruben-Montenegro-Ruiz-A041-095-571-BIA-May-31-2011",
  "https://www.scribd.com/document/198876490/Hector-Diaz-Mendoza-A044-371-296-BIA-May-23-2011",
  "https://www.scribd.com/document/198874316/Jose-Lomeli-Becerra-A092-122-323-BIA-May-20-2011",
  "https://www.scribd.com/document/198871199/Lacy-Ann-Shaw-A074-650-611-BIA-May-20-2011",
  "https://www.scribd.com/document/198868932/Cirilo-Aaron-Covarrubias-Teposte-A078-469-158-BIA-May-13-2011",
  "https://www.scribd.com/document/198867596/Omar-Alberto-Cisneros-Ordaz-A045-124-831-BIA-May-10-2011",
  "https://www.scribd.com/document/198858695/Barbara-Gibek-A073-150-567-BIA-Apr-29-2011",
  "https://www.scribd.com/document/198855562/Cesar-Perez-Hernandez-A077-323-561-BIA-April-28-2011",
  "https://www.scribd.com/document/198853617/Jose-Santos-Luna-Canales-A094-142-206-BIA-April-28-2011",
  "https://www.scribd.com/document/198852089/Wajid-Ali-Siddiqi-A095-473-104-BIA-April-26-2011",
  "https://www.scribd.com/document/198850225/Daria-Levasheva-A058-471-381-BIA-April-25-2011",
  "https://www.scribd.com/document/198847623/Sreedhar-Rao-Padma-Sri-A099-046-696-BIA-April-25-2011",
  "https://www.scribd.com/document/198845875/Roberto-Hernanez-Pacheco-A072-317-637-BIA-April-22-2011",
  "https://www.scribd.com/document/198833889/Marco-Antonio-Rivera-Carrillo-A200-607-697-BIA-April-22-2011",
  "https://www.scribd.com/document/198831011/Sulakhan-Singh-Lubana-A087-489-770-BIA-April-18-2011",
  "https://www.scribd.com/document/198829651/Xavier-Aristides-Mejia-Garcia-A098-119-800-BIA-April-15-2011",
  "https://www.scribd.com/document/198826118/Zaudi-Khan-A076-682-242-BIA-April-14-2011",
  "https://www.scribd.com/document/198433884/Ramon-Octavio-Bustillo-Delarca-A077-400-623-BIA-Dec-31-2013",
  "https://www.scribd.com/document/198432851/Jose-Luis-Gutierrez-Guzman-A205-716-539-BIA-Dec-31-2013",
  "https://www.scribd.com/document/198429668/Juan-Luis-Avalos-Avalos-A205-834-943-BIA-Dec-31-2013",
  "https://www.scribd.com/document/198427675/Adrian-Phillip-Moncrieffe-A038-581-600-BIA-Dec-30-2013",
  "https://www.scribd.com/document/198426206/Javed-Anwar-A077-044-013-BIA-Dec-30-2013",
  "https://www.scribd.com/document/198391591/Jozef-Szypulski-A099-030-552-BIA-Dec-30-2013",
  "https://www.scribd.com/document/198382907/Jose-Manuel-Isabel-Diaz-A205-500-422-BIA-Dec-30-2013",
  "https://www.scribd.com/document/196764005/Mickael-Christian-Martin-A076-549-309-BIA-Dec-27-2013",
  "https://www.scribd.com/document/196763359/Ismael-Daggy-Ibrahim-A205-009-375-BIA-Dec-26-2013",
  "https://www.scribd.com/document/196759951/Emmanuel-Amparo-Abreu-A087-287-722-BIA-Dec-24-2013",
  "https://www.scribd.com/document/196758842/Jose-Eustate-A047-128-564-BIA-Dec-20-2013",
  "https://www.scribd.com/document/196758117/Ivan-Hernandez-Trujillo-A088-333-112-BIA-Dec-19-2013",
  "https://www.scribd.com/document/196757245/Carlos-Quintanilla-Chicas-A200-234-259-BIA-Dec-18-2013",
  "https://www.scribd.com/document/193871497/Jorge-Antonio-Haro-Pena-A095-727-770-BIA-Dec-17-2013",
  "https://www.scribd.com/document/193870067/Ahmed-Nadeem-Malik-A092-006-532-BIA-Dec-17-2013",
  "https://www.scribd.com/doc/193868990/Juan-Espinoza-Torres-A200-978-484-BIA-Dec-16-2013",
  "https://www.scribd.com/document/193867952/Esau-Lara-A097-322-982-BIA-Dec-16-2013",
  "https://www.scribd.com/document/193867108/Kamleshwar-Prasad-A099-681-885-BIA-Dec-13-2013",
  "https://www.scribd.com/document/193865728/Merton-David-Lopez-A091-085-417-BIA-Dec-13-2013",
  "https://www.scribd.com/document/193864768/Salvador-Espinoza-Gonzalez-A021-576-036-BIA-Dec-13-2013",
  "https://www.scribd.com/document/192344677/Erick-Cruz-Bermejo-A205-497-572-BIA-Dec-12-2013",
  "https://www.scribd.com/document/192343172/Yakov-Grigorievich-Drabovskiy-A028-158-230-BIA-Dec-12-2013",
  "https://www.scribd.com/document/192341802/Julio-Cesar-Melgar-Quijano-A094-857-498-BIA-Dec-12-2013",
  "https://www.scribd.com/document/192341321/Rut-Betania-Castillo-de-Figueroa-A095-982-111-BIA-Dec-11-2013",
  "https://www.scribd.com/document/192340198/Victor-Rosel-A098-194-325-BIA-Dec-9-2013",
  "https://www.scribd.com/document/191833630/Leonel-Amador-Aguilar-Morales-A088-091-744-BIA-Dec-6-2013",
  "https://www.scribd.com/document/191832073/Chia-Hsu-Kuo-A095-151-004-BIA-Dec-3-2013",
  "https://www.scribd.com/document/191830559/Everline-Gesare-Nyabwari-A097-683-208-BIA-Dec-5-2013",
  "https://www.scribd.com/document/191829785/Salvador-Vaca-Bueno-A072-956-700-BIA-Dec-5-2013",
  "https://www.scribd.com/document/191827711/Ruben-Aviles-Diaz-A097-869-352-BIA-Dec-5-2013",
  "https://www.scribd.com/document/191822351/Arya-Etedali-A095-686-160-BIA-Dec-4-2013",
  "https://www.scribd.com/document/191820950/Juan-Quiroga-Briones-A028-323-575-BIA-Dec-3-2013",
  "https://www.scribd.com/document/191164365/Esteban-Nolasco-Gaspar-A201-145-750-BIA-Nov-29-2013",
  "https://www.scribd.com/document/191163977/Clement-Hugh-Hanson-Ewers-A096-433-109-BIA-Nov-29-2013",
  "https://www.scribd.com/document/191163453/Jose-Matias-Valazquez-A089-002-625-BIA-Nov-29-2013",
  "https://www.scribd.com/document/191162728/Armando-Parra-Reyes-A091-156-708-BIA-Nov-27-2013",
  "https://www.scribd.com/document/189577569/Jose-Luis-Gonzales-A029-158-835-BIA-Nov-26-2013",
  "https://www.scribd.com/document/189574025/Jayesh-K-Patel-A074-905-185-BIA-Nov-26-2013",
  "https://www.scribd.com/document/189572348/Hiep-Thanh-Nguyen-A073-306-230-BIA-Nov-26-2013",
  "https://www.scribd.com/document/189571423/Rainiere-Antonio-de-La-Cruz-Brito-A060-135-193-BIA-Nov-26-2013",
  "https://www.scribd.com/document/189569747/Jose-H-Hernandez-A094-054-192-BIA-Nov-25-2013",
  "https://www.scribd.com/document/189568494/Juan-Roberto-Guzman-A044-466-955-BIA-Nov-25-2013",
  "https://www.scribd.com/document/187627941/Calfus-Wentworth-Drummond-A075-443-280-BIA-Nov-22-2013",
  "https://www.scribd.com/document/187627194/Gabriel-Octavio-Arellan-A043-944-603-BIA-Nov-22-2013",
  "https://www.scribd.com/document/187626619/Ivan-Meza-Jurado-A200-821-185-BIA-Nov-21-2013",
  "https://www.scribd.com/document/187625762/Emerita-Casilla-Nunez-A042-893-719-BIA-Nov-20-2013",
  "https://www.scribd.com/document/187621207/Jose-Mauricio-Diaz-A029-012-746-BIA-Nov-19-2013",
  "https://www.scribd.com/document/186955013/Marten-Lorenzo-Ventura-Arias-A057-144-173-BIA-Nov-18-2013",
  "https://www.scribd.com/document/186953311/Susana-Rivero-Godoy-A024-718-991-BIA-Nov-18-2013",
  "https://www.scribd.com/document/186951107/Enrique-Manuel-Vasquez-Perez-A095-802-066-BIA-Nov-14-2013",
  "https://www.scribd.com/document/186948787/Martin-Gerardo-Velasco-Garcia-A089-850-114-BIA-Nov-14-2013",
  "https://www.scribd.com/document/186946751/Jose-Antonio-Soto-Moran-A078-744-043-BIA-Nov-14-2013",
  "https://www.scribd.com/document/186022216/Jose-Antonio-Monjazar-Fernandez-A200-611-977-BIA-Nov-13-2013",
  "https://www.scribd.com/document/186020346/Juan-Rafael-Santos-A074-191-655-BIA-Nov-13-2013",
  "https://www.scribd.com/document/186018200/Hiliario-Cisneros-Sanchez-A079-791-113-BIA-Nov-13-2013",
  "https://www.scribd.com/document/186017386/Francisco-Navarro-Acosta-A077-173-541-BIA-Nov-13-2013",
  "https://www.scribd.com/document/185489471/Kwok-Sum-Wong-A036-850-251-BIA-Nov-12-2013",
  "https://www.scribd.com/document/185488518/Elisa-Marbelly-Caceres-Chacon-A098-487-209-BIA-Nov-8-2013",
  "https://www.scribd.com/document/185487455/Misael-Najera-Najera-A089-744-149-BIA-Nov-8-2013",
  "https://www.scribd.com/document/185486124/Roy-Gerald-Moore-A079-734-564-BIA-Nov-8-2013",
  "https://www.scribd.com/document/185485004/Xingchen-Ye-A078-694-496-BIA-Nov-8-2013",
  "https://www.scribd.com/document/184444498/Cirano-Abertano-Leon-Nogales-A205-140-051-BIA-Nov-7-2013",
  "https://www.scribd.com/document/184440585/Victor-Hugo-Gomez-Cifuentez-A089-284-153-BIA-Nov-6-2013",
  "https://www.scribd.com/document/184439289/Do-Kyung-Lee-A089-047-352-BIA-Nov-5-2013",
  "https://www.scribd.com/document/184437013/Jose-Santos-Gomez-Amaya-A205-881-617-BIA-Nov-5-2013",
  "https://www.scribd.com/document/184435894/Kelechi-Pauline-Mbagwu-A078-437-923-BIA-Nov-4-2013",
  "https://www.scribd.com/document/184433139/Tui-Pele-Falo-A035-046-579-BIA-Nov-1-2013",
  "https://www.scribd.com/document/183280024/Juan-Carlos-Clase-A043-986-617-BIA-Oct-31-2013",
  "https://www.scribd.com/document/183277253/Magdalena-Pietryszcz-A098-236-466-BIA-Oct-31-2013",
  "https://www.scribd.com/document/183276263/Sara-Jasmin-Ashtaryeh-A075-444-016-BIA-Oct-31-2013",
  "https://www.scribd.com/document/183273460/Aminadad-Natanael-Mendez-Perez-A099-623-872-BIA-Oct-30-2013",
  "https://www.scribd.com/doc/183272325/Desmond-E-Burke-A022-492-290-BIA-Oct-29-2013",
  "https://www.scribd.com/document/183270815/Laura-Elizabeth-Gracia-Cerda-A089-823-448-BIA-Oct-28-2013",
  "https://www.scribd.com/document/183270217/Kanubhai-Lalbhai-Bhatt-A073-183-507-BIA-Oct-28-2013",
  "https://www.scribd.com/document/182351859/Michael-Mutukwa-A099-395-254-BIA-Oct-25-2013",
  "https://www.scribd.com/document/182350620/Jose-Juan-Garcia-Aguirre-A094-985-673-BIA-Oct-25-2013",
  "https://www.scribd.com/document/182348079/To-Nga-Thi-Dinh-A089-854-183-BIA-Oct-25-2013",
  "https://www.scribd.com/document/182344679/Esau-Rodriguez-A073-692-631-BIA-Oct-25-2013",
  "https://www.scribd.com/document/182343652/Rosa-Amparo-Salmeron-A072-163-739-BIA-Oct-24-2013",
  "https://www.scribd.com/document/181834734/Maria-Isabel-Guzman-Sanchez-A087-118-694-BIA-Oct-23-2013",
  "https://www.scribd.com/document/181832091/Anderson-Ferreira-A099-982-743-BIA-Oct-23-2013",
  "https://www.scribd.com/document/181830258/Lamin-Manjang-A096-803-746-BIA-Oct-23-2013",
  "https://www.scribd.com/document/181829063/Lashante-Tanotra-Jones-A088-958-878-BIA-Oct-23-2013",
  "https://www.scribd.com/document/181828197/Joseph-Delice-A075-842-401-BIA-Oct-22-2013",
  "https://www.scribd.com/document/181464444/Constantino-Santel-Montes-A094-090-659-BIA-Oct-21-2013",
  "https://www.scribd.com/document/181463484/Augustin-Moreno-Duarte-A200-867-579-BIA-Oct-21-2013",
  "https://www.scribd.com/document/181461974/Abner-Salazar-Herrera-A200-191-590-BIA-Oct-21-2013",
  "https://www.scribd.com/document/181460166/Juan-Martin-Diaz-Montes-A073-826-900-BIA-Oct-21-2013",
  "https://www.scribd.com/document/181458671/Maria-Ireri-Rivas-A089-573-062-BIA-Oct-18-2013",
  "https://www.scribd.com/document/181456206/Cresencio-Araujo-Navarro-A200-662-702-BIA-Oct-18-2013",
  "https://www.scribd.com/document/180502942/Oluwole-Olufunmi-A045-081-162-BIA-Oct-17-2013",
  "https://www.scribd.com/document/180501681/Jose-de-Jesus-Alcantar-Miramontes-A077-361-213-BIA-Oct-17-2013",
  "https://www.scribd.com/document/180499562/Paul-Muchai-Nganga-A087-196-423-BIA-Oct-17-2013",
  "https://www.scribd.com/document/180497589/Ismael-Rangel-Yllescas-A201-101-348-BIA-Oct-17-2013",
  "https://www.scribd.com/document/180494488/Pedro-Diaz-Vidales-A205-765-609-BIA-Oct-7-2013",
  "https://www.scribd.com/document/180332513/Jaime-Enrique-Valenzuela-Felix-A034-590-205-BIA-Oct-15-2013",
  "https://www.scribd.com/document/179612543/Rodrigo-Esquivel-Jimenez-A090-974-844-BIA-April-11-2011",
  "https://www.scribd.com/document/179606327/Yan-Zhen-Hu-A042-729-871-BIA-April-7-2011",
  "https://www.scribd.com/document/179603581/Cestmir-Drkula-A098-939-865-BIA-April-7-2011",
  "https://www.scribd.com/document/179598085/Juan-Pablo-Zea-Flores-A041-737-150-BIA-April-6-2011",
  "https://www.scribd.com/document/179595447/Shadi-Jamal-Almikdad-A047-852-543-BIA-April-4-2011",
  "https://www.scribd.com/document/179592253/Himanshu-Patel-A046-743-862-BIA-April-4-2011",
  "https://www.scribd.com/document/179033823/Jose-Maria-Islas-Gonzalez-A205-497-397-BIA-Oct-11-2013",
  "https://www.scribd.com/document/179031328/Victor-Enrique-Tally-Barrios-A041-736-376-BIA-Oct-8-2013",
  "https://www.scribd.com/document/179030270/Jorge-Ivan-Bautista-Santos-A043-996-234-BIA-Oct-7-2013",
  "https://www.scribd.com/document/179029208/Rony-Vladimir-Monge-A205-503-682-BIA-Oct-3-2013",
  "https://www.scribd.com/document/179027697/Luis-Perez-A200-687-958-BIA-Oct-3-2013",
  "https://www.scribd.com/document/179026850/Kaydian-Arielle-Rose-A300-321-202-BIA-Sept-30-2013",
  "https://www.scribd.com/document/179021836/Anna-Magdalena-Bojko-A089-059-545-BIA-Sept-30-2013",
  "https://www.scribd.com/document/179020852/Julia-Catherine-Holtzman-A072-043-863-BIA-Sept-30-2013",
  "https://www.scribd.com/doc/179020109/Carl-Alphanso-Thompson-A045-613-869-BIA-Sept-30-2013",
  "https://www.scribd.com/document/178499857/Upinderjit-Singh-A048-029-730-BIA-Sept-27-2013",
  "https://www.scribd.com/document/178497750/Maria-Lilian-Lopez-Ferrera-A206-005-069-BIA-Sept-27-2013",
  "https://www.scribd.com/document/178496947/Delfino-Flores-Sanchez-A201-101-216-BIA-Sept-27-2013",
  "https://www.scribd.com/doc/178496061/Carlos-Andres-Palaez-A096-985-160-BIA-Sept-27-2013",
  "https://www.scribd.com/document/178494909/Dulcinea-Alves-Da-Silva-A096-905-188-BIA-Sept-27-2013",
  "https://www.scribd.com/document/177551348/William-Enrique-Alvarado-Melendez-A094-405-564-BIA-May-10-2013",
  "https://www.scribd.com/document/177550351/Estefania-Torres-Bracamontes-A098-523-028-BIA-July-7-2011",
  "https://www.scribd.com/document/177189167/Jean-Daniel-Ahlijah-A205-829-470-BIA-Sept-26-2013",
  "https://www.scribd.com/document/177188077/Maritza-Salcido-Rocha-A200-832-424-BIA-Sept-26-2013",
  "https://www.scribd.com/document/177187103/Roberto-Ceja-Ruiz-A200-558-702-BIA-Sept-26-2013",
  "https://www.scribd.com/document/177186230/Julie-Broadhurst-Ball-A099-099-953-BIA-Sept-26-2013",
  "https://www.scribd.com/document/177185204/Kostas-Pantzialas-A097-515-404-BIA-Sept-26-2013",
  "https://www.scribd.com/document/176603462/Claudette-Collen-Hubbard-A030-085-111-BIA-Oct-8-2013",
  "https://www.scribd.com/document/176345714/V-V-AXXX-XXX-678-BIA-Oct-4-2013",
  "https://www.scribd.com/document/175364978/Tatiana-Vasquez-A098-079-947-BIA-March-31-2011",
  "https://www.scribd.com/document/175363608/Lee-Soon-Park-A072-972-500-BIA-March-31-2011",
  "https://www.scribd.com/document/175361890/Jose-Zacaria-Quinteros-A088-239-850-BIA-Mar-31-2011",
  "https://www.scribd.com/document/175358107/Luis-Alejandro-Enriquez-Palafox-A075-446-189-BIA-March-29-2011",
  "https://www.scribd.com/document/175356429/George-Ndungu-Kiragu-A094-217-234-BIA-March-29-2011",
  "https://www.scribd.com/doc/175355152/Salvador-Gonzales-Diaz-A030-105-700-BIA-March-28-2011",
  "https://www.scribd.com/document/175349729/Rabih-Almasri-A075-415-097-BIA-March-25-2011",
  "https://www.scribd.com/document/175348536/Maria-Sabastiana-Gonzalez-Ventura-A076-318-715-BIA-March-25-2011",
  "https://www.scribd.com/document/175346640/Maria-Charissa-Perez-Arriola-A088-741-448-BIA-March-25-2011",
  "https://www.scribd.com/document/175344644/Luis-Fernando-Martinez-A089-748-621-BIA-March-25-2011",
  "https://www.scribd.com/document/175327826/Bitar-Turou-A205-829-460-BIA-Oct-4-2013",
  "https://www.scribd.com/document/175158264/Erika-Zulema-Ortis-Umanas-A078-324-001-BIA-March-23-2011",
  "https://www.scribd.com/document/175154120/Anna-Margebadze-A096-021-996-BIA-March-21-2011",
  "https://www.scribd.com/document/175148703/Epifanio-Martinez-Juarez-A095-194-852-BIA-March-21-2011",
  "https://www.scribd.com/document/175144968/Luis-Alberto-Gonzalez-Banda-A078-469-139-BIA-March-21-2011",
  "https://www.scribd.com/document/175130160/Everton-Dane-Henry-A089-425-229-BIA-March-18-2011",
  "https://www.scribd.com/document/175124808/Mohammad-Hashemian-Tabriz-A046-960-320-BIA-March-17-2011",
  "https://www.scribd.com/document/175121095/Michelin-Fernand-Verne-A076-923-324-BIA-March-16-2011",
  "https://www.scribd.com/document/175110575/Ismail-Ibrahim-Abdelfattah-Mohammad-Khalil-A097-659-194-BIA-March-16-2011",
  "https://www.scribd.com/document/175108438/Oliver-Sherman-Muli-Wood-A041-164-168-BIA-March-16-2011",
  "https://www.scribd.com/document/175105004/Juan-Pablo-Villafana-Ramos-A070-734-138-BIA-March-15-2011",
  "https://www.scribd.com/doc/175073552/Lilibeth-Marquina-Romero-A096-018-999-BIA-March-14-2011",
  "https://www.scribd.com/document/175071609/Andelino-Vicente-Velasquez-Perez-A200-936-429-BIA-March-11-2011",
  "https://www.scribd.com/document/175064566/Wilson-Ivan-Mendez-Acevedo-A073-868-517-BIA-March-11-2011",
  "https://www.scribd.com/document/175062498/Michael-Eshun-A089-439-806-BIA-March-11-2011",
  "https://www.scribd.com/document/175061759/Antonio-Salazar-Montiel-A091-611-884-BIA-March-10-2011",
  "https://www.scribd.com/document/174754693/Alejandro-Hernandez-Garcia-A091-097-894-BIA-March-10-2011",
  "https://www.scribd.com/document/174742441/Francisco-Zendejas-Sanchez-A036-176-508-BIA-March-8-2011",
  "https://www.scribd.com/document/174561872/Gregorio-Vergara-A076-110-704-BIA-March-8-2011",
  "https://www.scribd.com/document/174551370/Julio-Cesar-Rios-A090-318-119-BIA-March-3-2011",
  "https://www.scribd.com/document/174547106/Joseph-Adams-Mpambile-A097-962-673-BIA-March-3-2011",
  "https://www.scribd.com/document/174543466/Edward-Mercado-A095-443-759-BIA-March-1-2011",
  "https://www.scribd.com/document/174541678/Jose-Noel-Meza-Perez-A029-269-568-BIA-Feb-28-2011",
  "https://www.scribd.com/document/174474487/Evgeny-Korinzer-A020-845-955-BIA-February-28-2011",
  "https://www.scribd.com/document/174471164/Susana-Beatriz-Vasquez-Ramos-A094-922-883-BIA-Feb-25-2011",
  "https://www.scribd.com/document/174465658/Susana-Lilibeth-Ferreira-Neves-A036-464-449-BIA-Feb-25-2011",
  "https://www.scribd.com/document/174458384/Estelito-Jr-Carpio-Adiova-A047-899-129-BIA-Feb-24-2011",
  "https://www.scribd.com/document/174455028/Pervez-Pasha-A042-436-375-BIA-Feb-24-2011",
  "https://www.scribd.com/document/174452655/Jorge-Sulbaran-A087-075-003-BIA-Feb-23-2011",
  "https://www.scribd.com/document/174449807/Anthony-Ariel-German-A099-232-178-BIA-Feb-22-2011",
  "https://www.scribd.com/document/174446497/Alex-Gamboa-A075-924-620-BIA-Feb-22-2011",
  "https://www.scribd.com/document/174137507/Ashot-Zhorahikovic-Ovsepyan-A045-201-416-BIA-Feb-17-2011",
  "https://www.scribd.com/document/174135001/Victor-Cervantes-Sanchez-A097-838-854-BIA-Feb-16-2011",
  "https://www.scribd.com/document/174132076/Omar-Cruz-A089-965-334-BIA-Feb-11-2011",
  "https://www.scribd.com/document/174130751/Miguel-Angel-Gutierrez-Melendez-A076-882-990-BIA-Feb-11-2011",
  "https://www.scribd.com/document/174129749/Maria-Aurora-Montes-de-Rodriguez-A043-950-329-BIA-Feb-11-2011",
  "https://www.scribd.com/document/174129027/Philip-Kenneth-Shirley-A087-066-710-BIA-Jan-21-2011",
  "https://www.scribd.com/document/174127684/Marcelo-Herrera-Martinez-A098-007-723-BIA-Feb-11-2011",
  "https://www.scribd.com/document/174126727/Braulio-Lituma-A077-027-270-BIA-Feb-11-2011-pdf",
  "https://www.scribd.com/document/173386313/Lesmany-Nunez-A074-019-651-BIA-Feb-9-2011",
  "https://www.scribd.com/document/173382540/Rodrigo-Gonzalez-Sanchez-A098-263-984-BIA-Feb-8-2011",
  "https://www.scribd.com/doc/173380708/Armando-G-Guardado-A094-392-670-BIA-Feb-3-2011",
  "https://www.scribd.com/document/173376881/Jesus-Manuel-Corral-Chavira-A092-800-902-BIA-Feb-1-2011",
  "https://www.scribd.com/document/173358349/Oliverio-Millan-Villegas-A091-874-177-BIA-Jan-28-2011",
  "https://www.scribd.com/document/173144045/Cesar-Brenner-Varillas-Calderon-A076-520-895-BIA-Jan-28-2011",
  "https://www.scribd.com/document/173131375/Gumaro-Huerta-Cruz-A092-735-405-BIA-Jan-25-2011",
  "https://www.scribd.com/document/173125745/Elden-Misael-Pantoja-Cabrera-A087-522-960-BIA-Jan-25-2011",
  "https://www.scribd.com/document/173120819/Claudine-Andeena-Ramdon-A045-878-319-BIA-Jan-11-2011",
  "https://www.scribd.com/document/173113823/Tashthed-Haider-Kabir-A096-114-231-BIA-Jan-21-2011",
  "https://www.scribd.com/document/173104635/Lindon-Ingram-A055-557-778-BIA-Jan-21-2011",
  "https://www.scribd.com/document/173103309/Benjamin-Luis-Garcia-A098-237-658-BIA-Jan-21-2011",
  "https://www.scribd.com/document/173102127/Eddy-Jose-Ramirez-A028-859-292-BIA-Jan-20-2011",
  "https://www.scribd.com/document/173099928/Abril-Carrera-Luciano-A088-749-575-BIA-Jan-20-2011",
  "https://www.scribd.com/document/172846700/Francisco-Ramirez-Reyes-A072-988-089-BIA-Jan-19-2011",
  "https://www.scribd.com/document/172833630/Jose-Miguel-Argueta-Sandoval-A046-177-199-BIA-Jan-18-2011",
  "https://www.scribd.com/document/172828265/Jesus-Enrique-Mendoza-Benitez-A092-161-477-BIA-Jan-18-2011",
  "https://www.scribd.com/document/172824342/Sadruddin-Ali-Bhai-Rajani-A074-088-126-BIA-Jan-12-2011",
  "https://www.scribd.com/document/172818499/Jorge-Enrique-Torres-A099-925-343-BIA-Jan-11-2011",
  "https://www.scribd.com/document/172811057/Delroy-Patterson-A035-685-599-BIA-Jan-10-2011",
  "https://www.scribd.com/document/172807092/Jose-Enrique-Perez-Veneros-A095-790-916-BIA-Jan-7-2011",
  "https://www.scribd.com/document/172798512/Francisco-Jose-Alvarez-Troncoso-A057-287-860-BIA-Jan-6-2011",
  "https://www.scribd.com/document/172793509/Chun-Lam-Chan-A047-957-053-BIA-Jan-6-2011",
  "https://www.scribd.com/document/172790788/Amir-Ali-Jalia-A096-751-896-BIA-Jan-6-2011",
  "https://www.scribd.com/document/172548482/Betania-Generosa-Madera-A041-592-043-BIA-Jan-5-2011",
  "https://www.scribd.com/document/172540461/Sandra-Flores-Nieto-A087-755-789-BIA-Jan-3-2011",
  "https://www.scribd.com/document/172537056/Rui-Li-A095-529-382-BIA-Jan-3-2011",
  "https://www.scribd.com/document/172532576/Kingsley-Chiagozie-Ezinwa-Ibeh-A055-574-599-BIA-Nov-4-2010",
  "https://www.scribd.com/document/172414600/M-N-AXX-XX8-795-BIA-Sept-26-2013",
  "https://www.scribd.com/document/172142987/Oscar-Giovanni-Castro-Rosa-A205-828-164-BIA-Sept-25-2013",
  "https://www.scribd.com/document/172140903/William-Noe-Perez-Cortez-A205-383-123-BIA-Sept-25-2013",
  "https://www.scribd.com/document/172138632/Andrei-Jose-Picado-A200-440-226-BIA-Sept-25-2013",
  "https://www.scribd.com/document/172137336/Ramon-Jose-Henares-A089-658-481-BIA-Sept-25-2013",
  "https://www.scribd.com/document/172135916/Juan-Pablo-Soto-Gonzalez-A200-562-227-BIA-Sept-24-2013",
  "https://www.scribd.com/document/172134227/Victor-Manuel-Parada-Villegas-A091-223-049-BIA-Sept-23-2013",
  "https://www.scribd.com/document/172132293/Edgar-Ortega-A045-624-720-BIA-Sept-23-2013",
  "https://www.scribd.com/document/171952033/Luis-Narciso-Sedeno-Trujillo-A088-190-240-BIA-Sept-22-2010",
  "https://www.scribd.com/document/171946698/Isaias-Rodriguez-Ovalle-A070-090-008-BIA-April-2-2010",
  "https://www.scribd.com/document/171938750/Julio-Cesar-Rios-A090-318-119-BIA-March-31-2010",
  "https://www.scribd.com/document/171917421/Tabassum-Saleheen-A097-967-736-BIA-July-20-2009",
  "https://www.scribd.com/document/171913772/Juan-Manuel-Llamas-Lopez-A24-101-784-BIA-Feb-16-2007",
  "https://www.scribd.com/document/171881154/Jose-R-Nolasco-A94-127-863-BIA-March-12-2004",
  "https://www.scribd.com/document/170937978/Salvador-Hernandez-Garcia-A097-472-829-BIA-Sept-20-2013",
  "https://www.scribd.com/document/170936484/Phillip-Andrew-Murray-A055-213-731-BIA-Sept-20-2013",
  "https://www.scribd.com/document/170935273/Glendon-A-White-A024-619-428-BIA-Sept-19-2013",
  "https://www.scribd.com/document/170933440/Carlos-Eenesto-Valle-Iglesias-A043-995-062-BIA-Sept-18-2013",
  "https://www.scribd.com/document/170930462/Marcelo-Castorena-Alonso-A205-866-201-BIA-Sept-17-2013",
  "https://www.scribd.com/document/170927699/Cesar-Medrano-Arias-A201-071-535-BIA-Sept-17-2013",
  "https://www.scribd.com/document/169227284/Miguel-Augusto-Ixpec-Chitay-A097-535-400-BIA-Sept-16-2013",
  "https://www.scribd.com/document/169226205/Fernando-Placencia-Castellanos-A205-285-337-BIA-Sept-16-2013",
  "https://www.scribd.com/document/169225121/Vivian-Rosibel-Contreras-Santos-A097-834-770-BIA-Sept-16-2013",
  "https://www.scribd.com/document/169224239/Raymond-Dakura-A087-673-826-BIA-Sept-13-2013",
  "https://www.scribd.com/document/169222419/Juan-Aguilar-Perez-A027-190-905-BIA-Sept-13-2013",
  "https://www.scribd.com/document/169221655/Yong-Chul-Kim-A098-447-978-BIA-Sept-12-2013",
  "https://www.scribd.com/document/169189929/Lennox-S-Reece-A034-707-338-BIA-Sept-11-2013",
  "https://www.scribd.com/document/167980959/Juan-Jose-Benitez-Recino-A201-176-938-BIA-Sept-10-2013",
  "https://www.scribd.com/document/167979354/Domingo-Pedro-Salvador-A087-008-675-BIA-Sept-10-2013",
  "https://www.scribd.com/document/167978291/Erwin-Stuardo-Vivar-Flores-A029-249-620-BIA-Sept-10-2013",
  "https://www.scribd.com/document/167977028/Carla-Adriana-Vazquez-Ferrel-A099-222-179-BIA-Sept-9-2013",
  "https://www.scribd.com/document/167974323/Nana-Antwi-Boasiako-A096-835-772-BIA-Sept-9-2013",
  "https://www.scribd.com/document/167122318/Felix-Musembi-Kyalo-A200-578-009-BIA-Sept-6-2013",
  "https://www.scribd.com/document/167121475/Sebastian-Lopez-Vasquez-A087-676-888-BIA-Sept-6-2013",
  "https://www.scribd.com/document/167120344/Alfred-Ajanga-Ahuta-A200-229-217-BIA-Sept-5-2013",
  "https://www.scribd.com/document/167118992/Jose-Gabriel-Martinez-A094-190-065-BIA-Sept-5-2013",
  "https://www.scribd.com/document/167117670/Odalis-Cabreja-Arias-A087-771-705-BIA-Sept-5-2013",
  "https://www.scribd.com/document/167116270/Hamid-Choudhry-A073-538-314-BIA-Sept-4-2013",
  "https://www.scribd.com/document/166048143/Bryan-Sanchez-Vazquez-A205-213-774-BIA-Aug-30-2013",
  "https://www.scribd.com/document/166046885/Raul-Eduardo-Juarez-A201-215-858-BIA-Aug-30-2013",
  "https://www.scribd.com/document/166045427/Guadalupe-Rondin-Nieves-A087-756-633-BIA-Aug-30-2013",
  "https://www.scribd.com/document/166043949/Ricardo-Cisneros-A075-789-006-BIA-Aug-30-2013",
  "https://www.scribd.com/document/166042317/Juan-Carlos-Gomez-A041-591-459-BIA-Aug-30-2013",
  "https://www.scribd.com/document/166041160/Domingo-Pacheco-Garcia-A205-062-933-BIA-Aug-29-2013",
  "https://www.scribd.com/document/166040087/Giovanni-Arroyo-Valverde-A205-753-076-BIA-Aug-28-2013",
  "https://www.scribd.com/document/165242822/Mahvash-Akram-A096-769-035-BIA-Aug-27-2013",
  "https://www.scribd.com/document/165227167/K-O-A-BIA-Aug-27-2013",
  "https://www.scribd.com/document/165221914/Dave-Lall-A095-571-038-BIA-Aug-27-2013",
  "https://www.scribd.com/document/165216552/Oscar-Antonio-Del-Cid-Melara-A205-875-351-BIA-Aug-26-2013",
  "https://www.scribd.com/document/165213728/Juan-Melvin-Perez-Baez-A079-213-261-BIA-Aug-26-2013",
  "https://www.scribd.com/document/163798690/Artemio-Guzman-Rodriguez-A200-234-211-BIA-Aug-23-2013",
  "https://www.scribd.com/document/163798031/Gladys-Margarita-Hernandez-A095-080-333-BIA-Aug-23-2013",
  "https://www.scribd.com/document/163796031/Nishad-Yogesh-Pandya-A088-276-788-BIA-Aug-22-2013",
  "https://www.scribd.com/document/163795187/Mirriam-Nekesa-Barasa-A087-905-491-BIA-Aug-22-2013",
  "https://www.scribd.com/document/163790994/Salomon-Martinez-Banuelos-A044-343-239-BIA-Aug-22-2013",
  "https://www.scribd.com/document/163789129/Manuel-Waldo-Gonzalez-Leyton-A073-809-412-BIA-Aug-21-2013",
  "https://www.scribd.com/document/163169412/Santokh-Singh-Parihar-A077-596-351-BIA-Aug-20-2013",
  "https://www.scribd.com/document/163167614/Oliver-Garcia-Guzman-A043-144-667-BIA-Aug-20-2013",
  "https://www.scribd.com/document/163166699/James-Martin-Lawlor-A038-733-245-BIA-Aug-16-2013",
  "https://www.scribd.com/document/163164902/Rogelio-Blackman-Hinds-A035-197-709-BIA-Aug-16-2013",
  "https://www.scribd.com/document/163163874/Tristan-Nathaniel-Francis-Gould-A027-911-731-BIA-Aug-16-2013",
  "https://www.scribd.com/document/163162909/Noe-Cuevas-Beltran-A205-153-916-BIA-Aug-15-2013",
  "https://www.scribd.com/document/163161368/Fernando-Barajas-Acevedo-A087-906-037-BIA-Aug-15-2013",
  "https://www.scribd.com/document/163159537/Sewdat-Rajpaul-A058-358-908-BIA-Aug-15-2013",
  "https://www.scribd.com/document/162206538/Wilson-Orlando-Escobar-A095-082-121-BIA-Aug-14-2013",
  "https://www.scribd.com/document/162204597/Ruben-Dario-Diaz-A090-661-830-BIA-Aug-14-2013",
  "https://www.scribd.com/document/162202178/Farzad-Saeed-Qureshi-A098-144-820-BIA-Aug-14-2013",
  "https://www.scribd.com/document/162198640/Jose-Luis-Ugas-Gil-A078-903-173-BIA-Aug-14-2013",
  "https://www.scribd.com/document/162197578/Gilberto-Aguilera-A093-006-263-BIA-Aug-14-2013",
  "https://www.scribd.com/document/162196677/Safraz-Khan-A043-452-893-BIA-Aug-14-2013",
  "https://www.scribd.com/document/162195573/Juan-Rodriguez-Martinez-A096-355-621-BIA-Aug-14-2013",
  "https://www.scribd.com/document/161413692/Marvin-Josue-Garcia-A097-535-345-BIA-Aug-13-2013",
  "https://www.scribd.com/document/161412508/Samuel-Owusu-Jr-A095-329-362-BIA-Aug-13-2013",
  "https://www.scribd.com/document/161411343/Dina-Issa-Muasher-A079-000-609-BIA-Aug-13-2013",
  "https://www.scribd.com/document/161410016/Ana-Maria-Scaranni-A089-118-049-BIA-Aug-13-2013",
  "https://www.scribd.com/document/161409054/Socchea-Yav-A089-655-240-BIA-Aug-13-2013",
  "https://www.scribd.com/document/161407958/Pablo-Travieso-Izquierdo-A087-518-131-BIA-Aug-12-2013",
  "https://www.scribd.com/document/161406520/Emmanuel-Peterson-Dontoh-A088-949-492-BIA-Aug-12-2013",
  "https://www.scribd.com/document/161404940/Jorge-Eduardo-Araiza-A201-221-422-BIA-Aug-12-2013",
  "https://www.scribd.com/document/160235538/Hervens-Timothe-A023-590-819-BIA-Aug-9-2013",
  "https://www.scribd.com/document/160234895/Osman-Orozco-Garcia-A043-745-973-BIA-Aug-9-2013",
  "https://www.scribd.com/document/160232718/Beverly-G-Burdeos-A200-574-873-BIA-Aug-8-2013",
  "https://www.scribd.com/document/160231180/Antonio-Roa-Carranza-A200-762-041-BIA-Aug-8-2013",
  "https://www.scribd.com/document/160229759/Vicente-Somera-Espino-III-A087-081-219-BIA-Aug-5-2013",
  "https://www.scribd.com/document/159161657/Noe-Jesus-Ascencio-Hernandez-A200-819-649-BIA-Aug-6-2013",
  "https://www.scribd.com/document/159160691/Edgar-Camacho-Infanzan-A089-956-952-BIA-Aug-6-2013",
  "https://www.scribd.com/document/159159104/Renat-Tokbaev-A089-274-160-BIA-Aug-6-2013",
  "https://www.scribd.com/document/159158431/Marco-Tulio-Ramirez-Samuel-A043-998-246-BIA-Aug-2-2013",
  "https://www.scribd.com/document/159157115/Rodrigue-Laventure-A018-098-964-BIA-Aug-2-2013",
  "https://www.scribd.com/document/158529516/Lorna-Maynigo-A094-876-389-BIA-July-31-2013",
  "https://www.scribd.com/document/158528012/Gzim-Imeri-A091-298-819-BIA-July-31-2013",
  "https://www.scribd.com/document/158527155/Jaqueline-Zanelatto-Martins-A089-249-051-BIA-July-31-2013",
  "https://www.scribd.com/document/158526037/Jeffry-Nelson-A058-001-093-BIA-July-31-2013",
  "https://www.scribd.com/document/157698671/Gustavo-Alexis-Murillo-Borjas-A097-744-763-BIA-July-30-2013",
  "https://www.scribd.com/document/157697873/Mark-L-Edmond-A099-116-705-BIA-July-29-2013",
  "https://www.scribd.com/document/157697253/Lucero-Pulido-Aguirres-A095-784-426-BIA-July-29-2013",
  "https://www.scribd.com/document/157696330/Fazliddin-Bahronov-A088-748-664-BIA-July-29-2013",
  "https://www.scribd.com/document/157462309/Willian-Josue-Rosales-Lazo-A095-021-153-BIA-April-21-2011",
  "https://www.scribd.com/document/157460506/Juan-Pablo-Garcia-Garcia-A087-940-270-BIA-March-4-2011",
  "https://www.scribd.com/document/157458063/Gilberto-Dominguez-Gutierrez-A201-021-861-BIA-March-21-2011",
  "https://www.scribd.com/document/157447990/Francisco-Onate-Vazquez-A079-362-130-BIA-April-14-2011",
  "https://www.scribd.com/document/157445906/Edgar-Armando-Valenzuela-Garcia-A079-651-539-BIA-March-10-2011",
  "https://www.scribd.com/document/157443290/Claudia-Lorena-Santos-Baca-A077-387-355-BIA-Feb-23-2011",
  "https://www.scribd.com/document/157441876/Daniel-Santiago-Jimenez-A097-628-426-BIA-April-27-2011",
  "https://www.scribd.com/document/157439932/Anthony-Ariel-German-A099-232-178-BIA-Feb-11-2011",
  "https://www.scribd.com/document/157437447/Gerardo-Candelario-Torres-A087-968-160-BIA-Jan-14-2011",
  "https://www.scribd.com/document/157032648/Tameka-Lorraine-Butler-A097-998-935-BIA-July-26-2013",
  "https://www.scribd.com/document/157031161/Maria-Regina-Enriquez-A090-496-769-BIA-July-26-2013",
  "https://www.scribd.com/document/157030162/Varinder-Singh-A201-156-040-BIA-July-23-2013",
  "https://www.scribd.com/document/157028755/Mohamad-Mahmoud-Abdul-Karim-A096-419-142-BIA-July-23-2013",
  "https://www.scribd.com/document/157027684/Jose-Gabriel-Amaya-A094-853-698-BIA-July-23-2013",
  "https://www.scribd.com/document/157026705/Joselito-Vasquez-Gomez-A094-222-573-BIA-July-23-2013",
  "https://www.scribd.com/document/156156423/Omar-Antonio-Velasquez-Aguirre-A099-477-431-BIA-July-22-2013",
  "https://www.scribd.com/document/156155147/Omar-Antonio-Velasquez-Aguirre-A099-477-431-BIA-Jan-18-2013",
  "https://www.scribd.com/document/156153861/Omar-Antonio-Velasquez-Aguirre-A099-477-431-BIA-Nov-19-2012",
  "https://www.scribd.com/document/156152872/Miguel-Vazquez-Nino-A200-676-857-BIA-July-22-2013",
  "https://www.scribd.com/document/156151789/Luis-Alberto-Lopez-Romero-A074-268-627-BIA-July-22-2013",
  "https://www.scribd.com/document/156150827/Ignacio-Javier-Perez-Hernandez-A092-259-726-BIA-July-18-2013",
  "https://www.scribd.com/document/155931594/Carlos-Antonio-Taracena-Herrera-A092-446-911-BIA-Feb-28-2011",
  "https://www.scribd.com/document/155930372/Eddy-Bismark-Nunez-Garrido-A099-115-048-BIA-Feb-3-2011",
  "https://www.scribd.com/document/155928968/Helia-de-La-Cruz-Palencia-A074-374-352-BIA-May-13-2011",
  "https://www.scribd.com/document/155927628/Marcelino-Simbron-Sanchez-A201-029-423-BIA-April-21-2011",
  "https://www.scribd.com/document/155925695/Ricardo-Jose-Rodriguez-A200-886-952-BIA-April-14-2011",
  "https://www.scribd.com/document/155924664/Salvador-Jr-Villareal-A092-722-540-BIA-March-17-2011",
  "https://www.scribd.com/document/155923493/Miguel-Barron-Villeda-A093-342-975-BIA-Feb-18-2011",
  "https://www.scribd.com/document/155530286/Ruben-Dario-Sanchez-Esquivel-A035-727-884-BIA-July-19-2013",
  "https://www.scribd.com/document/155528822/Ohene-Bobie-Nimako-A099-311-604-BIA-July-19-2013",
  "https://www.scribd.com/document/155527533/Francisco-Ruiz-Witchar-A087-951-655-BIA-July-17-2013",
  "https://www.scribd.com/document/154751909/Alexander-George-Fernandes-A097-644-447-BIA-July-16-2013",
  "https://www.scribd.com/document/154748623/Jose-Amaya-Portillo-A094-216-651-BIA-July-16-2013",
  "https://www.scribd.com/document/154747255/Omar-Ciro-Cruz-Rodriguez-A200-885-026-BIA-July-12-2013",
  "https://www.scribd.com/document/154746246/Pedro-Ramos-Garcia-A092-167-389-BIA-July-12-2013",
  "https://www.scribd.com/document/153970780/Jesus-Ivan-Ruiz-Estrada-A095-129-850-BIA-July-11-2013",
  "https://www.scribd.com/document/153932122/Kosei-Inoue-A089-244-823-BIA-July-9-2013",
  "https://www.scribd.com/document/153926969/Jorge-Ramirez-Banda-A086-922-186-BIA-July-9-2013",
  "https://www.scribd.com/document/153926065/Felicitas-Urias-Baeza-A041-776-048-BIA-July-9-2013",
  "https://www.scribd.com/document/153925212/Sangchul-Lee-A200-298-115-BIA-July-5-2013",
  "https://www.scribd.com/document/153924506/Francisco-Alcala-Rodriguez-A041-395-733-BIA-July-5-2013",
  "https://www.scribd.com/document/153923759/Oyekunle-Oyeniyi-A099-277-993-BIA-June-28-2013",
  "https://www.scribd.com/document/153923098/Danilo-Rejano-A076-210-552-BIA-June-28-2013",
  "https://www.scribd.com/document/152724704/Orlando-Garcia-Adan-A200-151-857-BIA-July-3-2013",
  "https://www.scribd.com/document/152723372/Ambrosio-Lorenzo-Aguilar-A075-408-836-BIA-July-2-2013",
  "https://www.scribd.com/document/152500690/Edita-Magdirila-Baua-A097-167-177-BIA-June-27-2013",
  "https://www.scribd.com/document/152499667/Jose-de-Jesus-Alvarez-Gudino-A095-748-846-BIA-June-26-2013",
  "https://www.scribd.com/document/152498939/Gerardo-Antonio-Gutierrez-Cabrera-A205-760-561-BIA-June-25-2013",
  "https://www.scribd.com/document/152497625/Javier-Escamilla-A076-534-101-BIA-June-25-2013",
  "https://www.scribd.com/document/152496351/Antonio-Moscol-Bernard-A035-047-966-BIA-June-25-2013",
  "https://www.scribd.com/document/152494011/Ernesto-Villazana-Banuelos-A037-837-474-BIA-June-25-2013",
  "https://www.scribd.com/document/152490842/Maria-Ireri-Rivas-A089-573-062-BIA-June-24-2013",
  "https://www.scribd.com/document/151355671/Jose-Adalid-Mendoza-Dias-A077-794-526-BIA-June-21-2013",
  "https://www.scribd.com/document/151354536/Redentor-Celino-Castrence-A040-495-188-BIA-June-21-2013",
  "https://www.scribd.com/document/151352893/Abdou-Koudous-Adissa-A205-656-760-BIA-June-20-2013",
  "https://www.scribd.com/document/151351938/Caroline-Rebecca-Ross-A087-202-743-BIA-June-20-2013",
  "https://www.scribd.com/document/151350994/Pablo-Nunez-Serrano-A041-402-248-BIA-June-20-2013",
  "https://www.scribd.com/document/151349891/Bernabe-Roman-Castillo-A205-587-621-BIA-June-19-2013",
  "https://www.scribd.com/document/150482340/Pedro-Malvais-Hernandez-A089-284-493-BIA-Jul-17-2012",
  "https://www.scribd.com/document/150133888/Christian-Rodriguez-A088-190-226-BIA-June-18-2013",
  "https://www.scribd.com/document/150133885/Diego-Alejandro-Alonso-Nuno-A089-711-493-BIA-June-17-2013",
  "https://www.scribd.com/document/150133886/Alisha-Rahim-Robinson-A075-559-747-BIA-June-18-2013",
  "https://www.scribd.com/document/149760179/Tamara-Jackeline-Aleman-A073-110-365-BIA-June-18-2013",
  "https://www.scribd.com/document/149758883/Mamoudou-Camara-A076-433-169-BIA-June-17-2013",
  "https://www.scribd.com/document/149757124/Robin-Joselito-Barrios-de-Leon-A087-472-600-BIA-June-14-2013",
  "https://www.scribd.com/document/149755079/Karen-Louise-Difeo-A074-093-925-BIA-June-14-2013",
  "https://www.scribd.com/document/148577141/Immacula-Presmy-A095-517-631-BIA-June-13-2013",
  "https://www.scribd.com/document/148573755/Gustavo-DeJesus-Garcia-Lainez-A205-655-936-BIA-June-12-2013",
  "https://www.scribd.com/document/148572713/Rodrigo-Ancanjo-Alves-A097-835-973-BIA-June-10-2013",
  "https://www.scribd.com/document/148571817/Ray-Lindsay-Simon-A044-269-366-BIA-June-11-2013",
  "https://www.scribd.com/document/148569372/Patricia-Angelica-Aguilar-A090-599-476-BIA-June-6-2013",
  "https://www.scribd.com/document/148568433/Segundo-Balay-Navarro-Jr-A087-213-062-BIA-June-6-2013",
  "https://www.scribd.com/document/148568002/Eden-Francois-A075-442-065-BIA-June-6-2013",
  "https://www.scribd.com/document/148377498/Ryan-Damion-Coulbourne-A041-457-839-BIA-Jan-18-2011",
  "https://www.scribd.com/document/148376065/Asbert-Fitzgerald-Joseph-A036-805-976-BIA-Sep-27-2011",
  "https://www.scribd.com/document/148374107/Muamar-Sayyed-Asad-A042-164-835-BIA-Feb-8-2012",
  "https://www.scribd.com/doc/148371778/Hyun-Jung-Kim-A070-972-045-BIA-Apr-23-2012",
  "https://www.scribd.com/document/148370353/Heyman-Martinez-A073-661-308-BIA-July-27-2012",
  "https://www.scribd.com/document/148360102/Francisco-Cruz-Alvarez-A076-375-249-BIA-Feb-28-2011",
  "https://www.scribd.com/document/148357501/Cheddi-Desmond-Jr-Dillon-A040-097-680-BIA-May-10-2011",
  "https://www.scribd.com/document/148357500/Cheddi-Desmond-Jr-Dillon-A040-097-680-BIA-Oct-24-2011",
  "https://www.scribd.com/document/148355719/Odane-Carlton-Dixon-A045-881-695-BIA-Mar-20-2012",
  "https://www.scribd.com/document/148352335/Jennsey-Josue-Zarzuela-A044-821-167-BIA-Dec-8-2011",
  "https://www.scribd.com/document/148352334/Jennsey-Josue-Zarzuela-A044-821-167-BIA-April-5-2011",
  "https://www.scribd.com/doc/148348819/Carlos-Arturo-Sequeira-A043-136-860-BIA-Aug-3-2011",
  "https://www.scribd.com/doc/148348818/Carlos-Arturo-Sequeira-A043-136-860-BIA-Sept-30-2011",
  "https://www.scribd.com/document/148344577/Enrique-Ramos-A022-833-741-BIA-Mar-8-2012",
  "https://www.scribd.com/document/148344578/Enrique-Ramos-A022-833-741-BIA-April-8-2011",
  "https://www.scribd.com/document/147667269/Ricardo-J-Sinclair-A095-965-914-BIA-June-23-2011",
  "https://www.scribd.com/document/147500301/Maria-Isabel-Perez-Castro-A095-266-046-BIA-Sept-5-2012",
  "https://www.scribd.com/document/147185911/Dolores-Soliz-Vigil-A077-240-379-BIA-May-31-2013",
  "https://www.scribd.com/document/147181719/Marie-N-Peli-A099-273-416-BIA-May-31-2013",
  "https://www.scribd.com/document/147180265/Tatiana-Johns-A046-791-561-BIA-June-4-2013",
  "https://www.scribd.com/document/147179006/Celeste-Ramirez-Perez-A205-672-704-BIA-June-4-2013",
  "https://www.scribd.com/document/146969066/Imad-Hani-Jubran-A047-214-217-BIA-Feb-25-2011",
  "https://www.scribd.com/document/146130394/Elisa-Cherone-Rojas-A087-947-634-BIA-May-27-2011",
  "https://www.scribd.com/document/146101450/Olumide-Goodness-Adeyinka-A099-182-755-BIA-April-15-2011",
  "https://www.scribd.com/document/146070174/Jose-Luis-Ramirez-A075-986-662-BIA-May-31-2013",
  "https://www.scribd.com/document/145933709/Richard-Michreka-Nyamwange-A029-043-107-BIA-March-17-2011",
  "https://www.scribd.com/document/145871549/Sumithra-Keerthi-Prasanna-Fernando-Conganige-A088-035-796-BIA-May-28-2013",
  "https://www.scribd.com/document/145869741/Dung-Tuan-Pham-A027-824-163-BIA-May-30-2013",
  "https://www.scribd.com/document/145702770/Hubert-Lukasz-Gul-A055-902-141-BIA-May-29-2013",
  "https://www.scribd.com/document/144946742/Arsalan-Azeem-A099-030-766-BIA-May-20-2013",
  "https://www.scribd.com/document/144945357/Son-Hoang-Nguyen-A097-683-305-BIA-May-16-2013",
  "https://www.scribd.com/document/144781260/Ricardo-Mata-Rosas-A076-404-712-BIA-April-20-2011",
  "https://www.scribd.com/document/144779160/Sergio-Gonzalez-Manjarrez-A093-108-092-BIA-May-22-2013",
  "https://www.scribd.com/document/144778217/Eliza-Valdez-Bernudez-Bautista-A035-383-901-BIA-May-22-2013",
  "https://www.scribd.com/document/144776986/Sheila-Sofia-Diaz-Aulis-A205-131-206-BIA-May-16-2013",
  "https://www.scribd.com/document/144068181/Eduardo-Yanez-A076-593-027-BIA-Oct-18-2012",
  "https://www.scribd.com/document/143126317/Jacks-Guy-Nkounga-Chattuais-A098-063-263-BIA-May-15-2013",
  "https://www.scribd.com/document/143119343/Ernesto-Gonzalez-Valencia-A088-063-861-BIA-May-14-2013",
  "https://www.scribd.com/document/142758672/Cuauhtemoc-Jesus-Morales-A095-949-625-BIA-Oct-18-2012",
  "https://www.scribd.com/document/142757632/Blanca-Lidia-Alfaro-Serrano-A098-121-479-BIA-Oct-25-2012",
  "https://www.scribd.com/document/142755206/Dhyana-Aderne-Goltz-A045-296-896-BIA-Nov-5-2012",
  "https://www.scribd.com/document/142755205/Dhyana-Aderne-Goltz-A045-296-896-BIA-Jun-12-2012",
  "https://www.scribd.com/document/142099972/Manuel-Jesus-Olivas-Matta-A021-179-705-BIA-Aug-9-2010",
  "https://www.scribd.com/document/141952946/Sydney-Okine-A097-168-058-BIA-May-10-2013",
  "https://www.scribd.com/document/141951646/Carmen-Anayanci-Portillo-Munoz-A077-793-500-BIA-May-10-2013",
  "https://www.scribd.com/document/141949784/Francisco-Flores-Alcala-A200-762-691-BIA-May-9-2013",
  "https://www.scribd.com/document/141948107/Ireneusz-Fraczek-A030-973-737-BIA-May-9-2013",
  "https://www.scribd.com/document/141947077/Erick-Guerrero-Silva-A071-914-925-BIA-May-7-2013",
  "https://www.scribd.com/document/141783884/Enrique-Soto-Garcia-A087-534-842-BIA-May-7-2013",
  "https://www.scribd.com/document/140630293/Jose-Javier-Linares-Valencia-A059-170-269-BIA-Aug-23-2012",
  "https://www.scribd.com/document/140612850/David-M-Zavala-A020-683-429-BIA-Apr-30-2012",
  "https://www.scribd.com/document/140609243/Abu-Bakarr-Dizo-Kamara-A200-515-968-BIA-Jun-8-2012",
  "https://www.scribd.com/document/140607450/Felicisimo-Cambronero-Lorbes-A078-003-871-BIA-Jun-26-2012",
  "https://www.scribd.com/document/140601622/Eden-Garcia-Castro-A090-065-334-BIA-Dec-29-2011",
  "https://www.scribd.com/document/140201051/Hai-Van-Ho-A096-733-063-BIA-Apr-30-2013",
  "https://www.scribd.com/document/140198431/Maria-Guadalupe-Ochoa-Alcantar-A076-610-961-BIA-Apr-30-2013",
  "https://www.scribd.com/document/140196719/Oscar-Antonio-Pando-Uribe-A044-355-610-BIA-Apr-30-2013",
  "https://www.scribd.com/document/140194281/Ali-Fares-A047-654-200-BIA-Apr-30-2013",
  "https://www.scribd.com/document/140021454/Ricardo-Lozano-Albarez-A205-502-364-BIA-Nov-7-2012",
  "https://www.scribd.com/document/139810918/Rony-Rene-Anariba-A095-094-694-BIA-Nov-16-2012",
  "https://www.scribd.com/document/139807442/Dexter-Bernard-Richards-A098-582-406-BIA-Nov-16-2012",
  "https://www.scribd.com/document/139291434/Luis-A-Reyes-A094-391-611-BIA-Dec-8-2011",
  "https://www.scribd.com/document/139102074/Ji-Yoon-Chung-A089-125-468-BIA-Oct-17-2011",
  "https://www.scribd.com/document/139093981/Carlos-Antonio-Vargas-Pena-A040-524-349-BIA-Apr-29-2013",
  "https://www.scribd.com/document/138918847/Trinath-Chigurupati-A095-576-649-BIA-Oct-26-2011",
  "https://www.scribd.com/document/138786067/Keon-Richmond-A097-519-056-BIA-Mar-13-2012",
  "https://www.scribd.com/document/138691743/Federico-Contreras-Torres-A036-157-819-BIA-Jan-20-2012",
  "https://www.scribd.com/document/138688105/Michael-McFarlane-A012-043-018-BIA-Jan-18-2012",
  "https://www.scribd.com/document/138685486/Edgar-Castruita-Almaraz-A043-779-581-BIA-Jan-18-2012",
  "https://www.scribd.com/document/138683318/Ramiro-Enrique-Rojas-A056-123-018-BIA-Jan-17-2012",
  "https://www.scribd.com/document/138680672/Michael-Zacky-Dean-A031-314-515-BIA-Jan-12-2012",
  "https://www.scribd.com/doc/138300601/Jurate-Virgycio-Milutyte-A047-841-053-BIA-Apr-22-2013",
  "https://www.scribd.com/document/138125832/Andrey-M-Farafonov-A077-690-710-BIA-Dec-5-2011",
  "https://www.scribd.com/document/138089740/Bernard-Paget-Hensley-James-A041-792-398-BIA-Apr-18-2013",
  "https://www.scribd.com/doc/138088661/Guat-Ngoh-Lim-A055-549-920-BIA-Apr-22-2013",
  "https://www.scribd.com/document/137980875/Adayn-Gonzalez-Riol-A094-931-009-BIA-Apr-18-2013",
  "https://www.scribd.com/document/137815992/Christopher-Charles-Kerr-A044-857-956-BIA-Dec-15-2011",
  "https://www.scribd.com/document/137814011/Mahamadou-Cisse-A097-981-226-BIA-Oct-5-2012",
  "https://www.scribd.com/document/137676126/Sheldon-Odinga-Wilson-A042-163-324-BIA-Jan-17-2012",
  "https://www.scribd.com/document/137562292/Juan-de-de-Jesus-Perez-Agudelo-A012-340-064-BIA-Oct-9-2012",
  "https://www.scribd.com/document/137322593/William-Francisco-Ayala-Chan-A200-244-079-BIA-Jan-13-2012",
  "https://www.scribd.com/document/136712875/Abduljabar-Ahmed-Mazid-A043-308-397-BIA-Apr-12-2013",
  "https://www.scribd.com/document/136610281/Anderson-Ferreira-A099-982-743-BIA-Mar-18-2011",
  "https://www.scribd.com/doc/136610280/Anderson-Ferreira-A099-982-743-BIA-Aug-9-2011",
  "https://www.scribd.com/document/135988244/Hicham-Sadik-A096-680-844-BIA-Jun-8-2012",
  "https://www.scribd.com/document/135634867/Rigoberto-Avila-Santoyo-A088-920-938-BIA-Sept-23-2011",
  "https://www.scribd.com/document/135613749/Marcelo-Alejandro-Cao-A089-207-044-BIA-Jan-31-2012",
  "https://www.scribd.com/document/135169606/Javier-Torres-Ponce-A028-803-028-BIA-Apr-3-2012",
  "https://www.scribd.com/document/135166683/Ronei-Ferreira-De-Souza-A096-181-961-BIA-Mar-28-2013",
  "https://www.scribd.com/document/135034041/Dong-Van-Nguyen-A027-392-198-BIA-Jun-12-2012",
  "https://www.scribd.com/doc/133810609/Maria-de-Jesus-Ortiz-Mejia-A093-138-113-BIA-Mar-7-2013",
  "https://www.scribd.com/document/133809871/Maria-de-Jesus-Ortiz-Mejia-A093-138-113-BIA-Mar-7-2013",
  "https://www.scribd.com/doc/133805393/Roberto-Mayorga-Ponce-A087-326-405-BIA-Mar-25-2013",
  "https://www.scribd.com/document/133239281/Vanik-Armen-Voskanyan-A058-470-495-BIA-Mar-19-2013",
  "https://www.scribd.com/document/132145788/Charles-H-Bresiac-A091-402-355-BIA-March-15-2013",
  "https://www.scribd.com/document/131697220/Juan-Carlos-Soto-Hernandez-A076-395-660-BIA-Mar-15-2013",
  "https://www.scribd.com/document/131695887/Sayed-Gad-Omargharib-A093-002-513-BIA-Mar-18-2013",
  "https://www.scribd.com/document/130698575/Juan-Carlos-Guevara-Mata-A097-535-291-BIA-June-14-2011",
  "https://www.scribd.com/document/130696775/Edinzon-Fernando-Yangua-Calva-A088-190-231-BIA-Nov-28-2011",
  "https://www.scribd.com/document/130627739/Vikramjeet-Sidhu-A044-238-062-BIA-Nov-30-2011",
  "https://www.scribd.com/document/130580679/Pedro-Rivera-Rodriguez-A072-801-823-BIA-Sept-13-2012",
  "https://www.scribd.com/document/130579577/Joefry-F-Noble-A200-574-891-BIA-Sept-13-2012",
  "https://www.scribd.com/document/130578801/Fowobi-George-A071-996-819-BIA-Sept-14-2012",
  "https://www.scribd.com/document/130438370/Joy-Atila-Litaba-A200-224-355-BIA-Sept-11-2012",
  "https://www.scribd.com/document/130435390/Julian-Ricardo-Castillo-A041-590-059-BIA-Nov-6-2012",
  "https://www.scribd.com/document/130434084/Ramkelawan-Robin-Ramesh-A099-275-577-BIA-Dec-11-2012",
  "https://www.scribd.com/document/130432695/Miguel-Angel-Aguirre-Barron-A201-217-294-BIA-Nov-29-2012",
  "https://www.scribd.com/document/130335228/Mario-Nunez-Parra-A092-283-469-BIA-Nov-16-2012",
  "https://www.scribd.com/document/130334620/Nilesh-Hasmukhbhai-Ahir-A096-348-639-BIA-Nov-16-2012",
  "https://www.scribd.com/document/130334056/Bok-Kyu-Kim-A095-750-395-BIA-Nov-30-2012",
  "https://www.scribd.com/document/130333625/Jorge-Ernesto-Pastor-Iturralde-A094-910-236-BIA-Nov-30-2012",
  "https://www.scribd.com/document/130283391/Jose-Manuel-Beltran-Jr-A087-511-270-BIA-Nov-30-2012",
  "https://www.scribd.com/document/130277061/Mykola-Shchupak-A076-577-376-BIA-Nov-7-2012",
  "https://www.scribd.com/document/130275680/Fernando-Linares-Isidoro-A095-729-470-BIA-Nov-30-2012",
  "https://www.scribd.com/document/130271195/Bernard-Akwasi-Acheampong-A077-551-747-BIA-Oct-31-2012",
  "https://www.scribd.com/document/130269927/Alfonso-Zuniga-Aguilar-A077-312-584-BIA-Oct-31-2012",
  "https://www.scribd.com/document/130267035/Jimmy-Ampong-Libres-A045-232-847-BIA-Sept-28-2012",
  "https://www.scribd.com/document/130266875/Maria-Hernandez-Rodriguez-A089-566-220-BIA-Oct-25-2012",
  "https://www.scribd.com/document/130265168/Juan-Carlos-Delira-Mata-A095-691-856-BIA-Mar-5-2013",
  "https://www.scribd.com/document/130265170/Servando-Pinon-Ramos-A074-787-749-BIA-Feb-28-2013",
  "https://www.scribd.com/doc/130265169/Jane-Moiko-Baker-A200-303-818-BIA-Feb-28-2013",
  "https://www.scribd.com/document/129296961/Jesus-Loya-Terrazas-A010-544-171-BIA-Nov-26-2012",
  "https://www.scribd.com/document/129293940/Miklos-Szabo-A088-946-366-BIA-Feb-25-2013",
  "https://www.scribd.com/document/129293937/Santo-Suarez-A077-947-844-BIA-Feb-25-2013",
  "https://www.scribd.com/doc/129293936/Lorenzo-Gerardo-Flores-A205-025-072-BIA-Feb-25-2013",
  "https://www.scribd.com/document/129293938/Wilson-Njogu-Njiru-A096-207-034-BIA-Feb-25-2013",
  "https://www.scribd.com/document/129293939/Anthony-Olivo-Cosme-A042-271-032-BIA-Feb-27-2013",
  "https://www.scribd.com/document/127449452/Carlos-Alberto-Zambrano-A088-741-973-BIA-Feb-13-2013",
  "https://www.scribd.com/document/127449451/Fernando-Linares-Isidoro-A095-729-470-BIA-Feb-13-2013",
  "https://www.scribd.com/document/127445748/Ljuljija-Alimi-A044-440-041-BIA-Feb-14-2013",
  "https://www.scribd.com/document/127445747/Nowel-Q-Dela-Cruz-A057-056-093-BIA-Feb-8-2013",
  "https://www.scribd.com/document/127445746/Ziyaad-Saban-A070-623-999-BIA-Feb-8-2013",
  "https://www.scribd.com/document/127081532/Sunipha-Damouni-A029-235-328-BIA-Jan-10-2013",
  "https://www.scribd.com/document/127057833/Anthony-Afunwa-A096-129-725-BIA-Dec-4-2012",
  "https://www.scribd.com/document/127055723/Krissia-Rosibel-Paz-Avalos-A200-123-950-BIA-Dec-6-2012",
  "https://www.scribd.com/document/127054677/Julian-Dominguez-Tinoco-A027-094-320-BIA-Dec-7-2012",
  "https://www.scribd.com/document/127047481/Osama-Mikho-Oraha-A096-742-979-BIA-Dec-10-2012",
  "https://www.scribd.com/document/127044906/Juan-Ramon-Belliard-Tejada-A044-824-573-BIA-Dec-13-2012",
  "https://www.scribd.com/document/127044219/Adonay-Migueles-Alfaro-A205-291-592-BIA-Dec-13-2012",
  "https://www.scribd.com/document/127030786/Joao-Silva-Laudelino-A088-268-610-BIA-Dec-14-2012",
  "https://www.scribd.com/document/127028716/Patrick-Anthony-Brown-A043-400-049-BIA-Dec-18-2012",
  "https://www.scribd.com/document/127025496/Abel-Ramon-Amaya-A073-987-815-BIA-Dec-21-2012",
  "https://www.scribd.com/document/127015386/Jose-Jesus-Parra-Duarte-A078-263-341-BIA-Feb-11-2013",
  "https://www.scribd.com/document/127014318/Isela-Guadalupe-Pinto-Reyes-A041-623-357-BIA-Jan-31-2013",
  "https://www.scribd.com/document/127013678/Jose-Francisco-Mendoza-Carrillo-A037-723-177-BIA-Jan-31-2013",
  "https://www.scribd.com/document/127013010/Rufino-Marquez-Rocha-A201-073-660-BIA-Jan-29-2013",
  "https://www.scribd.com/document/127012317/Ramiro-Carbajal-Esparza-A087-269-262-BIA-Jan-24-2013",
  "https://www.scribd.com/document/127010570/Raul-Bosveli-Garcia-Perez-A076-894-739-BIA-Jan-18-2013",
  "https://www.scribd.com/document/126943421/Iftikhar-Qureshi-A097-756-487-BIA-Jan-18-2013",
  "https://www.scribd.com/document/125786820/Demetrio-Enrique-Lugo-Torres-A096-599-329-BIA-Jan-17-2013",
  "https://www.scribd.com/document/125784472/Mi-Hwang-Huh-A045-809-070-BIA-Jan-9-2013",
  "https://www.scribd.com/document/125722233/Noe-Cesar-Hernandez-Avila-A079-531-484-BIA-Aug-30-2012",
  "https://www.scribd.com/document/125709281/Keno-Bashae-Jumpp-A087-145-628-BIA-Jan-31-2013",
  "https://www.scribd.com/document/125708363/Ferino-Sanchez-Seltik-A094-216-526-BIA-Jan-14-2013",
  "https://www.scribd.com/document/125706558/Veronica-Wagithi-Mugo-A093-459-287-BIA-Jan-18-2013",
  "https://www.scribd.com/document/125483234/Noe-Cesar-Hernandez-Avila-A079-531-484-BIA-Jan-18-2013",
  "https://www.scribd.com/document/124711434/Board-of-Immigration-Appeals-BIA-Citation-Guidelines"
]